# <center>Part2 DeepSeekV3预训练流程

![](https://cdn.eso.org/images/original/science-stars-formation-banner1.jpg)

## 0 DeepSeekv3训练必备硬件与学习路线

预训练是大语言模型构建过程中非常核心的环节、曾经它是一支“高岭之花”、其高昂的成本、复杂的流程让大部分公司望而却步，但随着DeepSeek系列模型的推出、随着unsloth框架频频推出更低精度的量化版本模型、以及随着国产GPU和海外GPU不断加入更低精度训练和推理专属的计算模块、大模型训练的成本越来越低、构建私有模型、构建专属模型成为许多企业和政府机构的实际需求。

在开源模型众多、各类量化模型层出不穷的今天，为什么我们的课程还要耗费巨大经历、为大家研发预训练技术、甚至是走通DeepSeekv3的预训练全流程？这是因为——

1. <font color="green">**预训练能够弥补微调（Fine-tuning）或 RAG（Retrieval-Augmented Generation）的能力不足、为行业创造真正专业的“垂类模型”，随着训练成本越来越低，预训练技术的需求会逐渐增长**</font>。微调和RAG虽然能提升模型在特定任务上的表现，但它们都无法从根本上改变模型的知识储备和表示能力，在下面的情况中你会需要预训练、而不止微调——
> - **知识密集型任务**：例如医学诊断、法律咨询、金融分析等领域，单纯的微调无法让模型掌握足够的领域知识，而在预训练阶段加入大规模医学、法律等专业文献可以显著提升模型的基础能力、构造真正的“垂类模型”。
> - **新概念引入、知识灌注**：比如医疗AI模型需要理解新的病症、新的治疗方法，仅靠微调可能无法让模型真正掌握这些知识，需要在预训练过程中引入相关数据并进行大规模建模。
> - **推理能力的培养**：微调和 RAG 通常更偏向于“记忆”已有知识，而 复杂推理能力（如数学、逻辑推理、代码理解等）必须依赖预训练阶段的架构设计、数据分布、目标函数优化等。

2. <font color="green">**DeepSeekv3带来了众多关键预训练技术、理解这些技术对未来掌握大规模模型训练至关重要**</font>。DeepSeekv3的MTP（Multi-token Prediction）技巧、以及Tensor + Pipeline + ZeRO 1 Data + Experts 4种类型的并行都需要在预训练过程中实现、而这些技术未来或许成为预训练领域非常核心的根基技术。

3. <font color="green">**这是我们首次实现工业级预训练项目、从数据准备到预训练的全流程、我们都配备了让你能够拓展到100B模型的预训练流程供你学习**</font>。这份价值百万的经验与拓展代码保障了学习的实用性、它成为你落地项目、研究和求职过程中非常高价值的课程内容。

### 0.1 授课形式与必备前置基础

在之前的课程中、我们已经基于MateConv Mini详解了预训练过程中的各类流程以及优化流程、预训练课程本身已有14+小时内容，因此本次课程中涉及的预训练内容将不会再细致展开探讨训练原理、而是按照如下流程安排——

1. 4小时带伙伴们跑通DeepSeek预训练全套代码、中间不穿插详细原理详解
2. 2~4小时解读DeepSeekv3预训练中MTP、如何兼容多种并行等等创新点

因此、<font color="red">**如果你需要的是前4小时跑通代码的流程、那你几乎无需任何基础知识即可学习**</font>、只需按照我所设置的流程一步步运行代码即可。

<font color="red">**如果你的目标是彻底吃透DeepSeek预训练流程、那你在跑通代码的同时，还会需要之前我为DeepSeek课程所设置的一系列基础**</font>——

> **1 【Transformer】1～4、10～16**

如果你不理解参数超参数的区别、如果你不理解QKV是如何诞生、你不能轻松说出Decoder-only架构的训练和推理有什么区别、你不能理解生成模型的输出层是怎么输出文字的、那你需要学习Transformer内容。

> **2 【llama】1～2、4～6、8～9、11～14、17～19**

DeepSeek架构与我们实现的LlaMA架构高度相似、在LlaMA架构中我们详解了MOE混合专家模型、详解了KV缓存机制、详解了门控机制、这些都是DeepSeek模型的关键基础，掌握LLaMA架构能让你在学习deepseek时事半功倍！

> **3 【分布式预训练】0.1、0.2、1～4，12~后续全部内容**

在分布式预训练章节中，我们搭建了基于Llama + MOE架构自建的【对话模型MateConv】、其环境搭建、数据收集、训练流程和训练脚本将会在后续的DeepSeekv3课程和DeepSeekR1课程中发挥巨大作用，了解经典的预训练流程将会对你有很大的帮助！如果你有足够的时间，可以将整个分布式预训练流程看完。

> **4 【DeepSeekV3】全部已有直播内容**

在之前的DeepSeek课程中我们详解了DeepSeek的模型文件`deepseekv3_model.py`，这一脚本正是我们接下来要进行训练的基础脚本。因此掌握DeepSeekv3架构原理会对你后续进行预训练有较好的帮助。

---

### 0.2 所需硬件基础及Autodl租赁指南

我的设备为、AutoDL租赁的——

```python
####################################################
####   4 x 64 vCPU Intel(R) Xeon(R) Gold 6430   ####
####   RAM 4 x 120G                             ####
####   4 x RTX4090, VRAM 4 x 24G                ####
####   Hard Disk Drive 1000G                    ####
####################################################
```

该设备的成本越为8.2r/小时、AutoDL支持的最大卡位是1机8卡、因此如果你和我一样使用4090，那你的成本最多是17r/小时。硬件无上限、当前代码支持A800、A100、RTX6000等显卡、如果你有更好的设备，也欢迎使用。

**你所需要的最低硬盘要求与我一样是1000G，最次也要有6~700G，CPU和显卡要求是我的一半、这是能够运行多进程分布式的最低要求**。最低要求的租赁成本约为每小时4r，数据处理所需时长大约为60小时、在我所设置的超参数下里所需的训练时长大约也为60小时（5个epoch）。

```python
####################################################
####   2 x 32 vCPU 型号任选                      ####
####   RAM 2 x 80G                              ####
####   2 x RTX4090, VRAM 2 x 24G                ####
####   Hard Disk Drive 1000G                    ####
####################################################
```

你可以通过缩小模型尺寸、或者缩小所使用的数据尺寸来完成整个训练流程、这样可以大幅缩减你所需的数据处理和训练时间。另外，**也可选择按量计费中的无卡模式开机（每小时只需0.1r，硬盘依然可以使用）**，无卡模式下不会启动GPU、配备的cpu等级也比较低、因此价格十分便宜，你可以在这个模式下先确定所有代码都到位、可尝试在此基础上进行数据预处理、代码编写、上传下载文件等等占用时间的操作。

同时，在选择环境及基础配置时我选择的是 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/117.png)

这一选择下默认的python是12.3版本，但是cuda版本比较合适。

- <font color="red">**什么时候需要开始租赁设备呢？租设备流程在哪里？**</font>

建议是可以把课程听完、对整个流程有自己的认知后、再租赁设备、照着课件开始一步步进行运行。当然，如果你GPU预算十分充足、也可以跟着课程一起开启GPU。

租设备流程看【第二阶段 环境搭建与分布式预训练】中的这两节内容 ↓ 但需要注意的是，这两节内容中已对环境进行了一些配置，在我们进行deepseekv3预训练的时候我们会重新进行配置，因此你只需要按照这个流程租好设备、设置好final shell等流程即可，**无需follow之前课程中对环境进行的配置**。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/118.png)

到这个环节就好了 ↓ 无需配置requirements.txt。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/120.png)

---

## 1 DeepSeekv3 mini参数设置与技术选型

### 1.1 技术框架与训练框架选型

在如此低的硬件设备要求下，我们是如何实现工业中的训练流程的？在工业中，我们通常有以下三种模型规模与尺寸划分 ↓

| **模型规模**         | **1B~7B** | **7B~72B**  | **72B以上**  |
|----------------------|--------------------------------------|--------------------------------------|--------------------------------------|
| **数据 Token 量**    | **100B~750B tokens** | **750B ~ 10T tokens** | **10T~15T** |
| **数据存储存储需求**   | **400GB~3TB** | **3TB~30TB** | **30TB~45TB** |

在这样的规模划分下，通常所使用的技术框架有 ↓

| 训练流程 | 适用于1B-7B模型<br>（数据400G-3T）      | 适用于7B~72B模型<br>（数据3T-30TB）   | 备注 |
|----------------------|----------------------|----------------------|----------------------|
| **数据获取方式**      | 手动下载      |hfd+Aria2+git-lfs+jq+Ray<br>多线程并行拉取/分布式拉取  | --- |
 | **数据存储方式**  | 硬盘存储<br>允许是csv等格式 | 服务器存储、硬盘或NAS、S3、Ceph，存储格式jsonl/bin | 如果数据量更大、可以补充Apache Parquet + Hive Metastore框架|
| **数据处理框架**      | polars、json          | HuggingFaceDatasets、PyArrow、Json、使用PackedDataset等方式 | --- |
| **数据清理与去重**    | 正则匹配、长短过滤、人工检查          | DataJucier |---|
| **Tokenization** | BPE + tiktoken         | 通常自定义BPE Tokenizer| --- |
| **训练框架**         | PyTorch、Huggingface Transformers | 单机多卡用DeepSpeed、Megatron-LM、多机多卡用FSDP、Colossal-AI | --- |
| **训练并行策略**      | 无并行或数据并行 | 3D 并行（TP+PP+DP）+<br> EP + ZeRO系列并行 |---|
| **混合精度训练**      | 无（FP32）             | FP16 / BF16 / FP8<br>低精度优化（QLoRA） | --- |
| **优化器**           | AdamW               | 单机多卡 Adamw、多机多卡FSDP Optimizer |---  |
| **推理加速框架**     | 无                     | TensorRT-LLM、vLLM、Triton | ---  |
| **部署方式**         | 单机或ONNX Runtime 部署 | 多节点/多卡分布式推理<br>（TensorRT-LLM + vLLM）      | --- |
| **模型存储格式**     | PyTorch .pt或HF Transformers Format| Safetensors或GGUF | ---|

---
<br>
<center><b>本次在课程中，我们实现的是<font color = "red">单机多卡分布式、支持7~72B模型训练</font>的大型模型训练流程，我们实现的模型尺寸为<font color = "red">0.3B DeepSeekv3 mini，为支持MTP的自定义DeepSeekv3模型，</font>我们是所使用的数据规模为<font color = "red">200G原始数据</font>。我们使用的是<font color = "red">小尺寸模型 + 大训练框架的组合策略</font>、在保证大家能够学到行业实际训练流程的同时、尽量压缩所需的算力和训练成本。</b></center>
<br>

---

故而，**在本次预训练流程中、我们最终所使用的技术选型如下 ↓**

| 训练流程 | 适用于1B-7B模型<br>（数据400G-3T）      | 适用于7B~72B模型<br>（数据3T-30TB）   | 备注 |
|----------------------|----------------------|----------------------|----------------------|
| **数据获取方式**      | 手动下载      |<font color="red">**hfd+Aria2+git-lfs+jq+Ray<br>多线程并行拉取/分布式拉取**  | --- |
 | **数据存储方式**  | 硬盘存储<br>允许是csv等格式 | <font color="red">**服务器存储、硬盘**</font>或NAS、S3、Ceph<font color="red">**存储格式jsonl/bin**</font> | 如果数据量更大、可以补充Apache Parquet + Hive Metastore框架|
| **数据处理框架**      | polars、json          | <font color="red">**HuggingFaceDatasets、PyArrow、Json、使用PackedDataset等方式** | --- |
| **数据清理与去重**    | 正则匹配、长短过滤、人工检查          | <font color="red">**DataJucier** |---|
| **Tokenization** | BPE + tiktoken         | <font color="red">**DeepSeek开源tokenizer**| --- |
| **训练框架**         | PyTorch、Huggingface Transformers | <font color="red">**单机多卡用DeepSpeed**</font>、Megatron-LM、多机多卡用FSDP、Colossal-AI | Megatron不支持ZeRO-1并行因此最终还是选择DS，另外我们将额外提供一套FSDP代码供大家学习和参考 |
| **训练并行策略**      | 无并行或数据并行 | <font color="red">**3D 并行（TP+PP+DP）+<br> EP + ZeRO系列并行** |---|
| **混合精度训练**      | 无（FP32）             | <font color="red">**FP16 / BF16**</font> / FP8<br><font color="red">**各类低精度优化**</font> | --- |
| **优化器**           | AdamW               | <font color="red">**单机多卡 Adamw**</font>、多机多卡FSDP Optimizer |---  |
| **推理加速框架**     | 无                     | <font color="red">**TensorRT-LLM、vLLM、Triton** | ---  |
| **部署方式**         | 单机或ONNX Runtime 部署 | 多节点/<font color="red">**多卡分布式推理<br>TensorRT-LLM + vLLM**      | --- |
| **模型存储格式**     | PyTorch .pt或HF Transformers Format| <font color="red">**Safetensors或GGUF** | ---|

### 1.2 架构梳理与模型超参数设置

本次训练的DeepSeekV3架构是基于DeepSeekv3开源技术报告、以及DeepSeekV3在Huggingface页面所公开的推理模型所构建——

<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/73-.png" alt="image-20250107200502389" style="zoom:30%;" />

在课程《Part 1 DeepSeekv3深度原理与架构精讲》中我们已详细讲解过该架构。在这个架构中、我们实现了MLA、LoRA KV缓存、DeepSeekMOE（带辅助损失）、支持动态长文本的YARN、专家并行、张量并行、管道并行、ZeRO-1数据并行、以及MTP（Multi-Token Prediction）。受技术限制没能实现的是DeepSeekMOE改进的偏置免负载均衡技术、DualPipe、FP8量化处理以及PF8量化后的一系列精度提升操作。

最终我们实现的模型架构存储在文件`deepseekv3_mtp_model.py`中，且配备的参数如下 ↓

```python
@dataclass
class ModelArgs(PretrainedConfig):
    """
    0.3B - Data class for defining model arguments and hyperparameters.
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # 增加kwargs.get以兼容代码中的属性引用
        self.max_batch_size = kwargs.get("max_batch_size", 8)
        self.max_seq_len = kwargs.get("max_seq_len", 4096)
        self.dtype = kwargs.get("dtype", "bf16")
        self.vocab_size = kwargs.get("vocab_size", 128000) # 按照deepseek的tokenizer重新修改
        self.dim = kwargs.get("dim", 1024) 
        self.inter_dim = kwargs.get("inter_dim", 2048)  
        self.moe_inter_dim = kwargs.get("moe_inter_dim", 512) #MoE的inter_dim
        self.n_layers = kwargs.get("n_layers", 8) 
        self.n_dense_layers = kwargs.get("n_dense_layers", 1)
        self.n_heads = kwargs.get("n_heads", 8) 
        # MoE
        self.use_moe = kwargs.get("use_moe", 1)
        self.n_routed_experts = kwargs.get("n_routed_experts", 4) #必须是gpu的倍数 
        self.n_shared_experts = kwargs.get("n_shared_experts", 1)  
        self.n_activated_experts = kwargs.get("n_activated_experts", 2) 
        self.n_expert_groups = kwargs.get("n_expert_groups", 1)
        self.n_limited_groups = kwargs.get("n_limited_groups", 1)
        self.score_func = kwargs.get("score_func", "softmax")
        self.route_scale = kwargs.get("route_scale", 1.0)
        # MLA
        self.q_lora_rank = kwargs.get("q_lora_rank", 0)
        self.kv_lora_rank = kwargs.get("kv_lora_rank", 256)  
        self.qk_nope_head_dim = kwargs.get("qk_nope_head_dim", 64)  
        self.qk_rope_head_dim = kwargs.get("qk_rope_head_dim", 32)  
        self.v_head_dim = kwargs.get("v_head_dim", 64)
        # YARN
        self.original_seq_len = kwargs.get("original_seq_len", 4096)
        self.rope_theta = kwargs.get("rope_theta", 10000.0)
        self.rope_factor = kwargs.get("rope_factor", 40)
        self.beta_fast = kwargs.get("beta_fast", 32)
        self.beta_slow = kwargs.get("beta_slow", 1)
        self.mscale = kwargs.get("mscale", 1.0)
        # MTP
        self.n_mtp_depths = kwargs.get("n_mtp_depths", 2)
```

这是更多小伙伴可以跑的0.15B参数模型的配置 ↓

```python
class ModelArgs(PretrainedConfig):
    """
    0.15B - Data class for defining model arguments and hyperparameters.
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # 增加kwargs.get以兼容代码中的属性引用
        self.max_batch_size = kwargs.get("max_batch_size", 8)
        self.max_seq_len = kwargs.get("max_seq_len", 2048)
        self.dtype = kwargs.get("dtype", "bf16")
        self.vocab_size = kwargs.get("vocab_size", 128000) # 按照deepseek的tokenizer重新修改
        self.dim = kwargs.get("dim", 512) 
        self.inter_dim = kwargs.get("inter_dim", 1024)  
        self.moe_inter_dim = kwargs.get("moe_inter_dim", 256) #MoE的inter_dim
        self.n_layers = kwargs.get("n_layers", 4) 
        self.n_dense_layers = kwargs.get("n_dense_layers", 1)
        self.n_heads = kwargs.get("n_heads", 8)
        # MoE
        self.use_moe = kwargs.get("use_moe", 1)
        self.n_routed_experts = kwargs.get("n_routed_experts", 4) #必须是gpu的倍数 
        self.n_shared_experts = kwargs.get("n_shared_experts", 1)  
        self.n_activated_experts = kwargs.get("n_activated_experts", 2) 
        self.n_expert_groups = kwargs.get("n_expert_groups", 1)
        self.n_limited_groups = kwargs.get("n_limited_groups", 1)
        self.score_func = kwargs.get("score_func", "softmax")
        self.route_scale = kwargs.get("route_scale", 1.0)
        # MLA
        self.q_lora_rank = kwargs.get("q_lora_rank", 0)
        self.kv_lora_rank = kwargs.get("kv_lora_rank", 64)  
        self.qk_nope_head_dim = kwargs.get("qk_nope_head_dim", 64)  
        self.qk_rope_head_dim = kwargs.get("qk_rope_head_dim", 32)  
        self.v_head_dim = kwargs.get("v_head_dim", 64)
        # YARN
        self.original_seq_len = kwargs.get("original_seq_len", 2048)
        self.rope_theta = kwargs.get("rope_theta", 10000.0)
        self.rope_factor = kwargs.get("rope_factor", 40)
        self.beta_fast = kwargs.get("beta_fast", 32)
        self.beta_slow = kwargs.get("beta_slow", 1)
        self.mscale = kwargs.get("mscale", 1.0)
        # MTP
        self.n_mtp_depths = kwargs.get("n_mtp_depths", 1)
```
同时，我在`config.py`中提供了1B大小以及3B大小对应的参数，可供大家参考。需要注意的是，`config.py`中的参数无法直接导入、如果你需要使用特定模型的参数、则必须复制粘贴`config.py`中的类到模型文件中。

| **模型规模**         | **预估训练时间** |
|----------------------|--------------------------------------|
| **0.02B参数 + 30G数据 MateConvMini**    | 15个epoch，约18~22个小时，90分钟/epoch |
| **0.15B参数 + 60G数据 MiniDeepSeek**   | 15个epoch，约4天时间，6~7小时/epoch |
| **0.3B参数 + 100G数据 MiniDeepSeek**   | 10个epoch，约1周时间，14小时/epoch|
| **0.3B参数 + 300G数据 MiniDeepSeek**   | 10个epoch，约2周时间，30小时/epoch|

数据量直接关系到一个epoch中有多少需要处理的数据、因此数据量增加会让模型的训练时间线性增加。模型规模的增加就复杂得多，一般认为模型规模增加之后、训练时间不会线性增加、而会指数级 + 线性混合增加，因此模型变大之后所需的时间会爆炸性增长。

## 2 DeepSeekv3训练与数据处理环境搭建

- **数据预训练阶段所需全部文件**

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/119.png)

文件结构如下 ↓
```python
minideepseek/
├── requirements_minids.txt      # 预训练所需的环境
├── test_v3model.py              # 测试deepseev3_mtp_model.py能否跑通
├── logger_utils.txt             # 进行logger控制和调试的代码
├── pretrain.py                  # 预训练代码
│
├── model/                       # model文件夹、存放模型文件
│   ├── config.py
│   ├── convert.py
│   ├── deepseev3_mtp_model.py   # 要训练的模型脚本，加上了mtp
│   ├── fp8_cast_bf16.py
│   ├── kernel.py
│
├── data/                        # data文件夹
│   ├── print_datas.py           # 对下载好的数据进行状态打印的脚本
│   ├── analyze_jsonl.py         # 查询各jsonl文件统计指标的脚本
│   ├── data-juicer-main.zip     # data-juicer库安装所需文件
│   ├── delete_files.sh          # 删除数据、降低数据大小的脚本
│   ├── hfd_revised.sh           # 分布式并行数据下载脚本（防429报错版）
│   ├── slimpajama_preclean.py   # slimpajama清理脚本
│   ├── pdfprocess.py            # pdf批量处理流程
│   ├── step1_pretrain_basic_dataprocess.py   # 数据处理第一步：JSONL化脚本
│   ├── data_jucier/             # 在step2做所使用的数据处理工具
│   ├── final_data/              # 存放最终处理好的数据    
│   ├── step2/                                # 数据处理第二步：数据清洗脚本
│   │   ├── minidsv3_starcoder.yaml           # 各个数据自身的yaml文件
│   │   ├── minidsv3_text_ape.yaml
│   │   ├── minidsv3_text_openr1.yaml
│   │   ├── minidsv3_text_skypile.yaml
│   │   ├── minidsv3_text_slimpajama.yaml
│   │   ├── requirements_step2.txt           # 第二步运行所需的环境
│   │   ├── run_step2_ape.sh                 # 各个数据上运行DJ的sh脚本
│   │   ├── run_step2_openr1.sh
│   │   ├── run_step2_skypile.sh
│   │   ├── run_step2_slimpajama.sh
│   │   ├── run_step2_starcoder.sh
│
├── tokenizer/                  # tokenizer文件夹
│   ├── lit_gpt/                # 辅助打包的库lit_gpt
│   ├── test_tokenizer.py       # 测试tokenizer是否正常运行的代码
│   ├── tokenizer.json          # deepseek开源的tokenizer架构本身
│   ├── tokenizer_config.json   # deepseek开源的tokenizer配置文件
│   ├── step3_prepare_data_for_pretrain.py   # 数据处理第三步：Tokenizer脚本
│   ├── step3_prepare_data_for_pretrain_upsample.py   # 将数据在tokenizer环节进行上采样的步骤
│   ├── step4_dataset.py        # 将数据抽样、打包成bin文件的脚本

```

- **在服务器上创建虚拟环境**

DeepSeekV3的训练分为数据处理、预训练、后训练（SFT微调+RL）共4个流程，在执行这4个流程时我们可能会涉及大量的libraries，为了避免各类libraries之间的冲突、我们往往会针对特定的流程建立特定的环境。在本次训练过程中，我将建立如下虚拟环境 ↓

| 序号 | 环境名称                | 主要用途             | 依赖文件                 |
|----|----------------------|------------------|----------------------|
| 1  | `mnds_training`      | 预训练的主环境，对torch、triton、tiktoken版本都提出了特定要求        | `requirements_minids.txt` |
| 2  | `mnds_data_clean_step2_` | 在数据预处理中、datajucier所运行的环境，对python版本、transformers版本有特定要求      | `requirements_step2.txt`  |
| 3  | `mnds_sft`           | SFT 微调环境      | `requirements_sft.txt`    |
| 4  | `mnds_grpo`          | 强化微调环境，对trl、python、transformers都提出了要求  | `requirements_grpo.txt`   |

这些环境共同存在于服务器上、与root环境区分开来，当我们执行不同的任务时，我们需要在不同的环境之间切换。不过，所有这些环境都共享相同的硬盘和文件存储空间，因此无论你在哪个虚拟环境里，你都可以访问特定的数据和脚本文件、只是启用这些脚本的环境和python不同。

当你打开finalshell、已经进入你的服务器后，就可以开始部署环境了。本次我们先部署mnds_training和mnds_data_clean_step2两个环境 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/120.png)

- **基础训练环境创建**

```shell
# 创建minideepseek与v3两层目录
mkdir ~/autodl-tmp/minideepseek/
cd ~/autodl-tmp/minideepseek
mkdir ~/autodl-tmp/minideepseek/v3
cd ~/autodl-tmp/minideepseek/v3

# 建立虚拟环境mnds_training
# 用source激活该环境
# 该环境位于v3目录下，每次要激活该环境时必须先回到v3目录下
python3 -m venv mnds_training
source mnds_training/bin/activate
```

确认自己在v3目录中后、手动将本地目录中的data、model、requirements_miniminds、test_v3model等文件全部上传。注意，不要直接上传整个minideepseek文件夹、会破坏相应的环境。

```shell
# mnds_training环境配置
# 其中requirements_minids.txt在v3目录下
pip install --upgrade pip
pip install --upgrade pip setuptools wheel
pip install -r requirements_minids.txt -i https://pypi.tuna.tsinghua.edu.cn/simple

# 检验模型能否顺利运行
touch test_v3model.py
python test_v3model.py
```

得到如下结果后表示模型能够在当前环境下运行、还能够看到当前模型的总参数量 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/121_.png)

- **data_jucier专用数据预处理环境搭建**

```shell
# 先退出原本的mnds_training环境
# 回到~/autodl-tmp/minideepseek/v3
deactivate
cd ~/autodl-tmp/minideepseek/v3

# 确保你有python3.10的环境
sudo apt update
sudo apt install python3.10 python3.10-venv python3.10-dev

# 建立虚拟环境 - 数据清洗用环境mnds_data_clean_step2_
python3.10 -m venv mnds_data_clean_step2_
source mnds_data_clean_step2_/bin/activate

# 设置huggingface镜像
export HF_ENDPOINT=https://hf-mirror.com

# 确保你的data目录下已上传step2所有文件
# 配置mnds_data_clean_step2环境
pip install --upgrade pip
pip install --upgrade pip setuptools wheel
sudo apt-get install -y build-essential cmake

# 安装环境requirements_step2.txt
pip install -r ~/autodl-tmp/minideepseek/v3/data/step2/requirements_step2.txt -i https://pypi.tuna.tsinghua.edu.cn/simple

# 确保你的data/step2目录下已上传data-jucier-main.zip
# 配置data-jucier
cd ~/autodl-tmp/minideepseek/v3/data
unzip data-juicer-main.zip
mv data-juicer-main data-juicer
cd ~/autodl-tmp/minideepseek/v3/data/data-juicer
pip install -v -e .

# 建立存储数据新目录
mkdir -p ~/autodl-tmp/minideepseek/v3/data/deep_clean

# 建立缓存目录
mkdir /root/autodl-tmp/minideepseek/v3/data/data_jucier_cache
mkdir /root/autodl-tmp/minideepseek/v3/data/data_jucier_cache/temp

```

## 3 巨量数据构建与并行下载实战

### 3.1 DeepSeekV3预训练数据的组成

在DeepSeekv3构建过程中，我们使用了58%中文、21%英文及其他语言、15%代码、5%数学的方式进行构建，总数据集大小约为200G。关于数据的组合、DeepSeekV3本身是这样确定的 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/122.png)

为了降低训练成本，我们选择了在数据集打包时就已经经过一定清洗的数据、而没有选择完全没清洗过的数据。当然、这些数据还要再经过清洗流程才能够使用、但耗费的时间与人力会远远低于直接在raw data上进行清洗。

| 数据集编号 | 数据集名称                 | 数据属性 | 数据量级           | 存储格式         | 是否经过数据清洗 |
|-----------|----------------------------|--------------------------------|--------------------|------------------|------------------|
| 1         | Skypile-150B                | 中文文本                       | 117G/620GB             | JSONL            | 是               |
| 2         | SlimPajama                   | 以英文为主的混合语言文本        | 42G/1TB               | JSONL（压缩）    | 是               |
| 3         | Starcoder                    | 代码数据                        | 30GB/768GB    | Parquet          | 是               |
| 4         | OpenR1Math                   | 数学CoT数据                        | 8G+            | JSONL            | 是               |
| 5         | APE210K                      | 中文数学CoT数据                    | 49MB | JSONL            | 是               |
| *6         | FinPDFs                      | 从一系列数据分析报告/财报rawdata中提取出的数据                    | 50GB | 多模态PDFs            | 否               |

由于每个数据集的存储格式不同、因此数据集所占用的内存大小本身并不完全代表数据量的大小，但依照存储的格式来判断，通常来说数据量是 csv < Parquet < JSON < JSONL < Pyarrow Parquets）。由于尺寸不同、因此我们一般会参考数据所含的tokens总量。**但是在将数据转化成标准化的jsonl之前，我们很难确定单一jsonl文件所含的tokens总量是多少，我们甚至很难确认我们下载的文件本身的大小是多少**，因此在数据下载完成后，你可以在下载好的目录下方进行下面的命令、来查看你具体下载了多少数据。

```shell
cd /root/autodl-tmp/minideepseek/v3/data
du -sh */
```

---

### 3.2 Huggingface Token的申请

**首先要在数据页面进行email注册**（不一定所有数据都需要，但是starcoder需要）——

> ![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/17.png)

---

然后要进行权限申请，不仅要针对代码类数据进行特定的token申请，还需要针对普通数据进行下载类token的申请。

代码数据一般都是量非常大的数据、其特点不仅在于数据量大、还需要我们规定特定的编程语言进行下载，因此CLI代码会与传统文字数据集略有差别。首先，代码数据集的下载会需要权限、因此我们要先申请相应的access token——

1. **挂上梯子、注册Huggingface账号**👉https://huggingface.co/

2. **登录后、在个人profile页面找到自己的username**

> - 点击右上角——
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/51.png)

> - 复制username——
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/52.png)

3. **登录后、在这个页面下建立属于自己的AccessToken**👉https://huggingface.co/settings/tokens

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/53.png)

4. **在下面的页面中、建立token名字、并且为所需要的数据集申请权限**

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/54_.png)

点击create token、获得token后直接复制，**你将不会有第二次复制token的机会、因此务必要在这个时候复制token**。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/56.png)

token建好后，可以在token页面中看到、但此时你只能看到你的token名称、将不能再复制具体的access token码了。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/57.png)

---

### 3.3 预训练数据的多线程并行下载

在开始下载之前、请务必确保你的data文件夹中已经有了hfd_revised.sh文件，你需要手动上传它，它是经过修改后、专门用来防止429报错的脚本。

```shell
# 先退出虚拟环境
# 回到~/autodl-tmp/minideepseek/v3
# 激活训练环境，下载的数据和数据处理第一阶段都是在训练环境中运行
deactivate
cd ~/autodl-tmp/minideepseek/v3
source mnds_training/bin/activate

# 创建v3下方一系列目录
#mkdir ~/autodl-tmp/minideepseek/v3/model
#mkdir ~/autodl-tmp/minideepseek/v3/data

# 进入data目录、准备开始下载数据
cd ~/autodl-tmp/minideepseek/v3/data

# 设置镜像站
export HF_ENDPOINT=https://hf-mirror.com

#下载hfd.sh文件到当前的目录
wget https://hf-mirror.com/hfd/hfd.sh

#文件hfd.sh被赋予执行权限
chmod a+x hfd.sh                                    

#更新软件包索引
sudo apt update

#安装aria2c
sudo apt install aria2

#安装git-lfs
sudo apt install git-lfs 

#安装jq辅助下载
sudo apt install jq -y 

# 设置目录文件
mkdir -p ~/autodl-tmp/minideepseek/v3/data/skypile          
mkdir -p ~/autodl-tmp/minideepseek/v3/data/slimpajama
mkdir -p ~/autodl-tmp/minideepseek/v3/data/starcoder
mkdir -p ~/autodl-tmp/minideepseek/v3/data/openr1
mkdir -p ~/autodl-tmp/minideepseek/v3/data/ape210k

# 确保hfd_revised.sh在~/autodl-tmp/minideepseek/v3/data目录下
# 赋予hfd_revised.sh调用权限
chmod a+x hfd_revised.sh
```

#### 3.3.1 多线程并行下载脚本

- **skypile**：https://huggingface.co/datasets/Skywork/SkyPile-150B/

你可以用正则表达式选择你要下载的文件 ↓

```shell
# 只下载一部分、包括243G文件
# 注意更换你自己的hf token ip
#【Time Warning：4线程、5文件并行条件下3小时】
#【确保你的系统可以执行4线程、5文件并行拉取】
./hfd_revised.sh Skywork/SkyPile-150B --dataset \
  --include "data/2023-14_zh_middle_*.jsonl" \
  --include "data/2023-14_zh_head_*.jsonl" \
  --include "data/2023-06_zh_middle_*.jsonl" \
  --include "data/2023-06_zh_head_*.jsonl" \
  --include "data/2022-49_zh_middle_*.jsonl" \
  --include "data/2022-49_zh_head_*.jsonl" \
  --include "data/2022-40_zh_middle_*.jsonl" \
  --include "data/2022-40_zh_head_*.jsonl" \
  --include "data/2022-33_zh_middle_*.jsonl" \
  --include "data/2022-33_zh_head_*.jsonl" \
  --include "data/2022-27_zh_middle_*.jsonl" \
  --include "data/2022-27_zh_head_*.jsonl" \
  --include "data/2022-21_zh_middle_*.jsonl" \
  --include "data/2022-21_zh_head_*.jsonl" \
  --include "data/2022-05_zh_middle_*.jsonl" \
  --include "data/2022-05_zh_head_*.jsonl" \
  --hf_username TsaiTsai0929 \
  --hf_token hf_xxxxxx \
  --tool aria2c \
  -x 4 -j 5\
  --local-dir ~/autodl-tmp/minideepseek/v3/data/skypile
```

下载完毕之后目录显示为`/root/autodl-tmp/minideepseek/v3/data/skypile/data`，自带一层更深的data目录，具体数据为 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/124.png)

- **Slim Pajama**：https://huggingface.co/datasets/cerebras/SlimPajama-627B/

slimpajama单一数据太小、启用多线程或文件并行很快就会触发429报错、因此只能单一文件缓缓下载。

```shell
# 只下载一部分、包括178G文件
# 你可以根据自己的需求调整所需的目录
#【Time Warning：10小时】
./hfd_revised.sh cerebras/SlimPajama-627B --dataset \
  --include "train/chunk1/*" \
  --include "train/chunk2/*" \
  --include "train/chunk3/*" \
  --hf_username TsaiTsai0929 \
  --hf_token hf_xxxxxx \
  --tool aria2c \
  -x 1 -j 1 \
  --local-dir ~/autodl-tmp/minideepseek/v3/data/slimpajama
```

下载完毕之后目录显示为`/root/autodl-tmp/minideepseek/v3/data/slimpajama/train/chunk1`，自带更深的train/chunk1目录，具体数据为 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/125.png)

- **Starcoder**：https://huggingface.co/datasets/bigcode/starcoderdata

```shell
##########################################################
## 该数据下载之前、务必进行email注册、以及申请单独的token  ##
##########################################################

# 下载部分数据、选择了大约16种语言，大约150G
#【Time Warning：4线程、5并行下2小时】
#【确保你的系统可以执行4线程、5文件并行拉取】
./hfd_revised.sh bigcode/starcoderdata --dataset \
  --include "python/*" \
  --include "sql/*" \
  --include "matlab/*" \
  --include "javascript/*" \
  --include "java/*" \
  --include "json/*" \
  --include "c/*" \
  --include "rust/*" \
  --include "go/*" \
  --include "typescript/*" \
  --include "kotlin*" \
  --include "swift/*" \
  --include "julia/*" \
  --include "markdown/*" \
  --include "html/*" \
  --hf_username TsaiTsai0929 \
  --hf_token hf_xxxx \
  --tool aria2c \
  -x 4 -j 5 \
  --local-dir ~/autodl-tmp/minideepseek/v3/data/starcoder
```

下载完毕之后目录显示为`/root/autodl-tmp/minideepseek/v3/data/starcoder/`，目录下会有大量编程语言独立的文件夹，具体数据为 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/126.png)

在每个编程文件夹的内部，会看到parquet文件 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/127.png)

- **OpenR1-Math-220K**：https://huggingface.co/datasets/open-r1/OpenR1-Math-220k

```shell
#【Time Warning：4线程、5并行下5分钟】
./hfd_revised.sh open-r1/OpenR1-Math-220k --dataset \
  --include "all/*" \
  --hf_username TsaiTsai0929 \
  --hf_token hf_xxxx \
  --tool aria2c \
  -x 4 -j 5 \
  --local-dir ~/autodl-tmp/minideepseek/v3/data/openr1
```

下载完毕之后目录显示为`/root/autodl-tmp/minideepseek/v3/data/openr1/all`，自带更深一层的目录all，具体数据为 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/128.png)

- **APE220K**：https://huggingface.co/datasets/open-r1/OpenR1-Math-220k

```shell
#【Time Warning：10s】
./hfd_revised.sh MU-NLPC/Calc-ape210k --dataset \
  --include "data/*" \
  --hf_username TsaiTsai0929 \
  --hf_token hf_xxxx \
  --tool aria2c \
  -x 4 -j 5 \
  --local-dir ~/autodl-tmp/minideepseek/v3/data/ape210k
```

下载完毕之后目录显示为`/root/autodl-tmp/minideepseek/v3/data/ape210k/data`，自带更深一层的目录data，具体数据为 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/129.png)

#### 3.3.2 下载后数据量与比例控制

下载完成后、你可以通过下面的代码来控制实际数据量 ↓

```shell
# 进入data文件夹
cd ~/autodl-tmp/minideepseek/v3/data

# 查看这个目录下的一级文件夹所占内存大小
du -h --max-depth=1 ~/autodl-tmp/minideepseek/v3/data
```

这样你就可以看到 ↓ **由于我已经对数据进行了筛选，因此你刚下载时看到的数据大小会与我截图中不一致**。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/130.png)

如果你希望降低数据总量大小，或者调整数据比例，你可以使用`delete_files.sh`代码。这段代码可以随机删除目录下的文件，**一旦删除无法撤销、只能重新下载，因此务必谨慎**。在finalshell中点击文件、即可打开脚本，可以按照下面的方式修改，并按ctrl s进行保存。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/131.png)

当设置好你需要的目录后，运行下面的代码 ↓ 即可开始对特定目录进行按比例删除，以此来控制特定数据集的大小。当然，你也可以手动点进文件夹对数据进行删除、然后再查看数据相应的比例与大小 ↓

```shell
# 进入data文件夹
cd ~/autodl-tmp/minideepseek/v3/data

# 运行文件删除脚本
chmod a+x detele_files.sh
bash delete_files.sh

# 再查看数据的大小与比例
du -h --max-depth=1 ~/autodl-tmp/minideepseek/v3/data
```

#### 3.3.3 如何查看/读取特定数据？

在finalshell的linux环境中打开jupyter、运行`print_data.ipy`即可看到打印的结果（打开jupyter的过程参考九天老师在环境部署中所写的指南，要使用SSH隧道工具哦）↓

```shell
cd ~/autodl-tmp/minideepseek/v3
source mnds_training/bin/activate

# 创建Jupyter Kernel
pip install jupyterlab
pip install ipykepip install ipykernelrnel
python -m ipykernel install --user --name mnds_traning --display-name "Python (mnds_traning)"

# 启动jupyter lab
jupyter lab --allow-root
```

**（以下的代码粘贴自`print_data.ipy`文件，实际你需要在shell环境中开启jupyter去执行该代码才能看到和我一样的结果）**

- **skypile**

In [1]:
import json

# 指定 JSONL 文件路径
file_path = "/root/autodl-tmp/minideepseek/v3/data/skypile/data/2022-40_zh_head_0000.jsonl"

# 读取 JSONL 文件的第一行
with open(file_path, "r", encoding="utf-8") as file:
    first_line = file.readline().strip()  # 读取第一行并去掉首尾空格或换行符
    first_json = json.loads(first_line)  # 解析 JSON 格式

# 打印第一行内容
print(json.dumps(first_json, indent=4, ensure_ascii=False))  # 格式化打印 JSON 数据

{
    "text": "近日，由微软(中国)有限公司携手实验室合作伙伴张江集团打造的微软人工智能和物联网实验室，即微软第四个全球实验室在张江人工智能岛正式启用并投入运营。蜜度信息(媒体大数据专家)同ABB、泛亚汽车(大型跨国企业)等30家企业，一道成为实验室首批赋能企业。\n微软人工智能和物联网实验室，是微软为推动人工智能和物联网解决方案及应用的创新、研发和产业化而专门设立的全球性研发机构，旨在通过为企业和合作伙伴开发物联网产品和解决方案提供全方位支持，切实推动技术创新与制造、零售、医疗、金融、城市建设等行业数字化转型的深度融合和创新发展。\n据了解，包括蜜度信息在内的首批30家被微软人工智能和物联网实验室赋能的企业，将在未来3-6个月得到微软的全方位支持，包括实验室提供的硬件及软件资源、微软云服务资源、接入微软生态系统与大中小产业资源实现联动等，同时这些被赋能的企业也将享受张江集团提供的创新及投资服务支持。\n蜜度信息，作为新浪微博投资的媒体大数据公司，每日采集微博、微信、新闻、报刊、政务、外媒、博客、论坛、视频、网站、客户端等全网11大信息源，其中，且独家拥有新浪微博全量政务舆情数据。据统计，蜜度信息日新增1亿+条数据。基于海量数据，蜜度信息通过产、学、研合作的方式，与不同机构分别推出了网络传播热度指数、情绪地图、新媒体账号影响力排行榜等一系列颇有影响力的产品。而接下来，在微软人工智能和物联网实验室的加持之下，蜜度信息也将进一步发挥其在数据采集与自然语言处理上全快准稳的优势，成为国内媒体大数据应用领域的佼佼者。（完）\n"
}


- **starcoder**

In [6]:
import pandas as pd

# 指定 Parquet 文件路径
file_path = "/root/autodl-tmp/minideepseek/v3/data/starcoder/python/train-00003-of-00059.parquet"

# 读取 Parquet 文件
df = pd.read_parquet(file_path, engine="pyarrow")

# 设置 Pandas 选项，确保完整显示内容
pd.set_option("display.max_columns", None)  # 显示所有列
pd.set_option("display.max_colwidth", None)  # 显示所有单元格内容
pd.set_option("display.width", 1000)  # 设置输出宽度，防止换行

# 展示表格结构

df.columns

Index(['max_stars_repo_path', 'max_stars_repo_name', 'max_stars_count', 'id', 'content'], dtype='object')

In [7]:
# 显示第一行
df.loc[0,"content"]

"from functools import partial\nfrom utils.fmap_visualize import FeatureMapVis_siamese,FeatureMapVis\nfrom pathlib import Path\nfrom utils.fmap_visualize import show_tensor, show_img\nimport numpy as np\nimport os\nimport cv2\nimport torch\nimport torchvision\n#from models.Models_tmp import SiameseCGNet_v2\n\ndef forward(self, img, img_metas=None, return_loss=False, **kwargs):\n    outs = self.forward(img)\n    return outs\n\n\ndef create_featuremap_vis(model=None, use_gpu=True, init_shape=(768, 768, 3)):\n    #model.forward = partial(forward, model)\n    featurevis = FeatureMapVis_siamese(model, use_gpu)\n    featurevis.set_hook_style(init_shape[2], init_shape[:2])\n    return featurevis\n\n\ndef _show_save_data(featurevis, input, img_orig, feature_indexs, filepath, is_show, output_dir):\n    show_datas = []\n    for feature_index in feature_indexs:\n        feature_map = featurevis.run(input, feature_index=feature_index)[0]\n        data = show_tensor(feature_map[0], resize_hw=input.

In [8]:
# Ipython display来展示

from IPython.display import display, Code

code_str = df.loc[0,"content"]
display(Code(code_str, language="python"))

from functools import partial
from utils.fmap_visualize import FeatureMapVis_siamese,FeatureMapVis
from pathlib import Path
from utils.fmap_visualize import show_tensor, show_img
import numpy as np
import os
import cv2
import torch
import torchvision
#from models.Models_tmp import SiameseCGNet_v2

def forward(self, img, img_metas=None, return_loss=False, **kwargs):
    outs = self.forward(img)
    return outs


def create_featuremap_vis(model=None, use_gpu=True, init_shape=(768, 768, 3)):
    #model.forward = partial(forward, model)
    featurevis = FeatureMapVis_siamese(model, use_gpu)
    featurevis.set_hook_style(init_shape[2], init_shape[:2])
    return featurevis


def _show_save_data(featurevis, input, img_orig, feature_indexs, filepath, is_show, output_dir):
    show_datas = []
    for feature_index in feature_indexs:
        feature_map = featurevis.run(input, feature_index=feature_index)[0]
        data = show_tensor(feature_map[0], resize_hw=input.shape[:2], show_split=False, is_show=False)[0]
        am_data = cv2.addWeighted(data, 0.5, img_orig, 0.5, 0)
        show_datas.append(am_data)
    if is_show:
        show_img(show_datas)
    if output_dir is not None:
        filename = os.path.join(output_dir,
                                Path(filepath).name
                                )
        if len(show_datas) == 1:
            cv2.imwrite(show_datas[0], filename)
        else:
            for i in range(len(show_datas)):
                fname, suffix = os.path.splitext(filename)
                cv2.imwrite(fname + '_{}'.format(str(i)) + suffix,show_datas[i])

def _show_save_data_siamese(featurevis, input, img_orig, feature_indexs, fmaps = None):
    show_datas = []
    imgs = img_orig.detach().cpu().numpy().transpose([0, 2, 3, 1])
    if fmaps is None:
        for feature_index in feature_indexs:
            feature_maps = featurevis.run(input, feature_index=feature_index)[0]
            am_data = []
            for img,feature_map in zip(imgs,feature_maps[:3]):
                data = show_tensor(feature_map, resize_hw=input.shape[2:], show_split=False, is_show=False)[0]
                am_data.append(255-data)
            show_datas.append(am_data)
    else:
        for i in range(len(fmaps)):
            am_data = []
            for img, feature_map in zip(imgs, fmaps[i]):
                data = show_tensor(feature_map, resize_hw=input.shape[2:], show_split=False, is_show=False)[0]
                am_data.append(data)
            show_datas.append(am_data)
    return show_datas

def show_featuremap_from_imgs(featurevis, feature_indexs, img_dir, is_show, output_dir):
    if not isinstance(feature_indexs, (list, tuple)):
        feature_indexs = [feature_indexs]
    img_paths = [os.path.join(img_dir, x) for x in os.listdir(img_dir) if 'jpg' in x]
    for path in img_paths:
        img = cv2.imread(path)
        # 这里是输入模型前的图片处理
        input = img.astype(np.float32).copy()
        # 显示特征图
        _show_save_data(featurevis, input, img, feature_indexs, path, is_show, output_dir)



if __name__ == '__main__':
    img_dir = '.'
    out_dir = './out'

    init_shape = (1024, 1024, 3)  # 值不重要，只要前向一遍网络时候不报错即可
    feature_index = [32, 70, 96, 155]  # 可视化的层索引
    use_gpu = True
    is_show = False
    model = torchvision.models.resnet50(pretrained=True)  # 这里创建模型
    #model = torch.load(path)
    if use_gpu:
        model.cuda()
    featurevis = create_featuremap_vis(model, use_gpu, init_shape)
    show_featuremap_from_imgs(featurevis, feature_index, img_dir, is_show, out_dir)

- **slimpajama**

In [17]:
# 先解压
!zstd -d /root/autodl-tmp/minideepseek/v3/data/slimpajama/train/chunk1/example_train_0.jsonl.zst -o /root/autodl-tmp/minideepseek/v3/data/slimpajama/train/chunk1/example_train_0_.jsonl

/root/autodl-tmp/minideepseek/v3/data/slimpajama/train/chunk1/example_train_0.jsonl.zst: 45705517 bytes 


In [18]:
import json

output_file = "/root/autodl-tmp/minideepseek/v3/data/slimpajama/train/chunk1/example_train_0_.jsonl"

# 读取 `.jsonl` 文件的第一行
with open(output_file, "r", encoding="utf-8") as jsonl_file:
    first_line = jsonl_file.readline().strip()
    if first_line:
        try:
            json_data = json.loads(first_line)
            print("第一行 JSON 数据：", json.dumps(json_data, indent=4, ensure_ascii=False))
        except json.JSONDecodeError:
            print("❌ JSON 格式错误！原始数据：", first_line)
    else:
        print("❌ 读取失败，文件为空！")

第一行 JSON 数据： {
    "text": "J.J. Abrams Returns To Write And Direct 'Star Wars: Episode IX'\n09/12/2017 11:11 am ET Updated Sep 12, 2017\nThe return of the J.J.\nBy Bill Bradley\nUPDATE: 4:00 p.m. ET — In addition to the director news, \"Star Wars\" announced that the premiere date for \"Episode IX\" will be Dec. 20, 2019.\nStar Wars: Episode IX is scheduled for release on December 20, 2019. pic.twitter.com/rDBqmuHX89\n— Star Wars (@starwars) September 12, 2017\nThe Force was with J.J. Abrams when he launched the new set of \"Star Wars\" films with \"The Force Awakens,\" so now Disney is bringing him back.\nAs Deadline reported on Tuesday, and according to a press release on StarWars.com, Abrams will return to write and direct \"Star Wars: Episode IX.\" The statement reads:\nA post shared by Star Wars (@starwars) on Sep 12, 2017 at 7:28am PDT\nAfter Disney unexpectedly parted ways with former \"Episode IX\" director Colin Trevorrow earlier this month, rumors that Rian Johnson, who is d

- **openr1**

在huggingface页面上也可看到相应的结构 → https://huggingface.co/datasets/open-r1/OpenR1-Math-220k

In [19]:
import pandas as pd

file_path = "/root/autodl-tmp/minideepseek/v3/data/openr1/all/default-00002-of-00010.parquet"

try:
    # 读取第一行数据
    df = pd.read_parquet(file_path)
    
    if not df.empty:
        first_row = df.iloc[0]  # 取第一行数据
        print("第一行数据：\n", first_row.to_json(indent=4, force_ascii=False))
    else:
        print("❌ 读取失败，文件为空！")

except FileNotFoundError:
    print(f"❌ 文件未找到：{file_path}")
except Exception as e:
    print(f"❌ 发生错误：{e}")

第一行数据：
 {
    "problem":"3. As shown in the figure, the side lengths of $\\triangle A B C$ are $A B=14, B C$ $=16, A C=26, P$ is a point on the angle bisector $A D$ of $\\angle A$, and $B P \\perp A D, M$ is the midpoint of $B C$. Find the value of $P M$ $\\qquad$",
    "solution":"3. 6 .\n\nFrom the figure, take $B^{\\prime}$ on $A C$ such that $A B^{\\prime}=A B=14$, then $B^{\\prime} C=12$. Since $\\triangle A B B^{\\prime}$ is an isosceles triangle, we know that the intersection point of $B B^{\\prime}$ and $A D$ is $P$ (concurrency of five lines), so $P$ is the midpoint of $B B^{\\prime}$.",
    "answer":"6",
    "problem_type":"Geometry",
    "question_type":"math-word-problem",
    "source":"cn_contest",
    "uuid":"3fd1e4c6-8bd4-53d2-b40a-7c72360707e4",
    "is_reasoning_complete":[
        true,
        true
    ],
    "generations":[
        "<think>\nAlright, let's tackle this geometry problem. So, we have triangle ABC with sides AB = 14, BC = 16, and AC = 26. Point P is on 

- **ape210K**

In [21]:
import pandas as pd

# 指定 Parquet 文件路径
file_path = "/root/autodl-tmp/minideepseek/v3/data/ape210k/data/train-00000-of-00001-b9f022a8492442e4.parquet"

# 读取 Parquet 文件
df = pd.read_parquet(file_path, engine="pyarrow")

# 设置 Pandas 选项，确保完整显示内容
pd.set_option("display.max_columns", None)  # 显示所有列
pd.set_option("display.max_colwidth", None)  # 显示所有单元格内容
pd.set_option("display.width", 1000)  # 设置输出宽度，防止换行

# 显示第一行
df.head(1)

id                                                                                                                                                                                                                                                                                         question                                                                   question_chinese               chain result  result_float equation
0  ape210k__01099539  The fifth grade students participated in the voluntary book donation activity. Class Five 1 donated 500 books, Class Five 2 donated 80% of Class Five 1, Class Five 3 donated 120% of Class Five 2, Class Five 1 and Class Five 3 Donate more books than who? (Please compare the two methods).  五年级同学参加义务捐书活动，五1班捐了500本，五2班捐的本数是五1班80%，五3班捐的本数是五2班120%，五1班和五3班比谁捐书多？(请用两种方法比较一下)．  <result>1</result>      1           1.0      x=1

---

## 4 多模态PDF规模化处理实战

### 4.1 大模型预训练数据处理的一般流程

大模型所使用的预训练数据预处理流程主要包括 **数据收集、格式转换、清洗、质量控制、分词、合并拼接** 等多个步骤。

```
【step 1】📂 raw data (PDF/HTML/TXT)
            ↓  (转换)
【step 2】📂 JSONL {"text": "..."}
            ↓  (清洗)
          📂 cleaned JSONL
            ↓  (质量控制)
【step 3】📂 filtered JSONL
            ↓  (tokenizer)
          📂 Tokenized JSONL {"tokens": [...]}
            ↓  (拼接)
          📂 Pretrain Data (bin/lmdb)
```

原始数据大约是下面这些来源——

- 📄 **公开文本数据**：CC (Common Crawl)、Wikipedia、BooksCorpus
- 📜 **领域数据**：法律文档、医学论文、代码（如 GitHub）
- 📊 **企业内部数据**：客服对话、文档、邮件
- 📰 **网页爬取**：新闻网站、论坛（爬取后可能需要清理）
- 📚 **PDF 文档**：论文、书籍（需 OCR + 结构解析）

通常来说，如果我们转换格式并将文件进行存储——

- **原始格式（PDF, HTML, TXT, CSV） → JSONL**
  - PDF → JSONL: `pdfplumber` + `pymupdf` + `pytesseract`
  - HTML → JSONL: `BeautifulSoup` + `html2text`
  - CSV → JSONL: `pandas`
  - 爬虫: `scrapy` / `requests`

如果我们需要更深度的存储，则 ↓
```
📂 Raw data (PDF/HTML/TXT/CSV)
    ↓  (转换)
📂 JSONL {"text": "..."}
    ↓  (压缩存储)
📂 Parquet (列式存储)
```

---

- **数据清洗 (`DataJuicer` 预处理)**
    - 去除低质量数据（乱码、HTML 代码、重复数据）
    - 正则化文本（统一编码、去除异常符号）
    - 过滤敏感信息（PII/用户隐私、违禁词过滤）

- 工具：
    - **`DataJuicer`** 🧃：大规模数据清洗工具
    - `langdetect` / `fasttext`：过滤非目标语言
    - `ftfy`：修正乱码
    - `textacy` / `regex`：去除特殊符号
    - `deduplication`：去重（MinHash/SimHash）

- **质量控制 (`filtered data`)**
数据清洗完成后，还需要 **质量控制** 以确保高质量数据：
    - 关键词过滤（避免不相关或不健康内容）
    - 长度筛选（过短的文本无价值）
    - 可读性检测（自动计算 Flesch-Kincaid 指数）
    - 标注与审查（人工 or 规则标注）

---

- **Tokenizer 预处理**
    - JSONL → Tokenized Data
    - 拼接/填充/截断：形成长序列，减少 `padding`、添加 `bos` / `eos` token**
    - 生成 `train.bin` / `train.json` / `h5py` / `lmdb` 格式、或者生成

- **Dataloder或者PackedDataset预训练数据加载**

### 4.2 多模态Rawdata到JSONL标准化文件

数据分析1000+行业报告.zip ↓ <br>
链接: https://pan.baidu.com/s/1EPWpAbCVjaoUenEikPTbng?pwd=5x4c <br>
提取码: 5x4c <br>


| 组件 | 作用 | 安装命令 |
|------|------|---------|
| pdfplumber | 解析 PDF 文本 & 表格 | `pip install pdfplumber` |
| PyMuPDF (fitz) | 解析 PDF & 提取图片 | `pip install pymupdf` |
| Tesseract OCR | 识别图片文字 | `sudo apt install tesseract-ocr` |
| pytesseract | Tesseract 的 Python 绑定 | `pip install pytesseract` |
| 中文 OCR | 处理中文图片文字 | `sudo apt install tesseract-ocr-chi-sim` |

```shell

# 必须转向有python10的环境
# 如果你不希望重建环境、那可以使用mnds_data_clean_step2_
# 先退出原本的mnds_training环境
# 回到~/autodl-tmp/minideepseek/v3
deactivate
cd ~/autodl-tmp/minideepseek/v3
source mnds_data_clean_step2_/bin/activate

# 更新pip
pip install --upgrade pip
pip install --upgrade pip setuptools wheel
sudo apt-get install -y build-essential cmake

# 安装你处理pdf所需的所有库

pip install pdfplumber
pip install pymupdf
sudo apt install -y tesseract-ocr
pip install pytesseract
sudo apt install -y tesseract-ocr-chi-sim

# 验证工具是否安装成功
python

import pdfplumber
import fitz
import pytesseract

print("pdfplumber version:", pdfplumber.__version__)
print("PyMuPDF version:", fitz.__doc__)
print("Tesseract OCR version:", pytesseract.get_tesseract_version())

exit()

# 如果安装依赖出现问题、你可以手动安装下面的依赖
#pip install pillow
#pip install pdfminer.six

# 建立存储数据新目录
mkdir -p ~/autodl-tmp/minideepseek/v3/data/finPDF
mkdir -p ~/autodl-tmp/minideepseek/v3/data/finPDF/output_jsonl

# 将1000+行业报告直接上传至data目录

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/132.png)

```shell
# 将你的pdf压缩文件上传至/root/autodl-tmp/minideepseek/v3/data
# 先进行解压，我所提供的pdf文件为mac压缩的zip文件，编码模式为mac的GBK
unzip /root/autodl-tmp/minideepseek/v3/data/1000+行业报告.zip -d /root/autodl-tmp/minideepseek/v3/data/reports

# 删除由于mac解压带来的无用文件
# __MACOSX文件夹下的文件必须删除、否则会报错
find /root/autodl-tmp/minideepseek/v3/data/reports -name "__MACOSX" -exec rm -rf {} +
find /root/autodl-tmp/minideepseek/v3/data/reports -name "._*" -exec rm -rf {} +
```

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/133.png)

```shell
# 执行下面的代码，注意其中的done也是需要执行的
find /root/autodl-tmp/minideepseek/v3/data/reports/行业报告 -type f -name "*.zip" | while read zipfile; do
    unzip -o "$zipfile" -d /root/autodl-tmp/minideepseek/v3/data/finPDF
done
```

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/134.png)

最终得到的文件是 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/135.png)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/136.png)

**接下来可以开始运行pdfprocess.py了** ↓

```shell
cd /root/autodl-tmp/minideepseek/v3/data

# 上传pdfprocess.py文件到data文件夹下
# 开始执行，我设置的目录是/root/autodl-tmp/minideepseek/v3/data/finPDF/
# 你可以尝试设置到其中一个子目录来检测代码
# 【TIME WARNING：32线程并行、2小时】
touch pdfprocess.py
python3 pdfprocess.py
```

开始运行后你会看到 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/144.png)

多线程处理脚本解读——

```python
import os
import json
import pdfplumber
import fitz  # PyMuPDF
import pytesseract
from multiprocessing import Pool
from PIL import Image
from io import BytesIO
from tqdm import tqdm  # 进度条库
import pdfminer.pdfparser

# 统一的 JSONL 输出文件夹
JSONL_OUTPUT_DIR = "/root/autodl-tmp/minideepseek/v3/data/finPDF/output_jsonl"
os.makedirs(JSONL_OUTPUT_DIR, exist_ok=True)  # 确保目录存在

def extract_text_from_pdf(pdf_path):
    """提取 PDF 里的文本、表格、OCR 解析的图像文本"""
    extracted_data = {"file": pdf_path, "text": ""}

    try:
        # 用 pdfplumber 解析文本和表格
        with pdfplumber.open(pdf_path) as pdf:
            # 提取文本
            text_content = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())

            # 提取表格
            table_text = []
            for page in pdf.pages:
                tables = page.extract_tables()
                for table in tables:
                    if table:
                        # 处理表格中的 None 值
                        table_text.append("\n".join(
                            ["\t".join(str(cell) if cell is not None else "" for cell in row) for row in table]
                        ))

            # 合并文本
            extracted_data["text"] = text_content + "\n\n" + "\n".join(table_text)

        # 用 PyMuPDF (fitz) 解析图片
        doc = fitz.open(pdf_path)
        image_texts = []
        for page_index in range(len(doc)):
            images = doc[page_index].get_images(full=True)  # 获取当前页的所有图片
            for img_index, img in enumerate(images):
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                image = Image.open(BytesIO(image_bytes))

                # 用 OCR 识别图像中的文本
                ocr_text = pytesseract.image_to_string(image)
                if ocr_text.strip():
                    image_texts.append(f"Page {page_index + 1} Image {img_index + 1}: {ocr_text}")

        # 合并 OCR 文本
        extracted_data["text"] += "\n\n" + "\n".join(image_texts)

    except (pdfminer.pdfparser.PDFSyntaxError, ValueError, TypeError, Exception) as e:
        extracted_data["text"] = f"ERROR: Unable to read PDF ({str(e)})"
        print(f"⚠️ 无法解析 PDF: {pdf_path}，跳过。错误信息: {e}")

    return extracted_data

def process_folder(pdf_folder):
    """处理单个文件夹中的所有 PDF 并保存为 JSONL"""
    pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]
    if not pdf_files:
        return  # 如果当前文件夹没有 PDF，跳过处理

    folder_name = os.path.basename(pdf_folder)  # 获取文件夹名称
    output_file = os.path.join(JSONL_OUTPUT_DIR, f"{folder_name}.jsonl")  # JSONL 统一存放到 JSONL_OUTPUT_DIR

    print(f"📂 正在处理文件夹: {pdf_folder}，发现 {len(pdf_files)} 个 PDF 文件...")

    with Pool(processes=32) as pool:  # 32 个进程并行处理 PDF
        results = list(tqdm(pool.imap(extract_text_from_pdf, pdf_files), total=len(pdf_files), desc=f"🔄 处理中: {folder_name}"))

    # 统计失败的 PDF 文件
    failed_pdfs = [r["file"] for r in results if "ERROR" in r["text"]]

    # 保存 JSONL 格式文件
    with open(output_file, "w", encoding="utf-8") as f:
        for item in results:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")  # 确保支持中文

    print(f"✅ 文件夹 {folder_name} 处理完成，结果已保存至 {output_file}")

    # 记录失败的 PDF
    if failed_pdfs:
        error_log = os.path.join(JSONL_OUTPUT_DIR, f"{folder_name}_failed_pdfs.txt")
        with open(error_log, "w", encoding="utf-8") as f:
            for pdf in failed_pdfs:
                f.write(pdf + "\n")
        print(f"⚠️ 部分 PDF 解析失败，失败文件列表已保存到: {error_log}")

def main():
    root_folder = "/root/autodl-tmp/minideepseek/v3/data/finPDF/"
    
    # 遍历根目录下的所有子文件夹
    for subdir, _, _ in os.walk(root_folder):
        if subdir == root_folder:
            continue  # 跳过根目录自身，只处理子目录
        process_folder(subdir)  # 处理每个子文件夹

if __name__ == "__main__":
    main()
```

最终运行的结果如下 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/146.png)

点开其中一份jsonl文件如下 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/143.png)

## 5 数据处理第一步：巨量数据并行jsonl化实战

大模型所使用的预训练数据预处理流程主要包括 **数据收集、格式转换、清洗、质量控制、分词、合并拼接** 等多个步骤。

```
【step 1】📂 raw data (PDF/HTML/TXT)
            ↓  (转换)
【step 2】📂 JSONL {"text": "..."}
            ↓  (清洗)
          📂 cleaned JSONL
            ↓  (质量控制)
【step 3】📂 filtered JSONL
            ↓  (tokenizer)
          📂 Tokenized JSONL {"tokens": [...]}
            ↓  (拼接)
          📂 Pretrain Data (bin/lmdb)
```

- **数据预处理阶段所需全部文件**
（不包括pretrain脚本与SFT脚本，仅限数据预处理）

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/119.png)

文件结构如下 ↓
```python
minideepseek/
├── requirements_minids.txt      # 预训练所需的环境
├── test_v3model.py              # 测试deepseev3_mtp_model.py能否跑通
│
├── model/                       # model文件夹、存放模型文件
│   ├── config.py
│   ├── convert.py
│   ├── deepseev3_mtp_model.py   # 要训练的模型脚本，加上了mtp
│   ├── fp8_cast_bf16.py
│   ├── kernel.py
│
├── data/                        # data文件夹
│   ├── analyze_jsonl.py         # 查询各jsonl文件统计指标的脚本
│   ├── data-juicer-main.zip     # data-juicer库安装所需文件
│   ├── delete_files.sh          # 删除数据、降低数据大小的脚本
│   ├── hfd_revised.sh           # 分布式并行数据下载脚本（防429报错版）
│   ├── slimpajama_preclean.py   # slimpajama清理脚本
│   ├── pdfprocess.py            # pdf批量处理流程
│   ├── print_data.ipy           # 查看数据状态的ipy
│   ├── step1_pretrain_basic_dataprocess.py   # 数据处理第一步：JSONL化脚本
│   ├── step2/                                # 数据处理第二步：数据清洗脚本
│   │   ├── minidsv3_starcoder.yaml           # 各个数据自身的yaml文件
│   │   ├── minidsv3_text_ape.yaml
│   │   ├── minidsv3_text_openr1.yaml
│   │   ├── minidsv3_text_skypile.yaml
│   │   ├── minidsv3_text_slimpajama.yaml
│   │   ├── requirements_step2.txt           # 第二步运行所需的环境
│   │   ├── run_step2_ape.sh                 # 各个数据上运行DJ的sh脚本
│   │   ├── run_step2_openr1.sh
│   │   ├── run_step2_skypile.sh
│   │   ├── run_step2_slimpajama.sh
│   │   ├── run_step2_starcoder.sh
│   ├── step3_prepare_data_for_pretrain.py   # 数据处理第三步：Tokenizer脚本
│
├── tokenizer/                  # tokenizer文件夹
│   ├── test_tokenizer.py       # 测试tokenizer是否正常运行的代码
│   ├── tokenizer.json          # deepseek开源的tokenizer架构本身
│   ├── tokenizer_config.json   # deepseek开源的tokenizer配置文件
│
```

### 5.1 数据预处理第一阶段执行脚本

```shell
# 依然使用训练环境
# 回到建立训练环境的目录、激活训练环境
cd ~/autodl-tmp/minideepseek/v3
source mnds_training/bin/activate

# 环境补完
pip install zstandard
pip install pyarrow

# 建立存储数据的新目录
mkdir -p ~/autodl-tmp/minideepseek/v3/data/basic_clean

cd ~/autodl-tmp/minideepseek/v3/data

# 测试模式，max_line = 5
# 【TIME WARNING：15分钟】
python step1_pretrain_basic_dataprocess.py     

# 完整模式
#【TIME WARNING: 6小时】
touch step1_pretrain_basic_dataprocess.py
python step1_pretrain_basic_dataprocess.py main

# 针对过于零碎的slim_pajama做预清洗
# 【TIME WARNING】
# 8进程并行、64G内存支持下约25分钟
touch slimpajama_preclean.py
python slimpajama_preclean.py

# 建立data层的缓存目录temp
mkdir /root/autodl-tmp/minideepseek/v3/data/temp

# 把原本的slimpajama文件
mv /root/autodl-tmp/minideepseek/v3/data/basic_clean/processed_slimpajama.jsonl /root/autodl-tmp/minideepseek/v3/data/temp
```

在这一阶段，我们的目标是将所有的数据都转化为标准JSONL格式 ↓ 即每个样本中只有一个键值对、键的内容为"text"、值的内容为当前样本所包含的所有文字——
```python
{"text":"xxxx"}
```

有的数据可能天然就遵循这样的格式，例如skypile ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/137.png)

**但是大部分数据都不可能遵循这样的格式、相反，大部分数据都有自己特殊的结构** ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/138.png)
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/139.png)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/140.png)


对于OpenR1-Math，在huggingface页面上也可看到相应的结构 → https://huggingface.co/datasets/open-r1/OpenR1-Math-220k

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/141.png)
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/142.png)

因此我采取的方案是 ↓

| **数据集**       | **输入格式**            | **处理步骤** | **基础去重** | **基础质量检查** | **输出格式** |
|----------------|---------------------|-----------|-------|--------|--------|
| **Skypile**    | `.jsonl`            | 解析 JSON，提取 `"text"` 字段 | ✅ | ✅ | `.jsonl` |
| **Slimpajama** | `.jsonl.zst`        | 解压 Zstd，解析 JSON，提取 `"text"` | ✅ | ✅ | `.jsonl` |
| **OpenR1**     | `.parquet`          | 提取 `problem_type`、`problem`、`solution`、`answer`、`generations` | ✅ | ✅ | `.jsonl` |
| **APE210K**    | `.parquet`          | 提取 `question_chinese`、`chain`、`equation`、`result` | ✅ | ✅ | `.jsonl` |
| **Starcoder**  | `.parquet`          | 提取 `content`，筛选 `max_stars_count` | ✅ | ✅ | `.jsonl` |

### 5.2 数据预处理第一阶段脚本解读

- **`FormatHandler`（基础格式处理类）**
| **方法名** | **功能解析** |
|-----------|-------------|
| `__init__()` | 初始化 `FormatHandler`，定义输入路径、输出路径、数据集名称，并创建一个去重的 `set`。 |
| `get_file_list()` | 递归获取 `input_path` 目录下的 `.jsonl` 和 `.parquet` 文件列表。 |
| `process_one_line(line, fout)` | 处理单行数据（抽象方法，需要子类实现）。 |
| `process_one_file(file_path, max_lines=None)` | 处理单个文件，逐行读取并调用 `process_one_line()`，去重并统计处理的行数。 |
| `process_all(max_lines=None)` | 处理 `input_path` 目录下的所有文件，调用 `process_one_file()` 并统计总处理时间。 |
| `quality_assurance(line)` | 进行文本质量检查，过滤短文本和换行符过多的文本。 |
| `quality_assurance_math(line)` | 数学数据的质量检查，确保文本长度大于 10。 |
| `zh_process(line)` | 处理中文文本，进行 Unicode 规范化、移除控制字符、格式化换行符等操作。 |

---

- **`SkypileFormatHandler`（Skypile 数据集处理类，继承 `FormatHandler`）**
| **方法名** | **功能解析** |
|-----------|-------------|
| `__init__()` | 初始化 `SkypileFormatHandler`，继承 `FormatHandler` 并初始化去重 `set`。 |
| `get_file_list()` | 递归获取 `input_path` 目录下的 `.jsonl` 文件列表。 |
| `process_one_line(line, fout)` | 解析 `.jsonl` 数据，提取 `"text"` 字段，去重后写入 `fout`。 |
| `process_one_file(file_path, max_lines=None)` | 逐行读取 `.jsonl` 文件，调用 `process_one_line()` 处理文本，统计有效行和无效行。 |
| `process_all(max_lines=None)` | 处理 `input_path` 目录下所有 `.jsonl` 文件，并统计总处理时间。 |

---

- **`SlimpajamaFormatHandler`（Slimpajama 数据集处理类）**
| **方法名** | **功能解析** |
|-----------|-------------|
| `__init__()` | 初始化 `SlimpajamaFormatHandler`，继承 `FormatHandler`。 |
| `get_file_list()` | 递归获取 `input_path` 目录下的 `.jsonl.zst` 文件列表。 |
| `process_one_file(file_path, max_lines=None)` | 读取 `.jsonl.zst`（Zstandard 压缩）文件，解压后逐行处理。 |
| `process_one_line(line, fout)` | 解析 `.jsonl` 数据，提取 `"text"` 字段，质量检查后写入 `fout`。 |

---

- **`StarcoderFormatHandler`（Starcoder 代码数据集处理类）**
| **方法名** | **功能解析** |
|-----------|-------------|
| `__init__()` | 初始化 `StarcoderFormatHandler`，继承 `FormatHandler`，支持语言过滤。 |
| `get_file_list()` | 获取 `.parquet` 文件列表，支持按编程语言分类筛选。 |
| `process_one_file(file_path, max_lines=None)` | 读取 `.parquet` 文件，解析 `content` 字段，进行代码质量检查。 |

---

- **`OpenR1FormatHandler`（OpenR1 数据集处理类）**
| **方法名** | **功能解析** |
|-----------|-------------|
| `__init__()` | 初始化 `OpenR1FormatHandler`，继承 `FormatHandler`。 |
| `get_file_list()` | 递归获取 `input_path` 目录下的 `.parquet` 文件列表。 |
| `process_one_file(file_path, max_lines=None)` | 读取 `.parquet` 文件，提取 `problem`、`solution`、`answer` 等字段，进行质量检查和数据清洗。 |

---

- **`APE210KFormatHandler`（APE210K 数学数据集处理类）**
| **方法名** | **功能解析** |
|-----------|-------------|
| `__init__()` | 初始化 `APE210KFormatHandler`，继承 `FormatHandler`。 |
| `process_one_file(file_path, max_lines=None)` | 读取 `.parquet` 文件，提取 `question_chinese`、`equation`、`result` 等字段，进行格式化处理。 |
| `process_one_line(row, fout)` | 解析单行数据，将数学题目、解题步骤、最终答案合并为格式化文本并写入 `fout`。 |

---

- **`test_run()`（测试模式）**
| **方法名** | **功能解析** |
|-----------|-------------|
| `test_run()` | 在测试模式下运行多个数据集的 `process_all()`，并对每个数据集生成 `.jsonl` 文件。 |

---

- **`main_run()`（正式模式）**
| **方法名** | **功能解析** |
|-----------|-------------|
| `main_run()` | 在正式模式下运行 `process_all()` 处理 OpenR1 数据集，并生成 `.jsonl`。 |

---

- **`if __name__ == "__main__":`**
| **方法名** | **功能解析** |
|-----------|-------------|
| `if __name__ == "__main__":` | 解析命令行参数，决定运行 `test_run()` 还是 `main_run()`，默认执行测试模式。 |

---

- `FormatHandler` 是所有格式处理器的 **基类**，提供通用功能（文件获取、去重、质量检查）。
- 各子类（如 `SkypileFormatHandler`、`SlimpajamaFormatHandler`）继承 `FormatHandler`，并根据特定数据集的格式实现 `process_one_line()`。
- **正式模式 `main_run()` 主要用于生产环境**，只处理 OpenR1 数据集。
- **测试模式 `test_run()` 适用于调试**，可快速验证多个数据集的处理逻辑。

## 6 数据处理第二步：data-jucier数据并行化清洗实战

Data-Juicer 是一个 **数据处理与清洗** 的框架，专门用于 **大规模数据预处理**，特别适用于 **大语言模型（LLM）预训练** 数据管道。它提供了一整套 **高效、可配置** 的 **Mapper（映射）、Filter（过滤）、Deduplicator（去重）** 操作，使得数据清理变得 **模块化、自动化、高效**。

### 6.1 Datajucier的执行脚本

首次进入第二阶段 ↓ 此时你应该已经部署好了Datajucier所需要的一切环境，因此可以开始直接执行step2的代码了 ↓

```shell
# 先退出原本的mnds_training环境
# 回到~/autodl-tmp/minideepseek/v3
deactivate
cd ~/autodl-tmp/minideepseek/v3
source mnds_data_clean_step2_/bin/activate

# 建立存储数据新目录
mkdir -p ~/autodl-tmp/minideepseek/v3/data/deep_clean
mkdir -p ~/autodl-tmp/minideepseek/v3/data/deep_clean/djed_ape210k
mkdir -p ~/autodl-tmp/minideepseek/v3/data/deep_clean/djed_openr1
mkdir -p ~/autodl-tmp/minideepseek/v3/data/deep_clean/djed_skypile
mkdir -p ~/autodl-tmp/minideepseek/v3/data/deep_clean/djed_slimpajama
mkdir -p ~/autodl-tmp/minideepseek/v3/data/deep_clean/djed_starcoder

# 建立一个单独的目录来存放原来的processed_slimpajama.jsonl
mkdir /root/autodl-tmp/minideepseek/v3/data/temp
mv /root/autodl-tmp/minideepseek/v3/data/basic_clean/processed_slimpajama.jsonl /root/autodl-tmp/minideepseek/v3/data/temp

# 进入data-jucier的脚本所在的目录
cd /root/autodl-tmp/minideepseek/v3/data/step2

# 运行下列代码、将jsonl分割成100~500MB的小文件
# 更小的文件类似于PackedDataset的方法
# 有助于提升data_jucier的并行化速度
python split_jsonl.py

###################################################
####   4 x 64 vCPU Intel(R) Xeon(R) Gold 6430  ####
####   RAM 4 x 120G，4 x RTX4090 24G           ####
###################################################

# 【TIME WARNING：120线程并行，约4小时】
# python analyze_jsonl.py /root/autodl-tmp/minideepseek/v3/data/basic_clean

# 检验data-jucier能否顺利运行
# 【TIME WARNING：8进程并行，约5分钟】
cd ~/autodl-tmp/minideepseek/v3/data/step2
chmod +x run_step2_ape.sh
bash run_step2_ape.sh

# 【TIME WARNING：8进程并行，约15分钟】
chmod +x run_step2_openr1.sh
bash run_step2_openr1.sh

# 【TIME WARNING：32进程并行、约3小时】
# 中间还需额外下载一些筛选用模型、务必设置hf镜像站的环境变量
chmod +x run_step2_skypile.sh
bash run_step2_skypile.sh

# 【TIME WARNING：32进程并行、约3小时】
# 中间还需额外下载一些筛选用模型、务必设置hf镜像站的环境变量
chmod +x run_step2_slimpajama.sh
bash run_step2_slimpajama.sh

# 【TIME WARNING：32进程并行、约3小时】
chmod +x run_step2_starcoder.sh
bash run_step2_starcoder.sh

# 执行完成后、可以通过ctrl+C返回CLI命令行
```

在执行时会看到下面的打印结果 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/145.png)

中断执行时，需要走下面的流程才能彻底释放显存 ↓ 否则显存会持续被占用。

```shell
source ~/.bashrc

pkill -9 python

rm -rf /root/autodl-tmp/minideepseek/v3/data/data_jucier_cache
rm -rf /root/autodl-tmp/minideepseek/v3/data/data_jucier_cache/temp
```

执行完毕后、你看到的输出结果为 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/148.png)

点进文件夹后、会看到大约按照300M和100M分配的数据 ↓ **在每个文件夹中，你需要删除排在第一位的stats.jsonl文件夹，为后续的处理做准备**。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/150.png)

### 6.2 data-jucier可完成的预处理

| 算子类型       | 算子名称        | 功能描述 |
|--------------|--------------|---------|
| **Mapper**   | `Mapper`     | 主要用于数据编辑和转换，对输入样本进行修改或增强 |
| **Filter**   | `Filter`     | 主要用于数据过滤，根据统计信息或规则删除不符合条件的数据 |
| **Deduplicator** | `Deduplicator` | 主要用于数据去重，利用哈希等方法消除重复样本 |
| **Selector** | `Selector`   | 主要用于数据选择，从数据集中筛选符合特定条件的子集 |
| **Grouper**  | `Grouper`    | 主要用于数据分组，将相关样本组合在一起 |
| **Aggregator** | `Aggregator` | 主要用于数据聚合，对分组后的数据进行合并或统计 |

更多预处理流程见附录。

**YAML 配置文件** 主要用于 Data-Juicer 框架，它 **定义了数据处理的全流程**，包括：
1. **数据源路径**（`dataset_path`）
2. **数据输出路径**（`export_path`）
3. **数据清洗步骤（`process` 下的所有操作）**：
   - **文本格式清理**
   - **字符清理**
   - **去除重复句子**
   - **长度过滤**
   - **相似度去重**
   - **句子结构修正**
   - **文本分词 & 统计**
4. **并行处理参数**（`np`）

---

我们设置了2种不同**初始算子组合**、并在数据筛选流程中不断改写其中的细节，我们的初始算子分别是：

> - **文本数据清洗算子**<br><br>
和官方推荐的文本清洗算子相比、我们删除了`fix_unicode_mapper`（修复 Unicode 编码错误）以及`flagged_words_filter`（过滤包含特定敏感词的文本），对于已经过整理和清洗的数据而言、无需再使用前两个修复错误的算子。另外、针对中文数据、我们删除了`remove_repeat_sentences_mapper`，这个算子大多用于英文、并且对于已经清洗好的中文数据我们也不必再使用。针对英文数据，我们还删除了`chinese_convert_mapper`（简繁体转换，专用于中文）这一专用于中文字的算子。<br><br>
| 算子名称                          | 功能描述                           | 可配置参数 |
|--------------------------------|--------------------------------|---------|
| `chinese_convert_mapper`      | 转换简繁体和日文汉字、**仅限中文数据**              | `mode`: 选择转换模式（如 `t2s`: 繁体转简体） |
| `clean_email_mapper`         | 删除邮件地址                      | 无 |
| `clean_html_mapper`          | 删除 HTML 代码                   | 无 |
| `clean_ip_mapper`            | 删除 IP 地址                      | 无 |
| `clean_links_mapper`         | 删除网页链接                      | 无 |
| `clean_copyright_mapper`     | 删除版权声明                      | 无 |
| `expand_macro_mapper`        | 展开 LaTeX 宏定义                 | 无 |
| `punctuation_normalization_mapper` | 标准化标点符号                | 无 |
| `remove_repeat_sentences_mapper` | 去除重复句子、**仅限英文数据**                  | `lowercase`: 是否忽略大小写, `ignore_special_character`: 是否忽略特殊字符, `min_repeat_sentence_length`: 最小重复句子长度 |
| `remove_specific_chars_mapper` | 移除特定字符                   | `chars_to_remove`: 指定要删除的字符列表 |
| `whitespace_normalization_mapper` | 标准化空白字符                | 无 |
| `alphanumeric_filter`        | 按字母/数字比例过滤文本            | `min_ratio`: 最小比例, `max_ratio`: 最大比例, `tokenization`: 是否基于 token 计算 |
| `average_line_length_filter` | 按平均行长度过滤文本              | `min_len`: 最小长度, `max_len`: 最大长度 |
| `character_repetition_filter` | 按字符重复率过滤文本              | `min_ratio`: 最小比例, `max_ratio`: 最大比例 |
| `maximum_line_length_filter` | 按最长行长度过滤文本              | `max_len`: 最大长度 |
| `perplexity_filter`          | 按困惑度（Perplexity）过滤文本     | `lang`: 语言, `max_ppl`: 最大困惑度 |
| `special_characters_filter`  | 按特殊字符比例过滤文本            | `min_ratio`: 最小比例, `max_ratio`: 最大比例 |
| `text_length_filter`         | 按文本长度过滤                   | `min_len`: 最小长度, `max_len`: 最大长度 |
| `words_num_filter`           | 按单词数量过滤文本                | `lang`: 语言, `min_num`: 最小数量, `max_num`: 最大数量, `tokenization`: 是否使用 token |
| `word_repetition_filter`     | 按单词重复率过滤文本              | `lang`: 语言, `min_ratio`: 最小比例, `max_ratio`: 最大比例, `tokenization`: 是否使用 token |
| `document_simhash_deduplicator` | 基于 SimHash-LSH 进行去重 | `tokenization`: 分词方式（space, punctuation, character）, `window_size`: 窗口大小, `num_blocks`: 哈希块数, `hamming_distance`: 最大汉明距离, `lowercase`: 是否转换为小写 |

> - **代码/数学数据清洗算子**<br><br>
代码算子会更偏向于结构化、规则化匹配，尽量创造格式化数据的结构、完成字符、字词级别过滤等等。<br><br>
| 算子名称                            | 功能描述 |
|----------------------------------|-------------------------------------------|
| `clean_copyright_mapper`        | 删除代码文件开头的版权声明（必须包含单词 `copyright`） |
| `clean_email_mapper`            | 删除邮箱信息 |
| `clean_links_mapper`            | 删除链接，例如以 `http` 或 `ftp` 开头的链接 |
| `fix_unicode_mapper`            | 修复损坏的 Unicode（借助 `ftfy` 库） |
| `punctuation_normalization_mapper` | 将各种 Unicode 标点符号标准化为其 ASCII 等效项 |
| `alphanumeric_filter`           | 仅保留字母数字比例在指定范围内的样本 |
| `average_line_length_filter`    | 仅保留平均行长度在指定范围内的样本 |
| `character_repetition_filter`   | 仅保留字符重复率（char-level n-gram）在指定范围内的样本 |
| `maximum_line_length_filter`    | 仅保留最大行长度在指定范围内的样本 |
| `text_length_filter`            | 仅保留文本长度在指定范围内的样本 |
| `word_num_filter`               | 仅保留字数在指定范围内的样本 |
| `word_repetition_filter`        | 仅保留词重复率（word-level n-gram）在指定范围内的样本 |
| `document_simhash_deduplicator` | 使用 `SimHash` 在文档级别对样本去重 |

### 6.3 data-jucier所需参数设置

- **processed_ape210k.jsonl**
  
| 统计项 (过滤器)                          | 平均值  | 中位数  | 最小值 | 最大值 | 95% 分位数 |
|--------------------------------------|------|------|----|------|------|
| **单样本文本字符数 (text_length_filter)**    | 344.1  | 309.0  | 95  | 2765  | 638.0  |
| **单样本所含单词数 (words_num_filter)**         | 52.46  | 46.0  | 13  | 568  | 103.0  |
| **平均行长度 (average_line_length_filter)**  | 33.89  | 33.62  | 22.25  | 123.5  | 40.3  |
| **最长行长度 (maximum_line_length_filter)**  | 65.57  | 60.0  | 36  | 1366  | 94.0  |
| **字符重复率 (character_repetition_filter)** | 0.07  | 0.07  | 0.02  | 0.35  | 0.09  |
| **单词重复率 (word_repetition_filter)**  | 0.13  | 0.12  | 0.04  | 0.76  | 0.19  |
| **特殊字符比例 (special_characters_filter)**  | 0.22  | 0.22  | 0.1  | 0.49  | 0.25  |

考虑到ape本身数据量较小、最后决定的参数十分宽松。一般来说数学流程的单词重复率和字符重复率都很高，在很小的范围内就容易出现重复，因此要小心筛选。同时、对数学而言困惑度无意义、不进行困惑度筛选。

```python
    # 单样本文本字符数
  - text_length_filter:
      max_len: 2700 # 接近最大值

    # 单样本所含单词数
  - words_num_filter:
      min_num: 20 # 只筛下限不筛上限
      max_num: 99999 # 和openr1相比都很短

    # 平均行长度
  - average_line_length_filter:
      min_len: 20  # 比最小值更小
      max_len: 150  # 比最大值更大

    # 最长行长度
  - maximum_line_length_filter:
      max_len: 1000  # 接近最大值

    # 字符重复率
  - character_repetition_filter:
      max_ratio: 0.5  # 数学、重复率允许较高

    # 单词重复率
  - word_repetition_filter:
      rep_len: 10
      max_ratio: 0.75 # 数学、重复率允许较高
      
    # 特殊字符比例
  - alphanumeric_filter:
      tokenization: false
      min_ratio: 0.0  # 不设下限
      max_ratio: 0.5  # 数学、上限允许高一些

  - document_simhash_deduplicator:
      tokenization: space
      window_size: 6 # 十分宽松的去重
      lowercase: true
      ignore_pattern: '\p{P}'
      num_blocks: 6
      hamming_distance: 4
```

- **processed_openr1.jsonl**
  
| 统计项 (过滤器)                          | 平均值  | 中位数  | 最小值 | 最大值 | 95% 分位数 |
|--------------------------------------|------|------|----|------|------|
| **单样本文本字符数 (text_length_filter)**    | 29699.23  | 22220.0  | 756  | 196439  | 77882.0  |
| **单样本所含单词数 (words_num_filter)**         | 6035.55  | 4499.0  | 119  | 41181  | 16013.0  |
| **平均行长度 (average_line_length_filter)**  | 1162.5  | 834.43  | 17.77  | 9461.0  | 3186.56  |
| **最长行长度 (maximum_line_length_filter)**  | 15889.56  | 11683.0  | 251  | 79845  | 43239.0  |
| **字符重复率 (character_repetition_filter)** | 0.18  | 0.18  | 0.04  | 0.81  | 0.2  |
| **单词重复率 (word_repetition_filter)**  | 0.08  | 0.07  | 0.02  | 0.9  | 0.12  |
| **特殊字符比例 (special_characters_filter)**  | 0.15  | 0.15  | 0.03  | 0.46  | 0.23  |

推理文本可长可短、但是还是倾向于使用比较长的推理文本，因此上限基本都给得比较宽松、下限反而给得相对严格一些，但是这可能会筛掉很多简单的数学推理题目、让模型最终反而无法回答简单的问题。同时，同样作为数学题目、openr1对于重复词筛选也会非常敏感，因此也需要谨慎处理。

```python
    # 单样本文本字符数
  - text_length_filter:
      max_len: 80000 # 略超过95%中位数

    # 单样本所含单词数
  - words_num_filter:
      min_num: 150 # 保证推理文本不会太短
      max_num: 17000 # 略超过95%中位数

    # 平均行长度
  - average_line_length_filter:
      min_len: 30 # 保证推理文本不太短
      max_len: 8000  # 给得也比较宽松

    # 最长行长度
  - maximum_line_length_filter:
      max_len: 45000  # 略超过95%中位数

    # 字符重复率
  - character_repetition_filter:
      max_ratio: 0.35  # 极值太大、设置超过95%中位数

    # 单词重复率
  - word_repetition_filter:
      rep_len: 10
      max_ratio: 0.35 # 极值太大、设置超过95%中位数

    # 特殊字符比例
  - alphanumeric_filter:
      tokenization: false
      min_ratio: 0.0  # 不设下限
      max_ratio: 0.8  # 上限允许0.5

  - document_simhash_deduplicator:
      tokenization: space
      window_size: 6 # 十分宽松的去重
      lowercase: true
      ignore_pattern: '\p{P}'
      num_blocks: 6
      hamming_distance: 4

```

- **processed_skypile.jsonl**
  
| 过滤器 | 统计项 | 平均值 | 中位数 | 最小值 | 最大值 | 95% 分位数 |
|--------|--------|--------|--------|--------|--------|------------|
| **单样本文本字符数** | `text_length_filter` | 1028.12 | 741.0 | 5 | 203536 | 2681.0 |
| **单样本所含单词数** | `words_num_filter` | 97.08 | 69.0 | 0 | 21526 | 255.0 |
| **平均行长度** | `average_line_length_filter` | 134.06 | 109.0 | 5.0 | 175508.0 | 274.0 |
| **最长行长度** | `maximum_line_length_filter` | 233.84 | 195.0 | 5 | 194936 | 471.0 |
| **字符重复率** | `character_repetition_filter` | 0.05 | 0.05 | 0.01 | 1.0 | 0.07 |
| **单词重复率** | `word_repetition_filter` | 0.04 | 0.03 | 0.0 | 1.0 | 0.09 |
| **特殊字符比例** | `special_characters_filter` | 0.1 | 0.09 | 0.0 | 1.0 | 0.14 |

文本极值很大很多、在之后可能会重整文本分割方式、因此在设置时没有使用大力度进行筛选。但是对文本来说，过长的段落可能意味着文本质量不高，因此既希望保留尽量多的数据、同时也希望有一定的筛选力度，在无法抉择时就使用1/4 * 最大值的方式进行压制。

```python
  # 困惑度筛选，考虑在困惑度上进行宽松的筛选、给了比较大的困惑度
  - perplexity_filter: 
      lang: zh                                                
      max_ppl: 1500              
      
    # 单样本文本字符数
  - text_length_filter:
      max_len: 50000 # 极值与95%分位数差异太大、对文本来说，取极值的1/4

    # 单样本所含单词数
  - words_num_filter:
      min_num: 10 # 至少是一句话  
      max_num: 20000 # 接近最大值

    # 平均行长度
  - average_line_length_filter:
      min_len: 10 # 最小值的两倍
      max_len: 50000 # 极值与95%分位数差异太大、取极值的1/4

    # 最长行长度
  - maximum_line_length_filter:
      max_len: 50000  # 取极值的1/4

    # 字符重复率
  - character_repetition_filter:
      max_ratio: 0.35  # 极值与95%分位数差异太大、取极值的1/3

    # 单词重复率
  - word_repetition_filter:
   #   rep_len: 10   只对word-level有效、对中文无效
      max_ratio: 0.35 # 极值太大、取极值的1/3

    # 特殊字符比例
  - alphanumeric_filter:
      tokenization: false
      min_ratio: 0.0  # 不设下限
      max_ratio: 0.35  # 极值太大、取极值的1/3

```

合理的困惑度（PPL）值取决于数据类型、语言模型的复杂度、文本质量等多个因素。一般来说，**困惑度越低，文本的质量越好**，但**过低可能意味着文本太简单或有问题**。

| **数据类型**      | **理想困惑度范围 (PPL)** | **解释** |
|----------------|----------------|---------------------------|
| **高质量文本**（新闻、百科） | **10 - 500**  | 低困惑度，流畅且符合语法 |
| **学术论文** | **20 - 1000** | 句子结构复杂，困惑度较高 |
| **代码（编程语言）** | **50 - 300** | 代码结构稳定，但有关键字限制 |
| **社交媒体（短文本）** | **100 - 2000** | 口语化表达，变化较大 |
| **爬取的原始网页数据** | **500 - 5000+** | 噪音较多，可能包含垃圾文本 |

💡 **一般筛选建议**：
- **困惑度 > 1500** → **可能是低质量文本**（噪声、乱码、拼写错误）。
- **困惑度 < 50** → **可能是格式化数据**（如模板化内容，重复性高）。
- **困惑度 100-1000** → 适合**大多数高质量自然文本**。

- **processed_starcoder.jsonl**

| 过滤器 | 统计项 | 平均值 | 中位数 | 最小值 | 最大值 | 95% 分位数 |
|--------|--------|--------|--------|--------|--------|------------|
| **文本字符数** | `text_length_filter` | 3904.01 | 1298.0 | 2 | 2374708 | 13131.0 |
| **单词数** | `words_num_filter` | 444.53 | 146.0 | 1 | 524313 | 1477.0 |
| **平均行长度** | `average_line_length_filter` | 75.69 | 35.22 | 1.0 | 320654.5 | 125.52 |
| **最长行长度** | `maximum_line_length_filter` | 214.18 | 96.0 | 1 | 1025819 | 614.0 |
| **字符重复率** | `character_repetition_filter` | 0.16 | 0.14 | 0.0 | 1.0 | 0.32 |
| **单词重复率** | `word_repetition_filter` | 0.09 | 0.07 | 0.0 | 1.0 | 0.2 |
| **特殊字符比例** | `special_characters_filter` | 0.14 | 0.13 | 0.0 | 0.77 | 0.24 |


> **📌 代码数据 vs 文字数据的筛选差异**
| 方面  | 代码数据 | 文字数据 |
|------------|--------------------------------------|--------------------------------------------------|
| **结构** | 结构化、层次分明，有语法约束 | 非结构化，句法和段落灵活 |
| **完整性** | 代码片段通常需要完整才能执行 | 文字可以被切分、重组 |
| **换行/格式** | 代码换行是语义的一部分（如函数、类定义） | 文字换行主要是排版作用，不影响理解 |
| **冗余性** | 重复代码可能是必要的（如变量声明、导入库） | 过多重复文本通常是低质量内容 |
| **长度问题** | 代码短小精悍为佳，过长可能是自动生成的垃圾数据 | 文字数据往往有极大极小值，需要合理归一化 |
| **特殊字符** | 特殊符号（`{}`、`[]`、`<>`、`=`）在代码中不可缺 | 文字中过多特殊符号可能表示乱码或低质量文本 |

考虑到starcoder其实是经过严格清洗的数据、我们给的条件相对宽松。考虑到代码对完整性的要求较高，较长的代码可能具有较复杂的逻辑供模型学习，我们的筛选条件相对保守——

```python
    # 单样本文本字符数
  - text_length_filter:
      max_len: 1000000 # 长代码虽然也有冗余嫌疑、但大部分情况下具有意义，取极值的一半

    # 单样本所含单词数
  - words_num_filter:
      min_num: 20 # 不能太低，过短的代码可能是无意义的单行代码片段 
      max_num: 250000 # 取极值的一半

    # 平均行长度
  - average_line_length_filter:
      min_len: 20 # 同样不能太低
      max_len: 150000 # 极值与95%分位数差异太大、取极值的一半

    # 最长行长度
  - maximum_line_length_filter:
      max_len: 500000  # 取极值的一半

    # 字符重复率
  - character_repetition_filter:
      max_ratio: 0.6  # 在代码中字符会有较高的重复率、属于正常、取极值2/3

    # 单词重复率
  - word_repetition_filter:
      rep_len: 10
      max_ratio: 0.6 # 在代码中单词也会有较高的重复率，取极值2/3

    # 特殊字符比例
  - alphanumeric_filter:
      tokenization: false
      min_ratio: 0.0  # 不设下限
      max_ratio: 0.6  # 代码中需要特殊字符，所以不能严格限制

  - document_simhash_deduplicator:
      tokenization: space
      window_size: 6 #使用较大的window_size、使去重的颗粒度变粗
      lowercase: true
      ignore_pattern: '\p{P}'
      num_blocks: 6
      hamming_distance: 4
```

- **processed_slimpajama.jsonl**
  
| 过滤项                        | 平均值    | 中位数   | 最小值  | 最大值   | 95% 分位数 |
|-------------------------------|----------|---------|--------|---------|-----------|
| **文本字符数** (text_length_filter) | 3823.63  | 1981.0  | 20     | 100000  | 12422.0   |
| **单词数** (words_num_filter) | 634.15   | 327.0   | 1      | 28850   | 2061.0    |
| **平均行长度** (average_line_length_filter) | 3823.63  | 1981.0  | 20.0   | 100000.0 | 12422.0   |
| **最长行长度** (maximum_line_length_filter) | 3823.63  | 1981.0  | 20     | 100000  | 12422.0   |
| **字符重复率** (character_repetition_filter) | 0.16     | 0.16    | 0.02   | 1.0     | 0.19      |
| **单词重复率** (word_repetition_filter) | 0.06     | 0.05    | 0.0    | 1.0     | 0.1       |
| **特殊字符比例** (special_characters_filter) | 0.04     | 0.03    | 0.0    | 0.97    | 0.1       |

英文文本、处理逻辑基本与中文文本接近，既希望保留尽量多的数据、同时也希望有一定的筛选力度，在无法抉择时就使用1/4 * 最大值的方式进行压制。

```python
    # 单样本文本字符数
  - text_length_filter:
      max_len: 25000 # 极值与95%分位数差异太大、对文本来说，取极值的1/4

    # 单样本所含单词数
  - words_num_filter:
      min_num: 10 # 至少是一句话  
      max_num: 25000 # 接近最大值

    # 平均行长度
  - average_line_length_filter:
      min_len: 20 # 按最小值走
      max_len: 25000 # 极值与95%分位数差异太大、取极值的1/4

    # 最长行长度
  - maximum_line_length_filter:
      max_len: 25000  # 取极值的1/4

    # 字符重复率
  - character_repetition_filter:
      max_ratio: 0.35  # 极值与95%分位数差异太大、取极值的1/3

    # 单词重复率
  - word_repetition_filter:
      rep_len: 10   只对word-level有效、对中文无效
      max_ratio: 0.35 # 极值太大、取极值的1/3

    # 特殊字符比例
  - alphanumeric_filter:
      tokenization: false
      min_ratio: 0.0  # 不设下限
      max_ratio: 0.35  # 极值太大、取极值的1/3

    # 困惑度筛选，使用了默认值
  - perplexity_filter:
      lang: en
      max_ppl: 1500

  - document_simhash_deduplicator:
      tokenization: space
      window_size: 6 #使用较大的window_size、使去重的颗粒度变粗
      lowercase: true
      ignore_pattern: '\p{P}'
      num_blocks: 6
      hamming_distance: 4
```

## 7 数据处理第三步：数据合并与Tokenizer

现在我们已经有了清洗好的、经过质量控制的数据，接下来我们则需要用一段代码将所有数据进行打包和分词、并最终将数据处理为二进制bin文件、供后续的预训练进行使用。

```
【step 1】📂 raw data (PDF/HTML/TXT)
            ↓  (转换)
【step 2】📂 JSONL {"text": "..."}
            ↓  (清洗)
          📂 cleaned JSONL
            ↓  (质量控制)
【step 3】📂 filtered JSONL
            ↓  (tokenizer)
          📂 Tokenized JSONL {"tokens": [...]}
            ↓  (拼接)
          📂 Pretrain Data (bin/lmdb)
```

In [ ]:
zh(90G), en(90G), code(60G), math (8G), experts(2G)

在进行打包时，我们所使用的代码是`step3_prepare_data_for_pretrain.py`，其中涉及到的流程有——

1. **对数据进行混合及按比例采样**

2. **对数据加上前缀或者后缀脚标**

3. **使用multiprocessing启用多进程、同步对数据进行tokenizer**

4. **使用PackedDatasetBuilder将数据拆分成小型二进制bin文件**

最终我们为预训练准备的文件结构为 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/153.png)

### 7.1 数据合并与tokenizer的执行脚本

**在执行下面的脚本之前，请务必先删除data-jucier生成的数据中的stats那个jsonl** ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/150.png)

在下面的每个文件夹中都删除stats jsonl后，上传tokenizer文件夹至`/root/autodl-tmp/minideepseek/v3/`，你的文件夹里应该有如下的所有内容↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/151.png)

确保你的设备上有至少500G左右的硬盘空间（运行到这一步，你可以考虑删除最初下载的原始数据、二只保留step1和step2留下的数据）、开始运行下面的代码 ↓

```shell
# 使用训练的虚拟环境
deactivate
cd ~/autodl-tmp/minideepseek/v3
source mnds_training/bin/activate

# 建立新的数据存储目录
mkdir /root/autodl-tmp/minideepseek/v3/data/final_data

# 配置环境
pip install lightning
pip install jsonargpase

# 将step3代码上传至tokenizer
# 进入代码执行目录
cd /root/autodl-tmp/minideepseek/v3/tokenizer

# 执行step3代码
#【TIME WARNING：64进程并行、约4小时】
python step3_prepare_data_for_pretrain.py

python step3_prepare_data_for_pretrain_upsample.py
```

当你顺利运行后，你会看到 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/154.png)

你可以用下面的代码来检查你的多进程是否顺利被启用 ↓

```shell
htop
```

如果你看到有大量的进程在同步运行，其中CPU%的列下面有大量的100%或者接近100%的数字，则说明代码被顺利启用 ↓ 如果只有一个进程占比100%，则说明没有成功启用multiprocessing。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/152.png)

### 7.2 数据合并脚本解读

```python
import glob
import json
import os
import sys
from pathlib import Path
import multiprocessing as mp
import queue
import numpy as np
from tqdm import tqdm

# support running without installing as a package
wd = Path(__file__).parent.parent.resolve()
sys.path.append(str(wd))

import random
random.seed(666)
import lit_gpt.packed_dataset as packed_dataset
from lit_gpt import Config, Tokenizer
from transformers import AutoTokenizer
import time

filenames_sample = []
max_length = 131072  # 限制最大 token 长度，避免超长文本导致模型崩溃

# key: name value: dir

###################################
###  处理单个文本，执行 Tokenizer ###
###################################
def process_line_text(text, tokenizer, max_length=131072):
    """
    负责对文本进行 Tokenizer 处理，并把超长 Token 切分成多个小块进行存储。
    其中max_length是tokenizer的config设置中规定的数字。
    """
    text_ids = tokenizer.encode(text)

    if len(text_ids) > max_length:
        print(f"⚠️ 超长文本 ({len(text_ids)} tokens) 被拆分！")
        return [item for chunk in [text_ids[i:i+max_length] for i in range(0, len(text_ids), max_length)] for item in chunk]
    else:
        return [text_ids]

##################################################
###  处理 JSONL 文件，将其 Tokenize 并存入数据集 ###
#################################################
def process_jsonl_file(set_name, file_dir, builder, tokenizer, cache_lines_num):
    """
    读取 JSONL 文件，解析其中的 text 字段，并将其转换为 Token ID，存入 PackedDatasetBuilder 进行保存。
    其中，我们的 JSONL 文件有固定格式，为 {"text": "xxxx"}。
    """
    cache_text = ""
    counter = 0
    with open(file_dir, encoding="utf-8") as f:
        for row in tqdm(f):
            counter += 1
            try:
                # 直接提取 "text" 字段
                text = json.loads(row)["text"]
            except Exception as e:
                print(f"文件 {file_dir} 第 {counter} 行 读取错误: {e}")
                counter -= 1
                continue

            text += "<|im_end|>"  # 追加结束符
            cache_text += text

            # 每 cache_lines_num 行处理一次
            if counter % cache_lines_num == 0:
                text_ids = process_line_text(cache_text, tokenizer)
                builder.add_array(np.array(text_ids, dtype=builder.dtype))
                cache_text = "" 

        # 处理剩余缓存数据
        if cache_text:
            text_ids = process_line_text(cache_text, tokenizer)
            builder.add_array(np.array(text_ids, dtype=builder.dtype))

##############################
###  数据集上采样权重设置    ###
##############################
upsample_dict = {
    "djed_ape210K": 3,   # 该数据集增加 3 倍
    "djed_openr1": 2,    # 该数据集增加 2 倍
    "djed_starcoder": 1, # 该数据集正常采样
    "djed_skypile": 1,   # 该数据集正常采样
    "djed_slimpajama": 1 # 该数据集正常采样
}

########################################
###  处理多个 JSONL 文件（多进程并行） ####
########################################
def multiprocess_data(set_name, file_dir_list, builder, tokenizer, cache_lines_num, process_idx=0, upsample_dict=None):
    """
    多进程处理数据，负责调用 process_jsonl_file 处理多个 JSONL 文件。
    并且根据 `upsample_dict` 进行数据集上采样。
    """
    if upsample_dict is None:
        upsample_dict = {}

    try:
        for file_dir in file_dir_list:
            upsample_factor = upsample_dict.get(set_name, 1)  # 获取数据集的上采样倍数
            for _ in range(int(upsample_factor)):  # 多次处理该数据集
                t0 = time.time()
                process_jsonl_file(set_name, file_dir, builder, tokenizer, cache_lines_num)
                print(f"Process {process_idx} 处理 {file_dir} ({upsample_factor} 倍) 耗时 {time.time()-t0:.2f}s")

        print(f"{builder._prefix} 已处理 {builder._counter+1} 文件，共 {builder.all_tokens} tokens")
        builder.write_reminder()

    except Exception as e:
        print(f"multiprocess_data 处理 {set_name} 发生错误: {str(e)}")

##############################
###  遍历文件并创建进程队列 ###
##############################
def prepare_full(source_path: Path, checkpoint_dir: Path, destination_path: Path, : int, 
                 match: str = "", max_files=100000000000000, cache_lines_num=1000, process_num=64) -> None:
    """
    遍历 filename_sets 字典，根据 glob 匹配路径，找到需要处理的所有 JSONL 文件。
    并手动处理 multiprocessing 多进程操作，按进程数量 process_num 划分文件，每个进程处理一部分数据。
    """

    destination_path.mkdir(parents=True, exist_ok=True)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)

    for set_name, pattern in filename_sets.items():
        if match and match not in set_name:
            continue

        t0 = time.time()
        filenames = glob.glob(os.path.join(source_path, pattern), recursive=True)
        filenames = sorted(filenames)
        random.shuffle(filenames)
        filenames = filenames[:max_files]

        if not filenames:
            print(f"⚠️ No files found for {set_name}, skipping...")
            continue

        # 计算动态 `process_num`
        process_num = 1 if len(filenames) == 1 else min(64, len(filenames))

        print(f"📌 处理 {set_name}，文件数: {len(filenames)}, 启动进程数: {process_num}")

        builder_list = [
            packed_dataset.PackedDatasetBuilder(
                outdir=destination_path,
                prefix=f"{set_name}_process{i}",
                chunk_size = chunk_size,
                sep_token=tokenizer.pad_token_id,
                dtype="auto",
                vocab_size=len(tokenizer),
            ) for i in range(process_num)
        ]
        
        # 在文件数超过process_num的时候
        # 拉起排队、确保每个文件都能被处理到
        task_queue = queue.Queue()
        for filename in filenames:
            task_queue.put(filename)

        def worker(process_idx):
            while not task_queue.empty():
                try:
                    file_dir = task_queue.get_nowait()
                except queue.Empty:
                    break
                print(f"🛠️ Process {process_idx} handling {file_dir}")
                multiprocess_data(set_name, [file_dir], builder_list[process_idx % process_num],
                                  tokenizer, cache_lines_num, process_idx)

        process_list = []
        for process_idx in range(process_num):
            process = mp.Process(target=worker, args=(process_idx,))
            process_list.append(process)
            process.start()

        for process in process_list:
            process.join()

        print(f"✅ 处理完成 {set_name}，总耗时: {time.time()-t0:.2f}s")

##############################
###  运行主程序             ###
##############################
if __name__ == "__main__":
    """允许命令行调用 prepare 进行数据预处理"""
    import jsonargparse
    from jsonargparse import CLI
    CLI(prepare)

```

#### 7.2.1 PackedDataset的实现与使用

`PackedDatasetBuilder` 主要用于 **高效存储和管理 tokenized 数据，以便后续模型训练**。它的核心目标是：
- **将多个 token 序列打包成连续的存储格式**，减少 I/O 读取成本，提高训练时的数据加载效率。
- **使用固定大小的 chunk 进行存储**，以保证数据格式的统一性，同时优化 GPU 计算效率。
- **支持不同的数据类型（dtype）**，确保 token 数据占用最少的存储空间。

 `prepare_full()` 函数里，我们创建了多个 `PackedDatasetBuilder` 实例：
```python
builder = packed_dataset.PackedDatasetBuilder(
    outdir=destination_path,
    prefix=set_name+"_"+f"process{i}",
    chunk_size=chunk_size,
    sep_token=tokenizer.pad_token_id,  # 这里是填充 token
    dtype="auto",
    vocab_size=len(tokenizer),
)
```

| 参数 | 作用 |
|------|------|
| `outdir` | 存储输出数据的目录 |
| `prefix` | 输出文件的前缀，用于区分不同数据集 |
| `chunk_size` | **一个数据块的大小**，影响存储效率和训练加载速度 |
| `sep_token` | **填充 token**，用于填充序列以对齐 batch |
| `dtype` | **数据类型**（如 `int32` 或 `uint16`），可减少存储占用 |
| `vocab_size` | **词汇表大小**，用于确定 token 存储的 bit 位 

#### 7.2.2 如何选择合适的并行方式？

| **任务类型** | **文件类型** | **适合的并行方式** | **推荐的并行数（`np`）** | **优化建议** |
|--------------|--------------|-----------------|-------------------|--------------|
| **计算密集型任务**（如 Tokenization、模型推理、大量数学计算） | **单个大 JSONL 文件**（> 10GB） | **进程并行 (`multiprocessing`)** | **50% - 75% 的 CPU 物理核心数**（如 16 核 CPU，用 `np=8~12`） | **避免 `np` 过大导致 CPU 过载**，适当调整 batch 处理 |
| **计算密集型任务** | **多个小 JSONL 文件**（每个 < 100MB） | **进程并行 (`multiprocessing`)** | **CPU 核心数（如 16 核 CPU，用 `np=16`）** | **分批处理 JSONL，动态分配进程以提高吞吐量** |
| **I/O 密集型任务**（如 文件读取、日志分析、数据预处理） | **单个大 JSONL 文件**（> 10GB） | **线程并行 (`threading`)** | **1.5x - 2x 的 CPU 逻辑线程数（如 16 核 32 线程，可用 `np=48~64`）** | **使用 `asyncio` 或 `aiofiles` 进一步优化并发** |
| **I/O 密集型任务** | **多个小 JSONL 文件**（每个 < 100MB） | **线程并行 (`threading`)** | **逻辑线程数（如 16 核 32 线程，可用 `np=32`）** | **确保磁盘 I/O 不成为瓶颈，观察 `iostat -x 1`** |
| **混合任务（计算 + I/O）**（如 读取 JSONL + Tokenizer） | **单个大 JSONL 文件** | **进程并行 (`multiprocessing`) + I/O 预加载** | **CPU 物理核心数（如 16 核 CPU，用 `np=8~12`）** | **使用 `cache_lines_num` 控制 I/O 频率，避免磁盘过载** |
| **混合任务（计算 + I/O）** | **多个小 JSONL 文件** | **进程并行 (`multiprocessing`)** | **CPU 物理核心数（如 `np=16`）** | **预分配文件给进程，减少进程间通信开销** |

- **什么是I/O限制**？

I/O 限制（I/O Bottleneck）指的是 输入/输出（Input/Output）操作的速度成为系统的瓶颈，导致 CPU 或 GPU 计算资源无法充分利用。在数据处理任务（比如咱们在 step3 里转换 .jsonl 为 .bin）时，数据需要从磁盘读取、处理后再写入磁盘，如果磁盘 I/O 速度跟不上 CPU 计算速度，那么 I/O 就会成为整个任务的瓶颈。

- **既然数据处理是I/O密集型任务，为什么无论是data-jucier还是tokenizer这些流程，都使用multiprocessing而没有使用threading**？

在运行python代码的时候、Python 的全局解释器锁（GIL）会限制`threading`的性能。Python 解释器 允许多个线程存在，但 同一时间只有一个线程在运行 Python 计算（即使你有 64 核 CPU），threading 适用于 I/O 任务（如网络请求、数据库查询），但 如果涉及到 CPU 计算（如 Tokenization），threading 可能会被 GIL 限制，导致并行效果很差！

无论是datajucier还是tokenizer过程、我们都可能涉及到CPU上的运算（比如说字节对编码BPE、哈希值计算等等），这些都会受到GIL限制，如果我们使用的是线程并行、多线程会互相等待、无法真正并行运行Tokenization和各类计算。因此multiprocessing在实际大模型运行场景中会更适合作为数据清洗流程中的并行方式。

#### 7.2.3 Prefix-Suffix-Middle数据掩码

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/155.png)

根据Deepseek的论文所描述，**DeepSeekCoder-V2** 使用了一种 **"Fill-in-Middle (FIM)"** 的训练方法，这是一种 **填充中间的预测策略**。这种方法不影响传统的 **“预测下一个 Token”（Next Token Prediction, NTP）** 任务，同时让模型能够更好地**理解上下文信息，并预测文本的中间部分**。

| 训练方式 | 预测方式 | 适用任务 |
|----------|------------|------------|
| **Autoregressive (自回归模型)** | 仅预测下一个 token | 适用于 GPT 训练 |
| **FIM (Fill-in-Middle)** | 预测文本的中间部分 | 适用于代码补全、上下文填充 |

- **什么是 Prefix-Suffix-Middle (PSM) 结构？**
  
DeepSeekCoder-V2 采用了 **Prefix-Suffix-Middle (PSM)** 结构来组织数据。这种结构将一个完整的文本 **拆分成三部分**：

1️⃣ **Prefix (前缀部分，`f_pre`)**  
2️⃣ **Suffix (后缀部分，`f_suf`)**  
3️⃣ **Middle (中间部分，`f_middle`)**（需要模型预测的部分）

论文中表述的方式是 ↓
```
<|fim_begin|> f_pre <|fim_hole|> f_suf <|fim_end|> f_middle <|eos_token|>
```
- `<|fim_begin|>` → 表示 **填充任务的开始**
- `f_pre` → **前缀**（上下文信息）
- `<|fim_hole|>` → **填充位置**
- `f_suf` → **后缀**（提供部分上下文）
- `<|fim_end|>` → **结束填充**
- `f_middle` → **模型需要预测的文本部分**
- `<|eos_token|>` → **结束符**


假设我们有一个完整的句子：
> **"今天的天气很好，我们一起去公园散步吧！"**

如果采用 **Prefix-Suffix-Middle (PSM) 结构**，可能会拆分为：
- **前缀（Prefix, `f_pre`）：** `"今天的天气很好，"`
- **后缀（Suffix, `f_suf`）：** `"去公园散步吧！"`
- **中间部分（Middle, `f_middle`，模型需要预测）：** `"我们一起"`

转换为 PSM 结构：
```
<|fim_begin|> 今天的天气很好， <|fim_hole|> 去公园散步吧！ <|fim_end|> 我们一起 <|eos_token|>
```

模型在训练时，需要根据 `f_pre`（前缀）和 `f_suf`（后缀）来预测 `f_middle`（中间部分），因此如果要执行PSM、则是需要对数据先进行各个部分的标注、再交换顺序、再使用掩码将`f_middle`部分遮盖起来、还要生成能够与`f_middle`相匹配的标签，实际执行难度远远高于论文中简短的几句话。但是、这一定是一个对代码训练非常有效的方式，如果你的模型需要说某种特定的低资源语言、或者是模型核心负责写代码、那这个方式你一定要想办法实现。

## 8 数据处理第四步：将数据按照神经网络需求标准化

当我们有了完整的bin文件之后、整个数据处理的流程就只剩下最后一步：将数据通过PretrainDataset转变成能够与神经网络兼容的格式。这一步其实已经不算是数据预处理、而是常规的神经网络训练前的流程了。

```
【step 1】📂 raw data (PDF/HTML/TXT)
            ↓  (转换)
【step 2】📂 JSONL {"text": "..."}
            ↓  (清洗)
          📂 cleaned JSONL
            ↓  (质量控制)
【step 3】📂 filtered JSONL
            ↓  (tokenizer)
          📂 Tokenized JSONL {"tokens": [...]}
            ↓  (拼接)
【step 4】📂 Pretrain Data (bin/lmdb)
            ↓  (匹配pytorch + deepspeed规则)
          📂 能够索引的token
            ↓  (匹配神经网络所需结构)
          📂 分好批次的3d数据
```

这段代码位于`tokenizer`文件夹下的`step4_dataset.py`中、其核心功能包括：

1. **实现数据跨类别打乱（shuffle）、解决数据分布不均的问题**，确保dataloader读取的数据顺序是随机的。

2. **封装 \_\_getitem\__，适配 PyTorch/DeepSpeed 训练**，PretrainDataset 继承 torch.utils.data.Dataset，可以 直接被 Dataloader 使用，支持索引访问，dataset[i] 可以直接返回 (X, Y)，让 PyTorch 训练框架能高效加载数据，支持 多进程数据加载，为分布式做准备。

3. **使用memmap进行高效数据读取、节省内存**，通过np.memmap映射.bin文件到内存、避免一次性加载所有数据、还能提高I/O交换速度

4. **构造符合神经网络需求的X,Y训练样本**，需要兼容 DeepSeek 采用的 Causal Language Modeling（CLM）+ Multi-Token Prediction 训练方式，如果有更复杂的预测流程则还需要兼容其他预测流程。

这一步的代码将与预训练的`pretrain.py`一同使用、因此无需单独的shell脚本。具体的代码如下 ↓

```python

import os
import numpy as np
import torch
from torch.utils.data import Dataset
import random

##############################
###  预训练数据集类 (PretrainDataset) ###
##############################
class PretrainDataset(Dataset): 
    def __init__(self, data_path_lst, max_length=512, memmap=False, shuffle=True):
        """
        负责加载预训练数据集，并支持 memory-mapped 文件读取方式。

        Args:
            data_path_lst (list): 存储所有 .bin 预处理数据文件路径的列表
            max_length (int): 每个样本的最大长度（token 数量）
            memmap (bool): 是否使用 memory-mapped 方式读取数据，节省内存
            shuffle (bool): 是否对数据进行 shuffle 以增强训练效果
        """
        super().__init__()
        self.data_path_lst = data_path_lst
        self.max_length = max_length
        self.memmap = memmap
        self.shuffle = shuffle
        self.dtype = np.dtype('uint16')  # ✅ 设定数据类型

        # 存储文件偏移信息
        self.file_offsets = []
        self.total_samples = 0

        ####################################
        ###  遍历所有 .bin 文件，计算样本数 ###
        ####################################
        for file_path in self.data_path_lst:
            file_size = os.path.getsize(file_path)  # 获取文件大小（字节）
            num_samples = (file_size // self.dtype.itemsize) // max_length  # 计算该文件中包含多少个样本
            self.file_offsets.append((file_path, self.total_samples, num_samples))  # 记录文件偏移信息
            self.total_samples += num_samples  # 更新数据集中总样本数

        ####################################
        ###  如果使用 memmap，打开所有文件 ###
        ####################################
        if self.memmap:
            self.mmaps = [np.memmap(file, dtype=self.dtype, mode='r') for file, _, _ in self.file_offsets]
        else:
            self.mmaps = None  # 非 memmap 模式

        ####################################
        ###  生成样本索引，并进行 Shuffle ###
        ####################################
        self.indices = list(range(self.total_samples))  # 生成样本索引
        if self.shuffle:
            random.shuffle(self.indices)  # ✅ 确保数据是随机的，避免训练时的样本偏差

        print(f"Loaded {len(self.file_offsets)} files, total samples: {self.total_samples}")

    ###########################
    ###  获取数据集的总样本数 ###
    ###########################
    def __len__(self):
        return self.total_samples

    ############################
    ###  根据索引获取数据样本 ###
    ###########################
    def __getitem__(self, index):
        """
        通过索引找到对应的 .bin 文件，并返回该索引对应的样本数据。

        Args:
            index (int): 样本索引

        Returns:
            X (torch.Tensor): 输入 token 序列
            Y (torch.Tensor): 目标 token 序列（用于预测下一个 token）
        """
        index = self.indices[index]  # ✅ 使用 Shuffle 过的索引，保证训练数据顺序是随机的

        ################################
        ###  找到索引所在的 .bin 文件 ###
        ###############################
        for file_idx, (file_path, start_idx, num_samples) in enumerate(self.file_offsets):
            if index < start_idx + num_samples:
                local_index = index - start_idx  # 计算该索引在该文件中的位置
                break
        else:
            raise IndexError("Index out of range")  # ✅ 确保索引不会超出数据范围

        ##############################
        ###  从文件中读取数据       ###
        ##############################
        if self.memmap:
            # ✅ 使用 memory-mapped 方式读取数据，避免频繁 IO
            sample = self.mmaps[file_idx][local_index * self.max_length : (local_index + 1) * self.max_length]
        else:
            # ✅ 直接从二进制文件读取数据（非 memmap 模式）
            with open(file_path, 'rb') as f:
                f.seek(local_index * self.max_length * self.dtype.itemsize)  # 定位到正确的样本位置
                sample = np.frombuffer(f.read(self.max_length * self.dtype.itemsize), dtype=self.dtype)

        ##############################
        ###  生成 X 和 Y 作为训练输入/目标 ###
        ##############################
        X = np.array(sample[:-1], dtype=np.int64)  # 输入序列（去掉最后一个 token）
        Y = np.array(sample[1:], dtype=np.int64)  # 目标序列（向右移动一位）

        return torch.from_numpy(X), torch.from_numpy(Y) 
```

## 9 DeepSeekv3的预训练

### 9.1 预训练执行脚本

- **deepseekmini预训练数据集**
  
链接: https://pan.baidu.com/s/1-p4R2wGkgDbpwhTlwvLpVg?pwd=5v3z
提取码: 5v3z 

- **全部文件结构如下**
```python
minideepseek/v3
├── requirements_minids.txt      # 预训练所需的环境
├── test_v3model.py              # 测试deepseev3_mtp_model.py能否跑通
├── logger_utils.txt             # 进行logger控制和调试的代码
├── pretrain.py                  # 预训练代码
│
├── model/                       # model文件夹、存放模型文件
│   ├── config.py
│   ├── convert.py
│   ├── deepseev3_mtp_model.py   # 要训练的模型脚本，加上了mtp
│   ├── fp8_cast_bf16.py
│   ├── kernel.py
│
├── data/                        # data文件夹
│   ├── print_datas.py           # 对下载好的数据进行状态打印的脚本
│   ├── analyze_jsonl.py         # 查询各jsonl文件统计指标的脚本
│   ├── data-juicer-main.zip     # data-juicer库安装所需文件
│   ├── delete_files.sh          # 删除数据、降低数据大小的脚本
│   ├── split_data.py            # 进行数据分割的python脚本 
│   ├── hfd_revised.sh           # 分布式并行数据下载脚本（防429报错版）
│   ├── slimpajama_preclean.py   # slimpajama清理脚本
│   ├── pdfprocess.py            # pdf批量处理流程
│   ├── step1_pretrain_basic_dataprocess.py   # 数据处理第一步：JSONL化脚本
│   ├── data_jucier/             # 在step2做所使用的数据处理工具
│   ├── final_data/              # 存放最终处理好的数据    
│   ├── step2/                                # 数据处理第二步：数据清洗脚本
│   │   ├── minidsv3_starcoder.yaml           # 各个数据自身的yaml文件
│   │   ├── minidsv3_text_ape.yaml
│   │   ├── minidsv3_text_openr1.yaml
│   │   ├── minidsv3_text_skypile.yaml
│   │   ├── minidsv3_text_slimpajama.yaml
│   │   ├── requirements_step2.txt           # 第二步运行所需的环境
│   │   ├── run_step2_ape.sh                 # 各个数据上运行DJ的sh脚本
│   │   ├── run_step2_openr1.sh
│   │   ├── run_step2_skypile.sh
│   │   ├── run_step2_slimpajama.sh
│   │   ├── run_step2_starcoder.sh
│
├── tokenizer/                  # tokenizer文件夹
│   ├── lit_gpt/                # 辅助打包的库lit_gpt
│   ├── test_tokenizer.py       # 测试tokenizer是否正常运行的代码
│   ├── tokenizer.json          # deepseek开源的tokenizer架构本身
│   ├── tokenizer_config.json   # deepseek开源的tokenizer配置文件
│   ├── step3_prepare_data_for_pretrain.py   # 数据处理第三步：Tokenizer脚本
│   ├── step3_prepare_data_for_pretrain_upsample.py   # 上采样脚本
│   ├── step4_dataset.py        # 将数据抽样、打包成bin文件的脚本

```

其中，数据的目录为`/root/autodl-tmp/minideepseek/v3/data/final_data`，里面是已经处理好的bin文件 ↓

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/153.png)

具体执行脚本为 ↓

```shell
# 退出当前环境
deactivate

# 激活mnds_training环境
cd /root/autodl-tmp/minideepseek/v3
source mnds_training/bin/activate

###################################################
####   4 x 64 vCPU Intel(R) Xeon(R) Gold 6430  ####
####   RAM 4 x 120G，4 x RTX4090 24G           ####
###################################################

# 改变deepspeed版本以支持zero-1数据并行
mkdir -p ~/.triton/autotune
pip install deepspeed==0.16.1

# 使用 DeepSpeed 分布式训练，4张GPU并行、单CPU上64线程并行、执行 10个 epoch
# 【TIME WARNING：运行中后续补充】
deepspeed --master_port 29500 --num_gpus=4 pretrain.py --epochs 10
```

| **模型规模**         | **预估训练时间** |
|----------------------|--------------------------------------|
| **0.02B参数 + 30G数据 MateConvMini**    | 15个epoch，约18~22个小时，90分钟/epoch |
| **0.15B参数 + 60G数据 MiniDeepSeek**   | 15个epoch，约4天时间，6~7小时/epoch |
| **0.3B参数 + 100G数据 MiniDeepSeek**   | 10个epoch，约1周时间，14小时/epoch|
| **0.3B参数 + 300G数据 MiniDeepSeek**   | 10个epoch，约2周时间，30小时/epoch|

数据量直接关系到一个epoch中有多少需要处理的数据、因此数据量增加会让模型的训练时间线性增加。模型规模的增加就复杂得多，一般认为模型规模增加之后、训练时间不会线性增加、而会指数级 + 线性混合增加，因此模型变大之后所需的时间会爆炸性增长。

### 9.2 预训练脚本解读

DeepSeek预训练脚本是由MateConv预训练脚本修改而来、其核心板块与MateConv预训练脚本基本一致、包括下列功能：

 - `Logger()` → 负责日志记录
 - `get_lr()` → 计算动态学习率
 - `init_model()` → 初始化模型（支持断点续训）
 - `train_epoch()` → 训练单个 epoch
 - if \_\_main_\_
 - 单独的Dataset.py和model.py用于支持模型和数据的处理

单除此之外、DeepSeek预训练脚本还增加和删除了这些板块——

1. **修改原本的DDP支持为DeepSpeed的ZeRO-1数据并行、并且将自定义并行结构全部纳入DeepSpeed框架进行考虑**，这是由于DeepSeek所使用的专家并行、Tensor并行、DualPipe等流程无法与传统的数据并行兼容、因此DeepSeek在论文中明确说明他们只使用了ZeRO-1阶段数据并行，故而在DeepSeek的预训练代码中、数据加载时也不再由pytorch的`Distributed Sampler`来控制数据读取、分布式初始化状态也不再由`torch.distributed`控制，而是交由DeepSpeed直接管理数据并行与分布式初始化。

2. **补充DeepSeek本身要求的Multi-Token Prediction功能、为模型增加更多“远见”、以增加模型对长序列的预测能力**。为此、模型结构做出变化、计算的损失函数也发生了变化，deepseek的pretrain.py拥有单独的MTP损失计算函数以及单独的MTP损失环节。

除此之外、当前的训练流程适合**数据规模大、模型复杂度高（MTP + MoE）**，同时多卡训练、显存敏感，为了更好应对分布式的需求，在全新的训练代码中，我做出了这些改进 ↓

1. 训练稳定性（尤其是显存爆炸 / loss 爆炸问题）  
2. 多卡分布式兼容性  
3. DeepSpeed ZeRO 注册的正确性  
4. 日志与调试能力  
5. batch / micro_batch 的配置方式  

具体来说 ↓

| 方面 | MateConv训练代码 | minideepseek训练代码 |
|------|------------------|------------------|
| 🔁 `batch_size` 设计 | `args.batch_size`（全局） | `micro_batch_size + accumulation_steps` |
| 💥 ZeRO参数注册 | ❌ 没有 | ✅ 完整注册所有手动实现的并行层 |
| 💾 `load_state_dict` 处理 | 简单载入 | 自动 clean `module.` 前缀，避免多卡恢复冲突 |
| 🚨 loss debug/grad 检查 | ❌ 没有 | ✅ `grad_fn` 检查、NaN/Inf 检查、loss 拆解输出 |
| 🧠 内存调试 | ❌ 无 | ✅ `print_memory()` 跟踪每阶段内存 |
| 🔎 logger 机制 | `print` | ✅ `logging` + `logger_utils` 支持调试/切换日志级别 |
| 🔁 checkpoint 自动恢复 | 有，但无优化器注册 | ✅ 模型 & 优化器恢复都包含 |
| 🔁 `train_batch_size` 参数 | `args.batch_size`（全局） | 分离 micro_batch / accumulation 更灵活 |
| ⚙️ 设备设置 / 清理 | 较粗糙 | ✅ 启动前清理 GPU 释放显存 |
| 🧮 loss all-reduce | ❌ 没有 | ✅ 使用 `reduce_loss_across_gpus` 跨卡同步 |
| 🧵 DataLoader 设置 | num_worker 较低 | ✅ `num_worker=64` + `prefetch_factor=8` |

具体脚本如下 ↓

```python
import os
import gc
import platform
import argparse
import time
import math
import warnings
import glob
import json
import deepspeed
import torch
import torch.distributed as dist
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from contextlib import nullcontext
from model.deepseekv3_mtp_model import Transformer, ColumnParallelLinear, RowParallelLinear
from model.deepseekv3_mtp_model import ModelArgs
from tokenizer.step4_dataset import PretrainDataset
from pathlib import Path
from model.deepseekv3_mtp_model import get_rank, get_world_size
from logger_utils import setup_logger
import logging

torch.autograd.set_detect_anomaly(True)

logger = setup_logger()

#不需要调试时用这一行
logger.setLevel(logging.INFO)

#需要调试时启用下面这一行
#logger.setLevel(logging.DEBUG)


logger.info("🚀 Logger initialized")

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128,expandable_segments:True"

warnings.filterwarnings('ignore')

def create_ds_config():
    # ds_config动态生成函数
    ds_config = {
        "gradient_accumulation_steps": args.accumulation_steps,
        "train_micro_batch_size_per_gpu": args.micro_batch_size,
        "bf16": {"enabled": True},
        "zero_optimization": {"stage": 1},  # ✅ 启用 ZeRO-1
        "optimizer": {
            "type": "AdamW",
            "params": {
                "lr": args.learning_rate,
                "betas": [0.9, 0.999],
                "eps": 1e-8,
                "weight_decay": 0.01
            }
        },
        "scheduler": {
            "type": "WarmupLR",
            "params": {
                "warmup_min_lr": 0,
                "warmup_max_lr": args.learning_rate,
                "warmup_num_steps": 10000
            }
        }
    }

    # ✅ 将 JSON 数据写入文件
    config_path = os.path.join(args.save_dir, "ds_config.json")
    with open(config_path, "w") as f:
        json.dump(ds_config, f, indent=4) 

    return config_path

def print_memory(tag=""):
    logger.debug(f"\n📌 CUDA Memory Status {tag}:")
    logger.debug(f"   Allocated: {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB")
    logger.debug(f"   Reserved:  {torch.cuda.memory_reserved() / 1024 ** 2:.2f} MB")
    logger.debug(f"   Max Allocated: {torch.cuda.max_memory_allocated() / 1024 ** 2:.2f} MB")
    logger.debug(f"   Max Reserved:  {torch.cuda.max_memory_reserved() / 1024 ** 2:.2f} MB\n")

def Logger_main_rank(msg):
    # ✅ 让 DeepSpeed 处理 rank
    rank = int(os.environ.get("RANK", "0"))  
    if rank == 0:
        logger.info(msg)

def get_lr(it, all):
    warmup_iters = args.warmup_iters
    lr_decay_iters = all
    min_lr = args.learning_rate / 10

    if it < warmup_iters:
        return args.learning_rate * it / warmup_iters
    if it > lr_decay_iters:
        return min_lr
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (args.learning_rate - min_lr)

def reduce_loss_across_gpus(loss):
    """对 loss 进行 all-reduce 并取平均，返回的是一个 detached 的 tensor，仅用于 log"""
    if dist.is_available() and dist.is_initialized():
        reduced_loss = loss.detach().clone()
        dist.all_reduce(reduced_loss, op=dist.ReduceOp.SUM)
        reduced_loss /= get_world_size()
        return reduced_loss
    else:
        return loss

def compute_loss(logits, mtp_logits, targets, lambda_mtp, vocab_size):
    """
    计算 DeepSeek-V3 预训练的总损失 (L = L_main + L_MTP)

    Args:
        logits: [Bs, seq_len, vocab_size] —— 主模型输出
        targets: [Bs, seq_len] —— 已偏移好的目标（dataset 中已是 input+1）
        mtp_logits: list of [Bs, seq_len-k-1, vocab_size] —— 每个 k 的 MTP 输出
        lambda_mtp: MTP 损失的权重因子 λ

    Returns:
        总损失 (scalar tensor)
    """

    logger.debug(f"[DEBUG] logits max: {logits.max().item()}, min: {logits.min().item()}")
    
    for k, mtp_logit in enumerate(mtp_logits):
        logger.debug(f"[DEBUG] mtp_logits[{k}] max: {mtp_logit.max().item()}, min: {mtp_logit.min().item()}")
        
        if torch.isnan(mtp_logit).any():
            logger.error(f"[NaN] ❌ mtp_logits[{k}] contains NaN!")

        if torch.isinf(mtp_logit).any():
            logger.warning(f"[Inf] ⚠️ mtp_logits[{k}] contains Inf!")
    
    # ====== 主 loss ======
    main_loss = F.cross_entropy(logits.reshape(-1, vocab_size), targets.reshape(-1))
    logger.debug(f"[DEBUG] main_loss: {main_loss.item()}, requires_grad: {main_loss.requires_grad}, grad_fn: {main_loss.grad_fn}")

    # ====== MTP loss ======
    mtp_loss = None
    for k, mtp_logit in enumerate(mtp_logits):
        if targets.shape[1] <= k + 1:
            continue

        mtp_target = targets[:, k + 1:]
        expected_len = mtp_logit.size(1)
        mtp_target = mtp_target[:, :expected_len]

        # 检查是否有 NaN 或非法值
        if torch.isnan(mtp_logit).any():
            logger.error(f"[ERROR] NaN detected in mtp_logits[{k}]")
            continue
        if mtp_target.max() >= vocab_size:
            logger.error(f"[ERROR] mtp_target contains token >= vocab_size at k={k}")
            continue
        
        this_loss = F.cross_entropy(mtp_logit.reshape(-1, vocab_size), mtp_target.reshape(-1))
        logger.debug(f"[DEBUG] mtp_loss[{k}]: {this_loss.item()}, requires_grad: {this_loss.requires_grad}, grad_fn: {this_loss.grad_fn}")

        if mtp_loss is None:
            mtp_loss = this_loss
        else:
            mtp_loss = mtp_loss + this_loss

    if mtp_loss is not None:
        mtp_loss = (lambda_mtp / len(mtp_logits)) * mtp_loss
    else:
        mtp_loss = torch.tensor(0.0, device=main_loss.device, dtype=main_loss.dtype)
    logger.debug(f"[DEBUG] final mtp_loss: {mtp_loss.item()}, requires_grad: {mtp_loss.requires_grad}, grad_fn: {mtp_loss.grad_fn}")

    total_loss = main_loss + mtp_loss
    logger.debug(f"[DEBUG] total_loss: {total_loss.item()}, requires_grad: {total_loss.requires_grad}, grad_fn: {total_loss.grad_fn}")

    return total_loss

def train_epoch(epoch, wandb):
    epoch_start_time = time.time()
    for step, (X, Y) in enumerate(train_loader):
        step_start_time = time.time()
        X = X.to(args.device)
        Y = Y.to(args.device)

        lr = get_lr(epoch * iter_per_epoch + step, args.epochs * iter_per_epoch)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        with ctx:
            #logger.debug("="*20 + " 当前张量显存信息 " + "="*20)
            #print_memory("Before forward")
            forward_start = time.time()
            logits, mtp_logits = model(X,Y)
            logger.debug(f"logits grad check: requires_grad = {logits.requires_grad} | grad_fn = {logits.grad_fn}")
            logger.debug(f"mtp_logits grad check: requires_grad = {mtp_logits[0].requires_grad} | grad_fn = {mtp_logits[0].grad_fn}")
            #print_memory("After forward")
            if torch.isnan(logits).any():
                logger.debug("🚨 [NaN DETECTED] logits contains NaN")
            else:
                logger.debug("✅ logits looks OK")
            forward_end_1 = time.time()
            
            for i, mtp in enumerate(mtp_logits):
                if torch.isnan(mtp).any():
                    logger.debug(f"🚨 [NaN DETECTED] mtp_logits[{i}] contains NaN")
                else:
                    logger.debug(f"✅ mtp_logits[{i}] looks OK")
            
            loss = compute_loss(logits, mtp_logits, Y
                                , args.lambda_mtp, lm_config.vocab_size) / args.accumulation_steps
            avg_loss = reduce_loss_across_gpus(loss)
            logger.debug(f"LOSS grad check: requires_grad = {loss.requires_grad} | grad_fn = {loss.grad_fn}")
            forward_end_2 = time.time()
                            
            print_memory("After loss")
            
        with torch.autograd.set_detect_anomaly(True):
            scaler.scale(loss).backward()
            backward_end = time.time()
            
        del logits, Y

        if (step + 1) % args.accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)

            scaler.step(optimizer)
            scaler.update()

            optimizer.zero_grad(set_to_none=True)
        
        update_end = time.time()
        
        if step % args.log_interval == 0:
            current_time = time.time()
            epoch_spend_time = current_time - epoch_start_time

            if step > 100:
                # ✅ 新增：估算剩余 epoch 时间
                avg_time_per_step = epoch_spend_time / (step + 1)
                remaining_steps = iter_per_epoch - step - 1
                estimated_remaining_time = avg_time_per_step * remaining_steps
                estimated_remaining_min = estimated_remaining_time / 60
            else:
                estimated_remaining_min = -1 #暂不估计

            # ✅ 打印进度
            Logger_main_rank(
                f"Epoch:[{epoch}/{args.epochs}]({step}/{iter_per_epoch}) "
                f"loss:{avg_loss.item() * args.accumulation_steps:.3f} "
                f"lr:{optimizer.param_groups[-1]['lr']:.7f} "
                f"epoch_RemainingTime:{estimated_remaining_min / 60:.1f}min"
            )

            # ✅ 可选：记录 wandb
            if (wandb is not None) and (os.environ.get("RANK", "0") == "0"):
                wandb.log({
                    "loss": loss.item() * args.accumulation_steps,
                    "lr": optimizer.param_groups[-1]['lr'],
                    "epoch_RemainingTime(min)": estimated_remaining_time / 60
                })
                
        if (step + 1) % args.save_interval == 0 and (os.environ.get("RANK", "0") == "0"):
            model.eval()
            moe_path = '_moe' if lm_config.use_moe else ''
            ckp = f'{args.save_dir}/{args.model_name}.pth'  # 使用用户提供的模型名称

            if isinstance(model, torch.nn.parallel.DistributedDataParallel):
                state_dict = model.module.state_dict()
            else:
                state_dict = model.state_dict()

            torch.save(state_dict, ckp)
            Logger_main_rank(f"保存模型到 {ckp}")
            optimizer_state_path = f'{args.save_dir}/{args.model_name}_optimizer.pth'
            torch.save(optimizer.state_dict(), optimizer_state_path)
            Logger_main_rank(f"保存优化器状态到 {optimizer_state_path}")
            
            model.train()

def init_model():
    def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

    model = Transformer(lm_config, logger=logger)
    moe_path = '_moe' if lm_config.use_moe else ''

    def clean_state_dict(state_dict):
        return {k.replace("module.", ""): v for k, v in state_dict.items()}
        
    # 加入恢复训练的逻辑
    checkpoint_path = f'{args.save_dir}/{args.model_name}.pth'
    if os.path.exists(checkpoint_path):
        Logger_main_rank(f"加载模型检查点 {checkpoint_path}")
        state_dict = torch.load(checkpoint_path, map_location=args.device)
        model.load_state_dict(clean_state_dict(state_dict), strict=True)
    else:
        Logger_main_rank(f"没有找到模型检查点，开始从头训练")

    # ✅ 注册所有你手动做了分布式的模块的参数
    def register_external_parameters(model, optimizer):
        raw_model = model.module if hasattr(model, "module") else model
        optimizer = model.optimizer  # 拿到 ZeRO 优化器

        def try_register(param):
            try:
                deepspeed.zero.register_external_parameter(optimizer,param)
            except Exception as e:
                Logger_main_rank(f"⚠️ 注册参数失败: {e}")              

        # 注册 embedding 层
        try_register(raw_model.embed.weight)

        # 注册输出 head 层
        try_register(raw_model.head.weight)

        # 注册主模型 layers 中的 parallel linear
        for layer in raw_model.layers:
            for mod in layer.modules():
                if isinstance(mod, (ColumnParallelLinear, RowParallelLinear)):
                    try_register(mod.weight)
                    if mod.bias is not None:
                        try_register(mod.bias)
                    if hasattr(mod, 'scale') and mod.scale is not None:
                        try_register(mod.scale)

        # 注册 MTP 中的 projection 和 layers
        for proj in raw_model.mtp_projections:
            for param in proj.parameters():
                try_register(param)

        for layer in raw_model.mtp_layers:
            for mod in layer.modules():
                if isinstance(mod, (ColumnParallelLinear, RowParallelLinear)):
                    try_register(mod.weight)
                    if mod.bias is not None:
                        try_register(mod.bias)
                    if hasattr(mod, 'scale') and mod.scale is not None:
                        try_register(mod.scale)

        # 注册 MoE 模块中的 gate 和 experts
        for layer in raw_model.layers:
            if hasattr(layer.ffn, 'gate'):
                gate = layer.ffn.gate
                try_register(gate.weight)
                if gate.bias is not None:
                    try_register(gate.bias)

            if hasattr(layer.ffn, 'experts'):
                for expert in layer.ffn.experts:
                    if expert is not None:
                        for param in expert.parameters():
                            try_register(param)

    # ✅ 使用 DeepSpeed ZeRO-1 数据并行
    logger.debug("🚀 Initializing model with DeepSpeed...")
    Logger_main_rank("使用 DeepSpeed ZeRO-1")
    model, optimizer, _, _ = deepspeed.initialize(
        model=model,
        config=args.deepspeed,
        model_parameters=model.parameters()
    )
    
    # ✅ 执行注册
    register_external_parameters(model, optimizer)
    logger.debug(f"NaN check passed on rank {get_rank()}")
    return model

# torchrun --nproc_per_node 2 pretrain.py
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="DeepSeek Pretraining")
    parser.add_argument("--deepspeed", type=str, default=None, help="DeepSpeed config file")
    parser.add_argument("--out_dir", type=str, default="out", help="Output directory")
    parser.add_argument("--epochs", type=int, default=20, help="Number of epochs")
    parser.add_argument("--micro_batch_size", type=int, default=2, help="Micro batch size per GPU")
    parser.add_argument("--accumulation_steps", type=int, default=6, help="Gradient accumulation steps")
    parser.add_argument("--learning_rate", type=float, default=2e-4, help="Learning rate")
    parser.add_argument("--device", type=str, default="cuda:0" if torch.cuda.is_available() else "cpu", help="Device to use")
    parser.add_argument("--dtype", type=str, default="bfloat16", help="Data type")
    parser.add_argument("--use_wandb", action="store_true", help="Use Weights & Biases")
    parser.add_argument("--wandb_project", type=str, default="DeepSeek-Pretrain", help="Weights & Biases project name")
    parser.add_argument("--num_workers", type=int, default=64, help="Number of workers for data loading")
    parser.add_argument("--data_path", type=str, default="/root/autodl-tmp/minideepseek/v3/data/final_data", help="Path to training data")
    parser.add_argument("--grad_clip", type=float, default=1.0, help="Gradient clipping threshold")
    parser.add_argument("--warmup_iters", type=int, default=0, help="Number of warmup iterations")
    parser.add_argument("--log_interval", type=int, default=100, help="Logging interval")
    parser.add_argument("--save_interval", type=int, default=1000, help="Model saving interval")
    parser.add_argument("--model_name", type=str, default="minideepseekbase", help="模型名称，用于保存和加载检查点")
    # 新增关于MTP的相关参数lambda
    parser.add_argument("--lambda_mtp", type=float, default=0.5, help="MTP loss weight λ")
    parser.add_argument("--local_rank", type=int, default=-1, help="Local rank for distributed training")

    args = parser.parse_args()

    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    lm_config = ModelArgs()
    max_seq_len = lm_config.max_seq_len
    args.save_dir = os.path.join(args.out_dir)
    
    os.makedirs(args.save_dir, exist_ok=True)
    os.makedirs(args.out_dir, exist_ok=True)
    checkpoint_path = f'{args.save_dir}/{args.model_name}.pth'

    # 动态创建DeepSpeed配置
    ds_config_path = create_ds_config()
    args.deepspeed = ds_config_path
    
    tokens_per_iter = args.micro_batch_size * max_seq_len
    torch.manual_seed(1337)
    device_type = "cuda" if "cuda" in args.device else "cpu"
    args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    args.wandb_run_name = f"minideepseek-Pretrain-Epoch-{args.epochs}-BatchSize-{args.micro_batch_size}-LearningRate-{args.learning_rate}"

    ctx = nullcontext() if device_type == "cpu" else torch.cuda.amp.autocast()
    
    if args.use_wandb and (os.environ.get("RANK", "0") == "0"):
        import wandb
        wandb.init(project=args.wandb_project, name=args.wandb_run_name)
    else:
        wandb = None

    data_path_list = sorted([str(p) for p in Path(args.data_path).glob("*.bin")])
    logger.debug(f"🔍 Looking for .bin files in: {args.data_path}")
    logger.debug(f"📦 Found {len(data_path_list)} bin files")
    for path in data_path_list[:5]:
        logger.debug(f"  - {path}")
    train_ds = PretrainDataset(data_path_list, max_length=max_seq_len, memmap=True)
    #train_sampler = DistributedSampler(train_ds) if ddp else None
    train_loader = DataLoader(
        train_ds,
        batch_size=args.micro_batch_size,
        pin_memory=True,
        drop_last=False,
        shuffle=False, # 在step4里使用shuffle了
        num_workers=args.num_workers,
        prefetch_factor=8,  # 预取 8 个 batch
        persistent_workers=True  # 让 worker 持续运行
    )

    model = init_model()
    logger.debug(f"[CHECK] head weight stats: max={model.head.weight.max().item()}, min={model.head.weight.min().item()}, mean={model.head.weight.mean().item()}")
    if hasattr(model.head, "scale") and model.head.scale is not None:
        logger.debug(f"[CHECK] head scale stats: max={model.head.scale.max().item()}, min={model.head.scale.min().item()}")
    for name, param in model.named_parameters():
        if torch.isnan(param).any():
            print(f"⚠️ Parameter {name} contains NaN!")
    # torch 2.2.0版本不支持更智能的bf16自动混合精度训练，因此在设置中只能保留float16
    # 但是由于我们是基于bf16进行训练，因此本质上AMP不会被触发
    scaler = torch.cuda.amp.GradScaler(enabled=(args.dtype == 'float16'))
    #scaler = torch.cuda.amp.GradScaler(enabled=(args.dtype in ['float16', 'bfloat16']))
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
    
    # 恢复优化器状态
    optimizer_state_path = f'{args.save_dir}/{args.model_name}_optimizer.pth'
    if os.path.exists(checkpoint_path):
	    optimizer_state_path = f'{args.save_dir}/{args.model_name}_optimizer.pth'
	    if os.path.exists(optimizer_state_path):
	        Logger_main_rank(f"加载优化器状态 {optimizer_state_path}")
	        optimizer.load_state_dict(torch.load(optimizer_state_path, map_location=args.device))
	    else:
	        Logger_main_rank(f"没有找到优化器状态，使用新的优化器")

    if False and platform.system() != 'Windows' and float(torch.__version__.split('.')[0]) >= 2:
        Logger_main_rank("compiling the model... (takes a ~minute)")
        unoptimized_model = model
        model = torch.compile(model)

    iter_per_epoch = len(train_loader)
    for epoch in range(args.epochs):
        train_epoch(epoch, wandb)
```

### 9.3 预训练模型脚本的变形

```python

import math
import os
from dataclasses import dataclass
from typing import Tuple, Optional, Literal, List
import torch
import torch.distributed as dist
import math
import torch.nn.init as init
from torch import nn
import torch.nn.functional as F
import torch.distributed as dist
from transformers import PretrainedConfig
from model.kernel import act_quant, weight_dequant, fp8_gemm
from logger_utils import setup_logger
import logging

#分块量化的块尺寸
block_size = 128

#设置矩阵乘法所使用的精度
gemm_impl: Literal["bf16", "fp8"] = "bf16"
#设置注意力的计算方法
attn_impl: Literal["naive", "absorb"] = "absorb"
#设置环境变量
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

@dataclass
class ModelArgs(PretrainedConfig):
    """
    Data class for defining model arguments and hyperparameters.

    Attributes:
        max_batch_size (int): Maximum batch size.
        max_seq_len (int): Maximum sequence length.
        dtype (Literal["bf16", "fp8"]): Data type for computations.
        vocab_size (int): Vocabulary size.
        dim (int): Model dimension.
        inter_dim (int): Intermediate dimension for MLP layers.
        moe_inter_dim (int): Intermediate dimension for MoE layers.
        n_layers (int): Number of transformer layers.
        n_dense_layers (int): Number of dense layers in the model.
        n_heads (int): Number of attention heads.
        n_routed_experts (int): Number of routed experts for MoE layers.
        n_shared_experts (int): Number of shared experts for MoE layers.
        n_activated_experts (int): Number of activated experts in MoE layers.
        n_expert_groups (int): Number of expert groups.
        n_limited_groups (int): Number of limited groups for MoE routing.
        score_func (Literal["softmax", "sigmoid"]): Scoring function for MoE routing.
        route_scale (float): Scaling factor for routing scores.
        q_lora_rank (int): LoRA self.rank for query projections.
        kv_lora_rank (int): LoRA self.rank for key-value projections.
        qk_nope_head_dim (int): Dimension for query-key projections without positional embeddings.
        qk_rope_head_dim (int): Dimension for query-key projections with rotary embeddings.
        v_head_dim (int): Dimension for value projections.
        original_seq_len (int): Original sequence length.
        rope_theta (float): Base for rotary positional encoding.
        rope_factor (float): Scaling factor for extended sequence lengths.
        beta_fast (int): Fast beta correction factor.
        beta_slow (int): Slow beta correction factor.
        mscale (float): Scaling factor for extended attention.
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # 增加kwargs.get以兼容代码中的属性引用
        self.max_batch_size = kwargs.get("max_batch_size", 8)
        self.max_seq_len = kwargs.get("max_seq_len", 4096)
        self.dtype = kwargs.get("dtype", "bf16")
        self.vocab_size = kwargs.get("vocab_size", 128000) # 按照deepseek的tokenizer重新修改
        self.dim = kwargs.get("dim", 1024) 
        self.inter_dim = kwargs.get("inter_dim", 2048)  
        self.moe_inter_dim = kwargs.get("moe_inter_dim", 512) #MoE的inter_dim
        self.n_layers = kwargs.get("n_layers", 8) 
        self.n_dense_layers = kwargs.get("n_dense_layers", 1)
        self.n_heads = kwargs.get("n_heads", 8) 
        # MoE
        self.use_moe = kwargs.get("use_moe", 1)
        self.n_routed_experts = kwargs.get("n_routed_experts", 4) #必须是gpu的倍数 
        self.n_shared_experts = kwargs.get("n_shared_experts", 1)  
        self.n_activated_experts = kwargs.get("n_activated_experts", 2) 
        self.n_expert_groups = kwargs.get("n_expert_groups", 1)
        self.n_limited_groups = kwargs.get("n_limited_groups", 1)
        self.score_func = kwargs.get("score_func", "softmax")
        self.route_scale = kwargs.get("route_scale", 1.0)
        # MLA
        self.q_lora_rank = kwargs.get("q_lora_rank", 0)
        self.kv_lora_rank = kwargs.get("kv_lora_rank", 256)  
        self.qk_nope_head_dim = kwargs.get("qk_nope_head_dim", 64)  
        self.qk_rope_head_dim = kwargs.get("qk_rope_head_dim", 32)  
        self.v_head_dim = kwargs.get("v_head_dim", 64)
        # YARN
        self.original_seq_len = kwargs.get("original_seq_len", 4096)
        self.rope_theta = kwargs.get("rope_theta", 10000.0)
        self.rope_factor = kwargs.get("rope_factor", 40)
        self.beta_fast = kwargs.get("beta_fast", 32)
        self.beta_slow = kwargs.get("beta_slow", 1)
        self.mscale = kwargs.get("mscale", 1.0)
        # MTP
        self.n_mtp_depths = kwargs.get("n_mtp_depths", 2)

def get_rank_world_size():
    if dist.is_initialized():
        return dist.get_rank(), dist.get_world_size()
    elif "RANK" in os.environ and "WORLD_SIZE" in os.environ:
        return int(os.environ["RANK"]), int(os.environ["WORLD_SIZE"])
    else:
        return 0, 1

def get_rank():
    return get_rank_world_size()[0]

def get_world_size():
    return get_rank_world_size()[1]

def get_logger(logger=None):
    return logger or logging.getLogger("fallback_logger")

def print_memory(tag=""):
    print(f"\n📌 CUDA Memory Status {tag}:")
    print(f"   Allocated: {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB")
    print(f"   Reserved:  {torch.cuda.memory_reserved() / 1024 ** 2:.2f} MB")
    print(f"   Max Allocated: {torch.cuda.max_memory_allocated() / 1024 ** 2:.2f} MB")
    print(f"   Max Reserved:  {torch.cuda.max_memory_reserved() / 1024 ** 2:.2f} MB\n")

class AllGatherWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        gathered = [torch.zeros_like(input) for _ in range(torch.distributed.get_world_size())]
        torch.distributed.all_gather(gathered, input)
        ctx.rank = torch.distributed.get_rank()
        ctx.world_size = torch.distributed.get_world_size()
        return torch.cat(gathered, dim=-1)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.chunk(ctx.world_size, dim=-1)[ctx.rank]

class AllReduceWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        torch.distributed.all_reduce(input)
        return input

    @staticmethod
    def backward(ctx, grad_output):
        # 反向传播时，同样 all_reduce，保持一致性
        torch.distributed.all_reduce(grad_output)
        return grad_output

#===================
# 1. 遵循张量并行的Embedding层
#===================

class ParallelEmbedding(nn.Module):
    """
    Embedding layer with parallelism support across distributed processes.

    Args:
        vocab_size (int): Vocabulary size.
        dim (int): Embedding dimension.
    """
    
    dtype = torch.bfloat16 if gemm_impl == "bf16" else torch.float8_e4m3fn
    
    def __init__(self, vocab_size: int, dim: int, logger=None):
        super().__init__()
        self.logger = get_logger(logger)
        self.rank = get_rank()
        self.world_size = get_world_size()
        self.vocab_size = vocab_size
        self.dim = dim

        #将需要向量化的token分到不同GPU上
        assert self.vocab_size % self.world_size == 0, f"Vocabulary size must be divisible by world size (world_size={self.world_size})"
        self.part_vocab_size = (self.vocab_size // self.world_size)

        #划分为 [0,500],[500,1000],[1000,1500]这样的区间
        self.vocab_start_idx = self.rank * self.part_vocab_size
        self.vocab_end_idx = self.vocab_start_idx + self.part_vocab_size

        #embedding层的权重等于一个GPU上的vocab_size乘以相应的权重
        self.weight = nn.Parameter(torch.empty(self.part_vocab_size, self.dim, dtype=self.dtype or Linear.dtype))
        nn.init.normal_(self.weight, mean=0.0, std=0.02)
        self.logger.debug(f"[DEBUG] ParallelEmbedding weight initialized: max={self.weight.max().item()}, min={self.weight.min().item()}, mean={self.weight.mean().item()}")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        assert x.max().item() < self.vocab_size, \
            f"Token ID {x.max().item()} 超过 vocab_size {self.vocab_size}"
    
        if self.world_size > 1:
            mask = (x >= self.vocab_start_idx) & (x < self.vocab_end_idx)
    
            # 初始化输出
            y = torch.zeros(x.shape + (self.dim,), device=x.device, dtype=self.weight.dtype)
    
            # 只处理 mask 内合法的局部 index
            y[mask] = F.embedding(x[mask] - self.vocab_start_idx, self.weight)
    
            dist.all_reduce(y)
        else:
            y = F.embedding(x, self.weight)
        return y

#===================
# 2. 支持混合精度 + 细粒度量化操作的线性函数与线性层
#===================

def linear(x: torch.Tensor, weight: torch.Tensor, bias: Optional[torch.Tensor] = None) -> torch.Tensor:
    """
    自定义的、能够实现FP8、BF16、FP16、FP32各种精度之间的转化和GEMM的线性运算函数

    这个函数的输入与nn.Linear层不同，函数需的是：
    - 需要被线性变换的数据X，以及
    - 线性变换所使用的权重weights，此时weights的结构就等同于
    
    补充：BF16和FP16的区别在于、都是16位数，但BF16是8个指数7个尾数（8E7M），FP16是5E10M，指数越大可表示的数值范围越大、尾数越大越精确。因此，BF16范围更大适用于梯度计算、激活值存储等场景、可以稳定训练；FP16更加精确、适用于权重存储等存储场景，更适合保存信息。
    """
    # 如果字节大于1（精度大于fp8），则应该是fp16或者fp32、直接使用非量化计算
    if weight.element_size() > 1:
        return F.linear(x, weight, bias)
    # 如果GEMM精度是bf16，那要将权重解量化为bf16
    # 当我们默认训练使用BF16的时候，不会进入elif
    # 下面的elif只能适应推理流程、不能用于训练
    elif gemm_impl == "bf16":
        weight = weight_dequant(weight, weight.scale)
        return F.linear(x, weight, bias)
    # 如果element_size不大于1，同时gemm_impl不是bf16（也就是fp8）
    # 就说明是支持fp8计算的，则无论什么数据输入都量化成fp8
    # 使用fp8精度进行fp8的GEMM
    else:
        x, scale = act_quant(x, block_size)
        y = fp8_gemm(x, scale, weight, weight.scale)
        if bias is not None:
            y += bias
        return y

class Linear(nn.Module):
    """
    自定义的、能够支持FP8精度的细粒度量化（主要是创建缩放因子进行缩放）的线性层。

    Args:
        in_features (int): Number of input features.
        out_features (int): Number of output features.
        bias (bool): Whether to include a bias term. Defaults to False.
        dtype (optional): Data type for the layer. Defaults to `torch.bfloat16`.
    """
    dtype = torch.bfloat16 if gemm_impl == "bf16" else torch.float8_e4m3fn

    def __init__(self, in_features: int, out_features: int, bias: bool = False, dtype = None, logger=None):
        super().__init__()
        self.logger = get_logger(logger)
        self.rank = get_rank()
        self.world_size = get_world_size()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.empty(out_features, in_features, dtype=self.dtype or Linear.dtype or Linear.dtype))

        # 权重初始化：kaiming_uniform
        with torch.no_grad():
            tmp_weight = torch.empty_like(self.weight, dtype=torch.float32)
            init.xavier_uniform_(tmp_weight, gain=1.0 / math.sqrt(2))
            self.weight.copy_(tmp_weight.to(self.weight.dtype))
            if torch.isnan(self.weight).any():
                self.logger.warning(f"[WARNING] Linear weight contains NaN after kaiming init! rank={self.rank}")
        
        # 如果字节 == 1，即weight精度是FP8，则执行归一化
        # 将FP8权重缩放到FP8的动态范围内、则会需要对每一个“块”构建缩放因子
        # 其中block_size是我们规定的“块”的大小，用下面的公式来确认缩放因子的具体数量
        if self.weight.element_size() == 1:
            scale_out_features = (out_features + block_size - 1) // block_size
            scale_in_features = (in_features + block_size - 1) // block_size
            self.weight.scale = self.scale = nn.Parameter(torch.empty(scale_out_features, scale_in_features, dtype=torch.float32))
        else:
            #如果不是fp8，则直接将scale设置为None来表示该参数不存在
            self.register_parameter("scale", None)
        if bias:
            self.bias = nn.Parameter(torch.empty(self.out_features),dtype=self.dtype or Linear.dtype)
            with torch.no_grad():
                init.zeros_(self.bias)
        else:
            self.register_parameter("bias", None)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass for the custom linear layer.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Transformed tensor after linear computation.
        """
        return linear(x, self.weight, self.bias)

#===================
# 3. 支持分布式并行（张量并行）的线性层
#===================

class ColumnParallelLinear(Linear):
    """
    具有列并行功能的线性层，将输出的features分布到不同的进程（GPU）上完成。

    Args:
        in_features (int): Number of input features.
        out_features (int): Total number of output features.
        bias (bool): Whether to include a bias term. Defaults to False.
        dtype (optional): Data type for the layer. Defaults to `torch.bfloat16`.
        gather_output (bool): 是否在前向中聚合输出（保持梯度计算图）
    """
    dtype = torch.bfloat16 if gemm_impl == "bf16" else torch.float8_e4m3fn
    
    def __init__(self, in_features: int, out_features: int, bias: bool = False, dtype=None, gather_output: bool = False, logger=None):
        self.logger = get_logger(logger)
        self.rank = get_rank()
        self.world_size = get_world_size()
        self.gather_output = gather_output

        assert out_features % self.world_size == 0, \
            f"Output features must be divisible by world size (world_size={self.world_size})"
        self.part_out_features = out_features // self.world_size

        super().__init__(in_features, self.part_out_features, bias, dtype)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass for column parallel linear layer.

        Args:
            x (torch.Tensor): Input tensor. Shape: [B, T, in_features]

        Returns:
            torch.Tensor: Output tensor. If gather_output=True, shape = [B, T, out_features]
                          Otherwise, shape = [B, T, out_features / world_size]
        """
        y = F.linear(x, self.weight, self.bias)  # y: [B, T, out_features / world_size]
        #self.logger.debug(f"[DEBUG] ColumnParallelLinear rank {self.rank} output shape: {y.shape}")

        if self.gather_output and self.world_size > 1:
            y = AllGatherWithGrad.apply(y)
        return y

class RowParallelLinear(Linear):
    """
    具有行并行功能的线性层，将输入的features分布到不同的进程（GPU）上完成。

    Args:
        in_features (int): Total number of input features.
        out_features (int): Number of output features.
        bias (bool): Whether to include a bias term. Defaults to False.
        dtype (optional): Data type for the layer. Defaults to `torch.bfloat16`.
    """
    dtype = torch.bfloat16 if gemm_impl == "bf16" else torch.float8_e4m3fn
    
    def __init__(self, in_features: int, out_features: int, bias: bool = False, dtype = None, logger=None):
        self.logger = get_logger(logger)
        self.rank = get_rank()
        self.world_size = get_world_size()
        assert in_features % self.world_size == 0, f"Input features must be divisible by world size (world_size={world_size})"
        self.part_in_features = in_features // self.world_size
        super().__init__(self.part_in_features, out_features, bias, dtype)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass for row parallel linear layer.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Transformed tensor with row-parallel computation.
        """
        #print("Row",self.rank, self.weight.shape)
        y = linear(x, self.weight)

        #如果是多线程运行，还需要进行all_reduce聚合，并加上偏置
        if self.world_size > 1:
            y = AllReduceWithGrad.apply(y)
        if self.bias is not None:
            y += self.bias
        return y

#===================
# 4. 并行化的RMS Norm 根均方归一化
#===================

class RMSNorm(nn.Module):
    """
    huggngface-style RMSNorm
    """
    dtype = torch.bfloat16 if gemm_impl == "bf16" else torch.float8_e4m3fn
    
    def __init__(self, dim, eps=1e-5, logger=None):
        super().__init__()
        self.logger = get_logger(logger)
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim,dtype=self.dtype or Linear.dtype))

    def forward(self, x):
        norm = x.pow(2).mean(-1, keepdim=True)
        x = x / torch.sqrt(norm + self.eps)
        return self.weight * x

#===================
# 5. 旋转位置编码
# deepseek从序列长度、频率生成、平滑校正等多个角度改进了旋转位置编码
# 其核心与llama所使用的旋转位置编码相同
#===================

def precompute_freqs_cis(args: ModelArgs) -> torch.Tensor:
    """
    对旋转位置变啊中所需要的指数权重进行预计算，方便后续使用。

    Args:
        args (ModelArgs): Model arguments containing positional embedding parameters.

    Returns:
        torch.Tensor: Precomputed complex exponential values for positional embeddings.
    """
    dim = args.qk_rope_head_dim
    seqlen = args.max_seq_len
    beta_fast = args.beta_fast
    beta_slow = args.beta_slow
    base = args.rope_theta
    factor = args.rope_factor

    def find_correction_dim(num_rotations, dim, base, max_seq_len):
        """
        Computes the correction dimension for a given number of rotations in the rotary positional embedding.

        Args:
            num_rotations (float): Number of rotations to compute the correction for.
            dim (int): Dimensionality of the embedding space.
            base (float): Base value for the exponential computation.
            max_seq_len (int): Maximum sequence length.

        Returns:
            float: The correction dimension based on the input parameters.
        """
        return dim * math.log(max_seq_len / (num_rotations * 2 * math.pi)) / (2 * math.log(base))

    def find_correction_range(low_rot, high_rot, dim, base, max_seq_len):
        """
        Computes the range of correction dimensions for rotary positional embeddings.

        Args:
            low_rot (float): Lower bound for the number of rotations.
            high_rot (float): Upper bound for the number of rotations.
            dim (int): Dimensionality of the embedding space.
            base (float): Base value for the exponential computation.
            max_seq_len (int): Maximum sequence length.

        Returns:
            Tuple[int, int]: The range of correction dimensions (low, high), clamped to valid indices.
        """
        low = math.floor(find_correction_dim(low_rot, dim, base, max_seq_len))
        high = math.ceil(find_correction_dim(high_rot, dim, base, max_seq_len))
        return max(low, 0), min(high, dim-1)

    def linear_ramp_factor(min, max, dim):
        """
        Computes a linear ramp function used to smooth values between a minimum and maximum range.

        Args:
            min (float): Minimum value for the ramp function.
            max (float): Maximum value for the ramp function.
            dim (int): Dimensionality of the ramp tensor.

        Returns:
            torch.Tensor: A tensor of shape (dim,) with values linearly interpolated between 0 and 1,
                clamped to the range [0, 1].
        """
        if min == max:
            max += 0.001
        linear_func = (torch.arange(dim, dtype=torch.float32) - min) / (max - min)
        ramp_func = torch.clamp(linear_func, 0, 1)
        return ramp_func

    freqs = 1.0 / (base ** (torch.arange(0, dim, 2, dtype=torch.float32) / dim))
    if seqlen > args.original_seq_len:
        low, high = find_correction_range(beta_fast, beta_slow, dim, base, args.original_seq_len)
        smooth = 1 - linear_ramp_factor(low, high, dim // 2)
        freqs = freqs / factor * (1 - smooth) + freqs * smooth

    t = torch.arange(seqlen)
    freqs = torch.outer(t, freqs)
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs)
    return freqs_cis


def apply_rotary_emb(x: torch.Tensor, freqs_cis: torch.Tensor) -> torch.Tensor:
    """
    Applies rotary positional embeddings to the input tensor.

    Args:
        x (torch.Tensor): Input tensor with positional embeddings to be applied.
        freqs_cis (torch.Tensor): Precomputed complex exponential values for positional embeddings.

    Returns:
        torch.Tensor: Tensor with rotary embeddings applied.
    """
    dtype = x.dtype
    x = torch.view_as_complex(x.float().view(*x.shape[:-1], -1, 2))
    freqs_cis = freqs_cis.view(1, x.size(1), 1, x.size(-1))
    y = torch.view_as_real(x * freqs_cis).flatten(3)
    return y.to(dtype)

#===================
# 6. MLA 潜在注意力机制
#===================

class MLA(nn.Module):
    """
    Multi-Headed Attention Layer (MLA).

    Attributes:
        dim (int): Dimensionality of the input features.
        n_heads (int): Number of attention heads.
        n_local_heads (int): Number of local attention heads for distributed systems.
        q_lora_rank (int): self.rank for low-rank query projection.
        kv_lora_rank (int): self.rank for low-rank key/value projection.
        qk_nope_head_dim (int): Dimensionality of non-positional query/key projections.
        qk_rope_head_dim (int): Dimensionality of rotary-positional query/key projections.
        qk_head_dim (int): Total dimensionality of query/key projections.
        v_head_dim (int): Dimensionality of value projections.
        softmax_scale (float): Scaling factor for softmax in attention computation.
    """
    dtype = torch.bfloat16 if gemm_impl == "bf16" else torch.float8_e4m3fn
    
    def __init__(self, args: ModelArgs, logger=None):
        super().__init__()
        self.logger = get_logger(logger)
        self.rank = get_rank()
        self.world_size = get_world_size()
        self.dim = args.dim
        self.n_heads = args.n_heads
        self.n_local_heads = args.n_heads // self.world_size
        self.q_lora_rank = args.q_lora_rank
        self.kv_lora_rank = args.kv_lora_rank
        self.qk_nope_head_dim = args.qk_nope_head_dim
        self.qk_rope_head_dim = args.qk_rope_head_dim
        self.qk_head_dim = args.qk_nope_head_dim + args.qk_rope_head_dim
        self.v_head_dim = args.v_head_dim

        if self.q_lora_rank == 0:
            self.wq = ColumnParallelLinear(self.dim, self.n_heads * self.qk_head_dim, logger=self.logger)
            self.logger.debug("[DEBUG] wq weight NaN: %s", torch.isnan(self.wq.weight).any())
        else:
            self.wq_a = Linear(self.dim, self.q_lora_rank, logger=self.logger)
            self.q_norm = RMSNorm(self.q_lora_rank, logger=self.logger)
            self.wq_b = ColumnParallelLinear(self.q_lora_rank, self.n_heads * self.qk_head_dim, logger=self.logger)
        self.wkv_a = Linear(self.dim, self.kv_lora_rank + self.qk_rope_head_dim, logger=self.logger)
        self.kv_norm = RMSNorm(self.kv_lora_rank, logger=self.logger)
        self.wkv_b = ColumnParallelLinear(self.kv_lora_rank, self.n_heads * (self.qk_nope_head_dim + self.v_head_dim), logger=self.logger)
        self.wo = RowParallelLinear(self.n_heads * self.v_head_dim, self.dim, logger=self.logger)
        self.softmax_scale = self.qk_head_dim ** -0.5
        if args.max_seq_len > args.original_seq_len:
            mscale = 0.1 * args.mscale * math.log(args.rope_factor) + 1.0
            self.softmax_scale = self.softmax_scale * mscale * mscale

        if attn_impl == "naive":
            self.register_buffer("k_cache", torch.zeros(args.max_batch_size, args.max_seq_len, self.n_local_heads, self.qk_head_dim), persistent=False)
            self.register_buffer("v_cache", torch.zeros(args.max_batch_size, args.max_seq_len, self.n_local_heads, self.v_head_dim), persistent=False)
        else:
            self.register_buffer("kv_cache", torch.zeros(args.max_batch_size, args.max_seq_len, self.kv_lora_rank), persistent=False)
            self.register_buffer("pe_cache", torch.zeros(args.max_batch_size, args.max_seq_len, self.qk_rope_head_dim), persistent=False)

    def forward(self, x: torch.Tensor, start_pos: int, freqs_cis: torch.Tensor, mask: Optional[torch.Tensor]):
        """
        Forward pass for the Multi-Headed Attention Layer (MLA).

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, seq_len, dim).
            start_pos (int): Starting position in the sequence for caching.
            freqs_cis (torch.Tensor): Precomputed complex exponential values for rotary embeddings.
            mask (Optional[torch.Tensor]): Mask tensor to exclude certain positions from attention.

        Returns:
            torch.Tensor: Output tensor with the same shape as the input.
        """
        bsz, seqlen, _ = x.size()
        end_pos = start_pos + seqlen
    
        # 1. Q projection
        if self.q_lora_rank == 0:
            q = self.wq(x)
        else:
            q = self.wq_b(self.q_norm(self.wq_a(x)))
        self.logger.debug("[DEBUG] q before nope NaN: %s, inf: %s", torch.isnan(q).any(), torch.isinf(q).any())
        q = q.view(bsz, seqlen, self.n_local_heads, self.qk_head_dim)
        q_nope, q_pe = torch.split(q, [self.qk_nope_head_dim, self.qk_rope_head_dim], dim=-1)
        q_pe = apply_rotary_emb(q_pe, freqs_cis)
    
        # 2. KV projection
        kv = self.wkv_a(x)
        kv, k_pe = torch.split(kv, [self.kv_lora_rank, self.qk_rope_head_dim], dim=-1)
        k_pe = apply_rotary_emb(k_pe.unsqueeze(2), freqs_cis)
    
        if self.training:
            # === Training Mode ===
            q = torch.cat([q_nope, q_pe], dim=-1)
            kv = self.wkv_b(self.kv_norm(kv))
            kv = kv.view(bsz, seqlen, self.n_local_heads, self.qk_nope_head_dim + self.v_head_dim)
            k_nope, v = torch.split(kv, [self.qk_nope_head_dim, self.v_head_dim], dim=-1)
            k = torch.cat([k_nope, k_pe.expand(-1, -1, self.n_local_heads, -1)], dim=-1)

            if torch.isnan(q).any():
                logger.error("[NaN] ❌ Q input contains NaN before einsum")
            scores = torch.einsum("bshd,bthd->bsht", q, k) * self.softmax_scale
            if mask is not None:
                scores += mask.unsqueeze(1)
            scores = scores.softmax(dim=-1, dtype=torch.float32).type_as(x)
            x = torch.einsum("bsht,bthd->bshd", scores, v)
    
        else:
            # === Inference Mode ===
            if attn_impl == "naive":
                q = torch.cat([q_nope, q_pe], dim=-1)
                kv = self.wkv_b(self.kv_norm(kv))
                kv = kv.view(bsz, seqlen, self.n_local_heads, self.qk_nope_head_dim + self.v_head_dim)
                k_nope, v = torch.split(kv, [self.qk_nope_head_dim, self.v_head_dim], dim=-1)
                k = torch.cat([k_nope, k_pe.expand(-1, -1, self.n_local_heads, -1)], dim=-1)
                self.k_cache[:bsz, start_pos:end_pos] = k.detach()
                self.v_cache[:bsz, start_pos:end_pos] = v.detach()
    
                scores = torch.einsum("bshd,bthd->bsht", q, self.k_cache[:bsz, :end_pos]) * self.softmax_scale
            else:
                wkv_b = self.wkv_b.weight if self.wkv_b.scale is None else weight_dequant(self.wkv_b.weight, self.wkv_b.scale, block_size)
                wkv_b = wkv_b.view(self.n_local_heads, self.qk_nope_head_dim + self.v_head_dim, self.kv_lora_rank)
                q_nope = torch.einsum("bshd,hdc->bshc", q_nope, wkv_b[:, :self.qk_nope_head_dim])
                self.kv_cache[:bsz, start_pos:end_pos] = self.kv_norm(kv).detach()
                self.pe_cache[:bsz, start_pos:end_pos] = k_pe.squeeze(2).detach()
    
                scores = (torch.einsum("bshc,btc->bsht", q_nope, self.kv_cache[:bsz, :end_pos]) +
                          torch.einsum("bshr,btr->bsht", q_pe, self.pe_cache[:bsz, :end_pos])) * self.softmax_scale
    
            if mask is not None:
                scores += mask.unsqueeze(1)
            scores = scores.softmax(dim=-1, dtype=torch.float32).type_as(x)
    
            if attn_impl == "naive":
                x = torch.einsum("bsht,bthd->bshd", scores, self.v_cache[:bsz, :end_pos])
            else:
                x = torch.einsum("bsht,btc->bshc", scores, self.kv_cache[:bsz, :end_pos])
                x = torch.einsum("bshc,hdc->bshd", x, wkv_b[:, -self.v_head_dim:])
    
        x = self.wo(x.flatten(2))
        return x

#===================
# 7. DeepSeekMOE
#===================

class MLP(nn.Module):
    """
    支持张量并行的SwiGLU FFN前馈神经网络，并行方式为切分features，用于支持共享专家的构建。
    Multi-Layer Perceptron (MLP) used as a feed-forward layer.

    Attributes:
        w1 (nn.Module): Linear layer for input-to-hidden transformation.
        w2 (nn.Module): Linear layer for hidden-to-output transformation.
        w3 (nn.Module): Additional linear layer for feature transformation.
    """
    dtype = torch.bfloat16 if gemm_impl == "bf16" else torch.float8_e4m3fn
    
    def __init__(self, dim: int, inter_dim: int, logger=None):
        """
        Initializes the MLP layer.

        Args:
            dim (int): Input and output dimensionality.
            inter_dim (int): Hidden layer dimensionality.
        """
        super().__init__()
        self.logger = get_logger(logger)
        self.rank = get_rank()
        self.world_size = get_world_size()
        self.w1 = ColumnParallelLinear(dim, inter_dim, logger=self.logger)
        self.w2 = RowParallelLinear(inter_dim, dim, logger=self.logger)
        self.w3 = ColumnParallelLinear(dim, inter_dim, logger=self.logger)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass for the MLP layer.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Output tensor after MLP computation.
        """
        return self.w2(F.silu(self.w1(x)) * self.w3(x))

class Gate(nn.Module):
    """
    Gating mechanism for routing inputs in a mixture-of-experts (MoE) model.

    Attributes:
        dim (int): Dimensionality of input features.
        topk (int): Number of top experts activated for each input.
        n_groups (int): Number of groups for routing.
        topk_groups (int): Number of groups to route inputs to.
        score_func (str): Scoring function ('softmax' or 'sigmoid').
        route_scale (float): Scaling factor for routing weights.
        weight (torch.nn.Parameter): Learnable weights for the gate.
        bias (Optional[torch.nn.Parameter]): Optional bias term for the gate.
    """

    dtype = torch.bfloat16 if gemm_impl == "bf16" else torch.float8_e4m3fn
    
    def __init__(self, args: ModelArgs, logger=None):
        """
        Initializes the Gate module.

        Args:
            args (ModelArgs): Model arguments containing gating parameters.
        """
        super().__init__()
        self.logger = get_logger(logger)
        self.rank = get_rank()
        self.world_size = get_world_size()
        self.dim = args.dim
        self.topk = args.n_activated_experts
        self.n_groups = args.n_expert_groups
        self.topk_groups = args.n_limited_groups
        self.score_func = args.score_func
        self.route_scale = args.route_scale
        self.weight = nn.Parameter(torch.empty(args.n_routed_experts, args.dim, dtype=self.dtype or Linear.dtype))
        with torch.no_grad():
            tmp = torch.empty_like(self.weight, dtype=torch.float32)
            init.kaiming_uniform_(tmp, a=math.sqrt(5))
            self.weight.copy_(tmp.to(self.dtype or Linear.dtype))
            if torch.isnan(self.weight).any():
                self.logger.warning(f"[WARNING] Gate weight contains NaN! rank={self.rank}")
        
        if self.dim == 7168:
            self.bias = nn.Parameter(torch.empty(args.n_routed_experts, dtype=self.dtype or Linear.dtype))
            with torch.no_grad():
                init.zeros_(self.bias)
        else:
            self.bias = None

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass for the gating mechanism.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            Tuple[torch.Tensor, torch.Tensor]: Routing weights and selected expert indices.
        """
        scores = linear(x, self.weight)
        if self.score_func == "softmax":
            scores = scores.to(torch.float32).softmax(dim=-1).to(x.dtype)
        else:
            scores = scores.sigmoid()
        original_scores = scores
        if self.bias is not None:
            scores = scores + self.bias
        if self.n_groups > 1:
            scores = scores.view(x.size(0), self.n_groups, -1)
            if self.bias is None:
                group_scores = scores.amax(dim=-1)
            else:
                group_scores = scores.topk(2, dim=-1)[0].sum(dim=-1)
            indices = group_scores.topk(self.topk_groups, dim=-1)[1]
            mask = scores.new_ones(x.size(0), self.n_groups, dtype=bool).scatter_(1, indices, False)
            scores = scores.masked_fill_(mask.unsqueeze(-1), float("-inf")).flatten(1)
        indices = torch.topk(scores, self.topk, dim=-1)[1]
        weights = original_scores.gather(1, indices)
        if self.score_func == "sigmoid":
            weights /= weights.sum(dim=-1, keepdim=True)
        weights *= self.route_scale
        self.logger.debug(f"[DEBUG] Gate weights grad_fn: {weights.grad_fn}, requires_grad: {weights.requires_grad}")
        return weights.type_as(x), indices

class Expert(nn.Module):
    """
    Expert layer for Mixture-of-Experts (MoE) models.

    Attributes:
        w1 (nn.Module): Linear layer for input-to-hidden transformation.
        w2 (nn.Module): Linear layer for hidden-to-output transformation.
        w3 (nn.Module): Additional linear layer for feature transformation.
    """
    dtype = torch.bfloat16 if gemm_impl == "bf16" else torch.float8_e4m3fn
    
    def __init__(self, dim: int, inter_dim: int, logger=None):
        """
        Initializes the Expert layer.

        Args:
            dim (int): Input and output dimensionality.
            inter_dim (int): Hidden layer dimensionality.
        """
        super().__init__()
        self.logger = get_logger(logger)
        self.rank = get_rank()
        self.world_size = get_world_size()
        self.w1 = Linear(dim, inter_dim, logger=self.logger)
        self.w2 = Linear(inter_dim, dim, logger=self.logger)
        self.w3 = Linear(dim, inter_dim, logger=self.logger)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass for the Expert layer.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Output tensor after expert computation.
        """
        return self.w2(F.silu(self.w1(x)) * self.w3(x))


class MoE(nn.Module):
    """
    Mixture-of-Experts (MoE) module.

    Attributes:
        dim (int): Dimensionality of input features.
        n_routed_experts (int): Total number of experts in the model.
        n_local_experts (int): Number of experts handled locally in distributed systems.
        n_activated_experts (int): Number of experts activated for each input.
        gate (nn.Module): Gating mechanism to route inputs to experts.
        experts (nn.ModuleList): List of expert modules.
        shared_experts (nn.Module): Shared experts applied to all inputs.
    """
    dtype = torch.bfloat16 if gemm_impl == "bf16" else torch.float8_e4m3fn

    def __init__(self, args: ModelArgs, logger=None):
        super().__init__()
        self.logger = get_logger(logger)
        self.rank = get_rank()
        self.world_size = get_world_size()
        self.dim = args.dim

        assert args.n_routed_experts % self.world_size == 0, \
            f"Number of experts must be divisible by world size (world_size={self.world_size})"

        self.n_routed_experts = args.n_routed_experts
        self.n_local_experts = args.n_routed_experts // self.world_size
        self.n_activated_experts = args.n_activated_experts

        self.experts_start_idx = self.rank * self.n_local_experts
        self.experts_end_idx = self.experts_start_idx + self.n_local_experts

        self.gate = Gate(args, logger=self.logger)
        self.experts = nn.ModuleList([
            Expert(args.dim, args.moe_inter_dim, logger=self.logger)
            if self.experts_start_idx <= i < self.experts_end_idx else None
            for i in range(self.n_routed_experts)
        ])
        self.shared_experts = MLP(args.dim, args.n_shared_experts * args.moe_inter_dim, logger=self.logger)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.training:
            return self._forward_train(x)
        else:
            with torch.no_grad():
                return self._forward_infer(x)

    def _forward_train(self, x: torch.Tensor) -> torch.Tensor:
        shape = x.size()
        x = x.view(-1, self.dim)
        weights, indices = self.gate(x)  # shape: [B*T, topk]
        y = torch.zeros_like(x)

        for i in range(self.experts_start_idx, self.experts_end_idx):
            expert = self.experts[i]
            idx, top = torch.where(indices == i)
            if idx.numel() == 0:
                continue
            x_selected = x[idx]
            self.logger.debug(f"[Expert {i}] x_selected stats: max={x_selected.max().item():.4f}, min={x_selected.min().item():.4f}")
            expert_out = expert(x_selected)
            if torch.isnan(expert_out).any():
                self.logger.error("[NaN] ❌ expert_out contains NaN before index_add")
            self.logger.debug(f"[Expert {i}] weights stats: max={weights[idx, top].max().item():.4f}, min={weights[idx, top].min().item():.4f}")
            y = y.index_add(0, idx, expert_out * weights[idx, top].unsqueeze(-1))

        z = self.shared_experts(x)

        if self.world_size > 1:
            dist.all_reduce(y)

        return (y + z).view(shape)

    def _forward_infer(self, x: torch.Tensor) -> torch.Tensor:
        shape = x.size()
        x = x.view(-1, self.dim)
        weights, indices = self.gate(x)

        flat_indices = indices.view(-1)
        flat_weights = weights.view(-1)
        total_experts = self.n_routed_experts

        counts = torch.bincount(flat_indices, minlength=total_experts)
        tokens_per_expert = counts.tolist()
        idxs = flat_indices.argsort()
        sorted_x = x[idxs]

        local_tokens = []
        for i in range(self.experts_start_idx, self.experts_end_idx):
            num_tokens = tokens_per_expert[i]
            if num_tokens == 0:
                continue
            start_idx = sum(tokens_per_expert[:i])
            end_idx = start_idx + num_tokens
            tokens_for_expert = sorted_x[start_idx:end_idx]
            expert_out = self.experts[i](tokens_for_expert)
            local_tokens.append(expert_out)

        local_output = torch.cat(local_tokens, dim=0) if local_tokens else sorted_x.new_empty(0)

        output_gather = [torch.empty_like(local_output) for _ in range(self.world_size)]
        dist.all_gather(output_gather, local_output)
        gathered_output = torch.cat(output_gather, dim=0)

        gathered_x = torch.empty_like(gathered_output)
        gathered_x[idxs] = gathered_output

        final_out = (
            gathered_x.view(*indices.shape, -1)
            .type(weights.dtype)
            .mul_(weights.unsqueeze(-1))
            .sum(dim=1)
            .type(x.dtype)
        )

        z = self.shared_experts(x)

        if self.world_size > 1:
            dist.all_reduce(final_out)

        return (final_out + z).view(shape)

#===================
# 8. Decoder layers
#===================

class Block(nn.Module):
    """
    Transformer block combining attention and feed-forward layers.

    Attributes:
        attn (nn.Module): Attention layer (MLA).
        ffn (nn.Module): Feed-forward network (MLP or MoE).
        attn_norm (nn.Module): Layer normalization for attention.
        ffn_norm (nn.Module): Layer normalization for feed-forward network.
    """
    dtype = torch.bfloat16 if gemm_impl == "bf16" else torch.float8_e4m3fn
    
    def __init__(self, layer_id: int, args: ModelArgs, logger=None):
        """
        Initializes the Transformer block.

        Args:
            layer_id (int): Layer index in the transformer.
            args (ModelArgs): Model arguments containing block parameters.
        """
        super().__init__()
        self.logger = get_logger(logger)
        self.rank = get_rank()
        self.layer_id = layer_id
        self.world_size = get_world_size()
        self.attn = MLA(args, logger=self.logger)
        self.ffn = MLP(args.dim, args.inter_dim, logger=self.logger) if layer_id < args.n_dense_layers else MoE(args, logger=self.logger)
        self.attn_norm = RMSNorm(args.dim, logger=self.logger)
        self.ffn_norm = RMSNorm(args.dim, logger=self.logger)

    def forward(self, x: torch.Tensor, start_pos: int, freqs_cis: torch.Tensor, mask: Optional[torch.Tensor]) -> torch.Tensor:
        """
        Forward pass for the Transformer block.

        Args:
            x (torch.Tensor): Input tensor.
            start_pos (int): Starting position in the sequence.
            freqs_cis (torch.Tensor): Precomputed complex exponential values for rotary embeddings.
            mask (Optional[torch.Tensor]): Mask tensor to exclude certain positions from attention.

        Returns:
            torch.Tensor: Output tensor after block computation.
        """
        self.logger.debug(f"[Attention Layer {self.layer_id}]")
        self.logger.debug(f"[DEBUG] input contains NaN: {torch.isnan(x).any()}")
        fn = self.attn_norm(x)
        self.logger.debug(f"[DEBUG] attn_norm NaN: {torch.isnan(fn).any()}")
        fn = self.attn(fn, start_pos, freqs_cis, mask)
        self.logger.debug(f"[DEBUG] attn NaN: {torch.isnan(fn).any()}")
        x = x + fn
        self.logger.debug(f"[DEBUG] first_res NaN: {torch.isnan(x).any()}")
        fn2 = self.ffn_norm(x)
        self.logger.debug(f"[DEBUG] ffn_norm NaN: {torch.isnan(fn2).any()}")
        fn2 = self.ffn(fn2)
        self.logger.debug(f"[DEBUG] ffn NaN: {torch.isnan(fn2).any()}")
        x = x + fn2
        self.logger.debug(f"[DEBUG] second_res NaN: {torch.isnan(x).any()}")
        return x

#===================
# 9. 完整的DeepSeek
#===================

class Transformer(nn.Module):
    """
    Transformer model with positional embeddings, multiple layers, and output projection.

    Attributes:
        max_seq_len (int): Maximum sequence length for the transformer.
        embed (nn.Module): Embedding layer for input tokens.
        layers (torch.nn.ModuleList): List of transformer blocks.
        norm (nn.Module): Layer normalization applied after all blocks.
        head (nn.Module): Output projection layer mapping to vocabulary size.
        freqs_cis (torch.Tensor): Precomputed complex exponential values for rotary embeddings.
    """    
    def __init__(self, args: ModelArgs, logger=None):
        """
        Initializes the Transformer model.

        Args:
            args (ModelArgs): Model arguments containing transformer parameters.
        """
        self.rank = get_rank()
        self.world_size = get_world_size()
        Linear.dtype = torch.bfloat16
        #Linear.dtype = torch.float8_e4m3fn if args.dtype == "fp8" else torch.bfloat16
        super().__init__()
        self.logger = get_logger(logger)
        self.max_seq_len = args.max_seq_len
        self.embed = ParallelEmbedding(args.vocab_size, args.dim, logger=self.logger) #在MTP模块中共享
        self.layers = nn.ModuleList([Block(i, args, logger=self.logger) for i in range(args.n_layers)])
        self.norm = RMSNorm(args.dim, logger=self.logger)
        self.head = ColumnParallelLinear(args.dim, args.vocab_size, dtype=torch.get_default_dtype(), gather_output=True, logger=self.logger)
        self.logger.debug(f"[CHECK] head weight stats: max={self.head.weight.max().item()}, min={self.head.weight.min().item()}, mean={self.head.weight.mean().item()}")
        self.register_buffer("freqs_cis", precompute_freqs_cis(args), persistent=False)

        # ✅ 构建MTPlayers所所需的组件
        # 其中embedding与output层是与之前的trans共享
        # 定义预测未来token的数量
        self.n_mtp_depths = args.n_mtp_depths
        self.mtp_rms_norm1 = RMSNorm(args.dim)  # 归一化 h_i^{k-1}
        self.mtp_rms_norm2 = RMSNorm(args.dim)  # 归一化 Emb(t_{i+k})
        # 按照未来token数量确定投影层的数量
        self.mtp_projections = nn.ModuleList([
            Linear(args.dim*2, args.dim, logger=self.logger) for _ in range(self.n_mtp_depths) 
        ])
        # 按照未来token数量确定要使用的decoder layer数量
        self.mtp_layers = nn.ModuleList([
            Block(args.n_layers + i, args, logger=self.logger) for i in range(self.n_mtp_depths) 
        ])

        def register_nan_hook(module, name, logger):
            def hook(module, grad_input, grad_output):
                logger.debug(f"[HOOK ✅] {name} backward triggered")
                for i, g in enumerate(grad_output):
                    if g is not None and torch.isnan(g).any():
                        logger.error(f"🚨 NaN detected in {name} grad_output[{i}]")
        
            def forward_hook(module, input, output):
                logger.debug(f"[HOOK 🚀] {name} forward triggered")
        
            module.register_forward_hook(forward_hook)
            module.register_full_backward_hook(hook)
        
        for i, layer in enumerate(self.layers):
            register_nan_hook(layer.ffn, f"main_ffn_{i}", self.logger)
            register_nan_hook(layer.attn.wo, f"main_attn_wo_{i}", self.logger)
        
        for i, layer in enumerate(self.mtp_layers):
            register_nan_hook(layer.ffn, f"mtp_ffn_{i}", self.logger)
            register_nan_hook(layer.attn.wo, f"mtp_attn_wo_{i}", self.logger)
        
        for i, proj in enumerate(self.mtp_projections):
            register_nan_hook(proj, f"mtp_proj_{i}", self.logger)
        
        register_nan_hook(self.head, "head", self.logger)
    
    def forward(self, tokens: Optional[torch.Tensor] = None, targets: Optional[torch.Tensor] = None, start_pos: int = 0):
        """
        Forward pass for the Transformer model.

        Args:
            tokens (torch.Tensor): Input tensor of token IDs with shape (batch_size, seq_len).
            start_pos (int, optional): Starting position in the sequence for rotary embeddings. Defaults to 0.

        Returns:
            torch.Tensor: Logits tensor of shape (batch_size, vocab_size).
        """
        seqlen = tokens.size(1)
        
        self.logger.debug(f"[DEBUG] input check: max={tokens.max().item()}, min={tokens.min().item()}, mean={tokens.float().mean().item()}")
        
        #print_memory("🔹 Before embedding")
        h = self.embed(tokens)
        self.logger.debug(f"[DEBUG] embed weight stats: max={self.embed.weight.max().item()}, min={self.embed.weight.min().item()}, mean={self.embed.weight.mean().item()}")
        self.logger.debug(f"[DEBUG] logits embd_h NaN: {torch.isnan(h).any()}")
        self.logger.debug(f"[DEBUG] h after emb: max={h.max().item()}, min={h.min().item()}, mean={h.mean().item()}")
        #print_memory("🔹 After embedding")
        
        freqs_cis = self.freqs_cis[start_pos:start_pos+seqlen]
        mask = None
        if seqlen > 1:
            mask = torch.triu(torch.ones(seqlen, seqlen, dtype=torch.bool, device=tokens.device), diagonal=1)
        
        # ✅ Main model计算
        # Decoderlayers输出结果
        for i, layer in enumerate(self.layers):
            h = layer(h, start_pos, freqs_cis, mask)
            self.logger.debug(f"[DEBUG] logits layer_mask_h NaN: {torch.isnan(h).any()}")
            self.logger.debug(f"[DEBUG] h in {i} layer: max={h.max().item()}, min={h.min().item()}, mean={h.mean().item()}")
            #print_memory(f"🔹 After layer {i}")

        self.logger.debug(f"[DEBUG] h before norm: max={h.max().item()}, min={h.min().item()}, mean={h.mean().item()}")
        # 获取模型输出，结构为[bs, seq_len, D]
        h = self.norm(h)  # [B, T, D]
        self.logger.debug(f"[DEBUG] logits norm_h NaN: {torch.isnan(h).any()}")
        self.logger.debug(f"[DEBUG] h before head: max={h.max().item()}, min={h.min().item()}, mean={h.mean().item()}")
        logits = self.head(h)  # [B, T, V]
        self.logger.debug(f"[DEBUG] logits head_NaN: {torch.isnan(h).any()}")
        #print_memory("🔹 After main model")
        
        mtp_logits = []
        for k in range(self.n_mtp_depths):
            if start_pos + k + 1 >= tokens.size(1):
                break
            h_k = h[:, :-k-1]  # [B, T-k-1, D]
            self.logger.debug(f"[DEBUG] mtp_index_h NaN: {torch.isnan(h_k).any()}")
            emb_k = self.embed(tokens[:, k+1:])  # [B, T-k-1, D]
            self.logger.debug(f"[DEBUG] mtp_emb_k NaN: {torch.isnan(emb_k).any()}")
            
            norm_h = self.mtp_rms_norm1(h_k)
            self.logger.debug(f"[DEBUG] mtp_norm_h NaN: {torch.isnan(norm_h).any()}")
            norm_emb = self.mtp_rms_norm2(emb_k)
            self.logger.debug(f"[DEBUG] mtp_norm_k NaN: {torch.isnan(norm_emb).any()}")
            combined_h = torch.cat([norm_h, norm_emb], dim=-1)  # [B, T-k-1, 2D]
            self.logger.debug(f"[DEBUG] mtp_combined_h NaN: {torch.isnan(combined_h).any()}")
            
            mtp_h = self.mtp_projections[k](combined_h)
            mtp_h = torch.clamp(mtp_h, min=-20.0, max=20.0)
            self.logger.debug(f"[DEBUG] mtp_h NaN check after projection: {torch.isnan(mtp_h).any()}")
            self.logger.debug(f"[DEBUG] mtp_h before head[{k}]: max={mtp_h.max().item()}, min={mtp_h.min().item()}, mean={mtp_h.mean().item()}")
            freqs_cis_k = self.freqs_cis[:mtp_h.size(1)]
            mask_k = torch.triu(torch.ones(mtp_h.size(1), mtp_h.size(1), dtype=torch.bool, device=tokens.device), diagonal=1)
            
            mtp_h = self.mtp_layers[k](mtp_h, 0, freqs_cis_k, mask_k)
            self.logger.debug(f"[DEBUG] mtp_h NaN check after layer: {torch.isnan(mtp_h).any()}")
            #print_memory(f"🔸 After MTP layer {k} - before head")
            mtp_logit = self.head(mtp_h)  # [B, T-k-1, V]
            #print_memory(f"🔸 After MTP layer {k} - after head")
            self.logger.debug(f"[DEBUG] mtp_logits[{k}] stats: max={mtp_logit.max().item()}, min={mtp_logit.min().item()}, mean={mtp_logit.mean().item()}")
            if torch.isinf(mtp_logit).any():
                self.logger.warning(f"[Inf] ⚠️ mtp_logits[{k}] contains Inf! Saving inputs for debug...")
                torch.save({
                    "mtp_h": mtp_h.detach().cpu(),
                    "head_weight": self.head.weight.detach().cpu(),
                    "head_bias": getattr(self.head, "bias", None).detach().cpu() if hasattr(self.head, "bias") else None
                }, f"debug_mtp_inf_k{k}.pt")
                
            mtp_logits.append(mtp_logit)
        
        return logits, mtp_logits

    @torch.inference_mode()
    def generate(self, tokens: torch.Tensor, start_pos: int = 0):
        """
        Forward pass for the Transformer model.

        Args:
            tokens (torch.Tensor): Input tensor of token IDs with shape (batch_size, seq_len).
            start_pos (int, optional): Starting position in the sequence for rotary embeddings. Defaults to 0.

        Returns:
            torch.Tensor: Logits tensor of shape (batch_size, vocab_size).
        """
        seqlen = tokens.size(1)
        h = self.embed(tokens)
        freqs_cis = self.freqs_cis[start_pos:start_pos+seqlen]
        mask = None
        if seqlen > 1:
            mask = torch.full((seqlen, seqlen), float("-inf"), device=tokens.device).triu_(1)
        for layer in self.layers:
            h = layer(h, start_pos, freqs_cis, mask)
        h = self.norm(h)[:, -1]
        logits = self.head(h)
        return logits

if __name__ == "__main__":
    torch.set_default_dtype(torch.bfloat16)
    torch.set_default_device("cuda")
    torch.manual_seed(0)
    args = ModelArgs()
    x = torch.randint(0, args.vocab_size, (2, 128))
    model = Transformer(args)
    print(model(x).size())

```

### 9.4 Multi-token Prediction多步预测

- **常规的MTP vs DeepSeek的MTP**

Multi-Token Prediction (MTP)** 指的是在一次前向传播中同时预测多个未来 token，相比于传统的 **自回归 (autoregressive) 单步预测**，MTP 允许模型在每个时间步输出多个 token，这就是说，在 Transformer 解码器的 **每个时间步 t**，模型不仅预测 **t+1 处的 token**，还会同时预测 **t+2, t+3, ..., t+D** 处的 token。具体地来说，我们会在每个预测位置使用 **独立的输出头 (independent output heads)**，每个输出头给出自己预测的token，这些输出头之间没有强制的依赖关系，这些预测的 token 彼此之间没有直接联系，因此**它们可能无法互相调整**。

```
设置D个输出头，然后——
t=0  ->  预测 {t+1, t+2, ..., t+D}
t=1  ->  预测 {t+2, t+3, ..., t+D+1}
t=2  ->  预测 {t+3, t+4, ..., t+D+2}
```
DeepSeek的MTP采用的是 **顺序预测 MTP**，并且强调它 **保留了完整的因果链**。这是它们的关键创新点。具体流程如下：

> - 在 Transformer 解码器的 **每个时间步 t**，模型仍然预测多个 token：
  1. **首先预测 t+1 处的 token**（与自回归模型相同）。
  2. **然后使用携带 t+1 预测信息的数据，再预测 t+2**。
  3. **再使用携带 {t+1, t+2} 预测信息的数据，继续预测 t+3**，依此类推。

> - **所有额外预测的 token 之间存在依赖关系**，每个新预测的 token 都是基于先前的预测 **因果地** 生成的，而不是独立生成的。

```
t=0  ->  用[t-N,....t-1,t] 预测 {t+1}
t=0  ->  用 t+1 时间点收集的预测信息 加上 [t-N+1,....t,t+1] 预测 {t+2}
t=0  ->  用 {t+1, t+2} 时间点收集的预测信息、加上 [t-N+2,....t+1,t+2] 预测 {t+3}
...
```

与传统 MTP **独立预测多个 token** 不同，DeepSeek 的 MTP 使得**额外预测的 token 之间具有依赖关系**，形成一个 **完整的因果链条**。这种方式加剧了前面的预测结果对后续预测结果的影响、对损失函数提出了更高的要求。

**那具体是如何实现这种关联的呢？↓**

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/training/158.png)

- **MTP的实现**

```python

    def forward(self, tokens: Optional[torch.Tensor] = None, targets: Optional[torch.Tensor] = None, start_pos: int = 0):
        """
        Forward pass for the Transformer model.

        Args:
            tokens (torch.Tensor): Input tensor of token IDs with shape (batch_size, seq_len).
            start_pos (int, optional): Starting position in the sequence for rotary embeddings. Defaults to 0.

        Returns:
            torch.Tensor: Logits tensor of shape (batch_size, vocab_size).
        """
        seqlen = tokens.size(1)
        
        self.logger.debug(f"[DEBUG] input check: max={tokens.max().item()}, min={tokens.min().item()}, mean={tokens.float().mean().item()}")
        
        #print_memory("🔹 Before embedding")
        h = self.embed(tokens)
        
        freqs_cis = self.freqs_cis[start_pos:start_pos+seqlen]
        mask = None
        if seqlen > 1:
            mask = torch.triu(torch.ones(seqlen, seqlen, dtype=torch.bool, device=tokens.device), diagonal=1)
        
        # ✅ Main model计算
        # Decoderlayers输出结果
        for i, layer in enumerate(self.layers):
            h = layer(h, start_pos, freqs_cis, mask)

        # 获取模型输出，结构为[bs, seq_len, D]
        h = self.norm(h)  # [B, T, D]
        logits = self.head(h)  # [B, T, V]
        
        mtp_logits = []
        for k in range(self.n_mtp_depths):
            if start_pos + k + 1 >= tokens.size(1):
                break
            h_k = h[:, :-k-1]  # [B, T-k-1, D]
            emb_k = self.embed(tokens[:, k+1:])  # [B, T-k-1, D]
            
            norm_h = self.mtp_rms_norm1(h_k)
            norm_emb = self.mtp_rms_norm2(emb_k)
            combined_h = torch.cat([norm_h, norm_emb], dim=-1)  # [B, T-k-1, 2D]
            
            mtp_h = self.mtp_projections[k](combined_h)
            mtp_h = torch.clamp(mtp_h, min=-20.0, max=20.0)
            freqs_cis_k = self.freqs_cis[:mtp_h.size(1)]
            mask_k = torch.triu(torch.ones(mtp_h.size(1), mtp_h.size(1), dtype=torch.bool, device=tokens.device), diagonal=1)
            
            mtp_h = self.mtp_layers[k](mtp_h, 0, freqs_cis_k, mask_k)
            mtp_logit = self.head(mtp_h)  # [B, T-k-1, V]
            self.logger.debug(f"[DEBUG] mtp_logits[{k}] stats: max={mtp_logit.max().item()}, min={mtp_logit.min().item()}, mean={mtp_logit.mean().item()}")
            if torch.isinf(mtp_logit).any():
                self.logger.warning(f"[Inf] ⚠️ mtp_logits[{k}] contains Inf! Saving inputs for debug...")
                torch.save({
                    "mtp_h": mtp_h.detach().cpu(),
                    "head_weight": self.head.weight.detach().cpu(),
                    "head_bias": getattr(self.head, "bias", None).detach().cpu() if hasattr(self.head, "bias") else None
                }, f"debug_mtp_inf_k{k}.pt")
                
            mtp_logits.append(mtp_logit)
        
        return logits, mtp_logits

    @torch.inference_mode()
    def generate(self, tokens: torch.Tensor, start_pos: int = 0):
        """
        Forward pass for the Transformer model.

        Args:
            tokens (torch.Tensor): Input tensor of token IDs with shape (batch_size, seq_len).
            start_pos (int, optional): Starting position in the sequence for rotary embeddings. Defaults to 0.

        Returns:
            torch.Tensor: Logits tensor of shape (batch_size, vocab_size).
        """
        seqlen = tokens.size(1)
        h = self.embed(tokens)
        freqs_cis = self.freqs_cis[start_pos:start_pos+seqlen]
        mask = None
        if seqlen > 1:
            mask = torch.full((seqlen, seqlen), float("-inf"), device=tokens.device).triu_(1)
        for layer in self.layers:
            h = layer(h, start_pos, freqs_cis, mask)
        h = self.norm(h)[:, -1]
        logits = self.head(h)
        return logits
```

### 9.5 预训练经验与debug流程详谈

- 9.5.1 预训练经验做简历项目，怎样才更真实？
- 9.5.2 数据bug(1)：应该如何选择适合的tokenizer？
- 9.5.3 数据bug(2)：分词后数据大小激增如何应对？
- 9.5.4 模型bug(1)：开源推理模型用于训练都有哪些陷阱？
- 9.5.5 模型bug(2)：多机多卡真的启用了吗？如何拉满GPU效率？
- 9.5.6 模型bug(3)：落地debug工具与代码有哪些？
- 9.5.7 模型bug(4)：如何修复系统性梯度问题？
- 9.5.8 模型bug(5)：训练时间太长应该如何改善？

## 10 FSDP多机多卡分布式训练代码一览

```python
### 📦 1. 基础引入和路径设置 ###
import glob
import math
import sys
import time
from pathlib import Path
from typing import Optional, Tuple, Union
import os
import lightning as L
import torch

from lightning.fabric.strategies import FSDPStrategy  # ✅ 使用 FSDP 策略
from torch.utils.data import DataLoader
from functools import partial

# 添加工程目录到路径中，支持模块 import
wd = Path(__file__).parent.parent.resolve()
sys.path.append(str(wd))

from transformers import AutoConfig
from lit_gpt.packed_dataset import create_dataloader
from lit_gpt.speed_monitor import SpeedMonitorFabric as Monitor
from lit_gpt.speed_monitor import estimate_flops
from lit_gpt.utils import get_default_supported_precision, num_parameters, step_csv_logger
from pytorch_lightning.loggers import WandbLogger
from lit_gpt.triton_cross_entropy import TritonCrossEntropyLoss
from loguru import logger

# ✅ 加载自定义模型（例如 Steel 模型）
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
sys.path.append(os.path.join(parent_dir, "model", "steel_modify_from_qwen_1_5"))
from modeling_steel import SteelForCausalLM, SteelDecoderLayer
from steel_llm_utils import compatible_tiny_llama_config

### 📁 2. 配置与训练参数设置 ###
name = "steel_llm"
out_dir = Path("/data/gu_data/ckpt") / name
TRAIN_DATA_DIR = Path("/data/gu_data/step3_input")
MODEL_PATH = "../model/steel_modify_from_qwen_1_5"
BLOCK_SIZE = 2048
RESUME = Path("/data/gu_data/ckpt/steel_llm/step-860000-iter-6880000-ckpt")
ONLY_RESUME_MODEL = False
ADD_NEW_DATA_DIR = None
IGNORE_INDEX = 151643
USE_FLASH_ATTN = True

### ✅ 多卡训练核心参数设置
num_of_devices = 8  # ✅ 设置 GPU 数量
micro_batch_size = 8
global_batch_size = 64 * num_of_devices  # ✅ 全局 batch 是所有卡的 batch 总和
gradient_accumulation_steps = global_batch_size // (micro_batch_size * num_of_devices)  # ✅ 梯度累积

### 学习率调度参数
learning_rate = 3e-4
max_step = 430000 * 2 + 220000
warmup_steps = 1000
log_step_interval = 20
eval_step_interval = 20000
save_step_interval = 20000

### 日志与监控工具
hparams = {k: v for k, v in locals().items() if isinstance(v, (int, float, str)) and not k.startswith("_")}
logger = step_csv_logger("out", name)
wandb_logger = WandbLogger()

### 🛠️ 3. FSDP 模型训练启动逻辑 ###
def setup(devices: int = num_of_devices, ...):
    precision = precision or get_default_supported_precision(training=True)

    config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)

    # ✅ 多卡使用 FSDP 策略
    if devices > 1:
        strategy = FSDPStrategy(
            auto_wrap_policy={SteelDecoderLayer},  # ✅ 按 Transformer Block 自动 wrap
            activation_checkpointing_policy=None,
            state_dict_type="full",
            limit_all_gathers=True,
            cpu_offload=False,
        )
    else:
        strategy = "auto"

    # ✅ 使用 Lightning Fabric 构建训练环境
    fabric = L.Fabric(devices=devices, strategy=strategy, precision=precision, loggers=[logger, wandb_logger])

    config.model_path = model_path
    config.use_flash_attn = USE_FLASH_ATTN
    config = compatible_tiny_llama_config(config, block_size)

    if devices > 1:
        fabric.launch(main, train_data_dir, val_data_dir, resume, config, only_resume_model)  # ✅ 启动多进程训练
    else:
        main(fabric, train_data_dir, val_data_dir, resume, config, only_resume_model)


### 🚀 4. 主训练流程逻辑（包含 resume、数据加载、模型初始化） ###
def main(fabric, train_data_dir, val_data_dir, resume, config, only_resume_model):
    monitor = Monitor(fabric)
    if fabric.global_rank == 0:
        out_dir.mkdir(parents=True, exist_ok=True)

    # ✅ 创建多进程 dataloader
    train_dataloader, val_dataloader, train_datasets, val_datasets = create_dataloaders(...)

    fabric.seed_everything(3407)

    # ✅ 初始化模型结构
    with fabric.init_module(empty_init=False):
        model = SteelForCausalLM(config)
        model.apply(model._init_weights)

    # ✅ 参数统计与 FLOPs 估算
    fabric.print(f"Total parameters {num_parameters(model):,}")
    with torch.device("meta"):
        meta_model = SteelForCausalLM(config)
        config.estimated_flops = estimate_flops(meta_model) * micro_batch_size

    # ✅ 设置 optimizer 与 FSDP 组网
    model = fabric.setup(model)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.05)
    optimizer = fabric.setup_optimizers(optimizer)

    # ✅ 从 checkpoint 恢复模型状态
    state = {"model": model, "optimizer": optimizer, "iter_num": 0, "step_count": 0}
    if resume or only_resume_model:
        state_dir = resume / "state.pth" if resume else only_resume_model / "state.pth"
        fabric.load(state_dir, state)
        fabric.print(f"Resumed from {state_dir}")

    train(fabric, state, train_dataloader, val_dataloader, monitor, resume, config, train_datasets, model)


### 🧪 5. 训练循环核心代码（包含梯度同步与日志记录） ###
def train(fabric, state, train_dataloader, val_dataloader, monitor, resume, config, train_datasets, huggingface_format_model):
    model, optimizer = state["model"], state["optimizer"]
    loss_func = TritonCrossEntropyLoss(ignore_index=IGNORE_INDEX)

    for train_data in train_dataloader:
        if state["iter_num"] >= max_step:
            break

        input_ids = train_data[:, :config.block_size]
        targets = train_data[:, 1:config.block_size+1]

        is_accumulating = (state["iter_num"] + 1) % gradient_accumulation_steps != 0
        with fabric.no_backward_sync(model, enabled=is_accumulating):  # ✅ 支持多卡的梯度累积
            logits = model(input_ids).logits
            loss = loss_func(logits, targets)
            fabric.backward(loss / gradient_accumulation_steps)

        if not is_accumulating:
            fabric.clip_gradients(model, optimizer, max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()
            state["step_count"] += 1

        state["iter_num"] += 1
        monitor.on_train_batch_end(...)

        # ✅ 多卡保存 checkpoint，只主进程保存
        if not is_accumulating and state["step_count"] % save_step_interval == 0:
            if fabric.global_rank == 0:
                ckpt_path = out_dir / f"step-{state['step_count']:06d}-iter-{state['iter_num']:06d}-ckpt"
                os.makedirs(ckpt_path, exist_ok=True)
                fabric.save(ckpt_path / "state.pth", state)
            fabric.barrier()  # ✅ 多进程间同步


### 🔍 6. 验证函数（用于 eval loss） ###
@torch.no_grad()
def validate(fabric: L.Fabric, model: torch.nn.Module, val_dataloader: DataLoader) -> torch.Tensor:
    model.eval()
    losses = []
    for val_data in val_dataloader:
        input_ids = val_data[:, :model.config.block_size]
        targets = val_data[:, 1:model.config.block_size+1]
        logits = model(input_ids)
        loss = TritonCrossEntropyLoss()(logits, targets)
        losses.append(loss.item())
    model.train()
    return torch.tensor(losses).mean()


### 🧵 7. CLI 接口启动器 ###
if __name__ == "__main__":
    from jsonargparse import CLI
    CLI(setup)
```

## 附录

- **文本数据高频清洗算子**

| 算子类型 | 具体操作名称                          | 功能描述 |
|---------|---------------------------------|-------------------------------------------|
| **Filter** | `character_repetition_filter` | 过滤字符重复率过高的文本 |
| **Filter** | `flagged_words_filter`        | 过滤包含特定敏感词的文本 |
| **Filter** | `perplexity_filter`           | 过滤困惑度（Perplexity）过高的文本 |
| **Filter** | `special_characters_filter`   | 过滤特殊字符占比过高的文本 |
| **Filter** | `text_length_filter`          | 过滤文本长度不符合设定范围的样本 |
| **Filter** | `token_num_filter`            | 过滤标记（token）数目不符合设定范围的文本 |
| **Filter** | `word_repetition_filter`      | 过滤词重复率过高的文本 |
| **Filter** | `words_num_filter`            | 过滤单词数量不符合设定范围的文本 |
| **Deduplicator** | `document_deduplicator`           | 基于内容的文档去重，移除重复或相似的文本文档 |
| **Deduplicator** | `document_minhash_deduplicator`   | 通过 MinHash 技术检测并去除相似文档 |
| **Deduplicator** | `document_simhash_deduplicator`  | 通过 SimHash 技术检测并去除相似文档 |
| **Mapper** | `clean_html_mapper`                   | 清理文本中的 HTML 标签和格式 |
| **Mapper** | `chinese_convert_mapper`              | 进行简繁体和日文汉字转换 |
| **Mapper** | `clean_links_mapper`                  | 删除文本中的超链接 |
| **Mapper** | `fix_unicode_mapper`                  | 修正 Unicode 编码错误 |
| **Mapper** | `punctuation_normalization_mapper`    | 进行标点符号规范化 |
| **Mapper** | `remove_non_chinese_character_mapper` | 删除非中文字符 |
| **Mapper** | `remove_repeat_sentences_mapper`      | 删除重复句子 |
| **Mapper** | `remove_comments_mapper`              | 移除文本中的注释 |
| **Mapper** | `whitespace_normalization_mapper`     | 进行空白字符标准化 |

---

- **文本数据其他常见清洗算子**

| 算子类型 | 具体操作名称                          | 功能描述 |
|---------|---------------------------------|-------------------------------------------|
| **Filter** | `alphanumeric_filter`         | 过滤字母/数字比例不在特定范围内的文本 |
| **Filter** | `average_line_length_filter`  | 过滤平均行长超出范围的文本 |
| **Filter** | `language_id_score_filter`    | 过滤语言识别得分过低的文本 |
| **Filter** | `maximum_line_length_filter`  | 过滤最大行长超过设定值的文本 |
| **Filter** | `stopwords_filter`            | 过滤包含过多停用词的文本 |
| **Filter** | `text_pair_similarity_filter` | 过滤文本对相似度过低的样本 |
| **Mapper** | `remove_long_words_mapper`            | 删除超长单词 |
| **Mapper** | `remove_specific_chars_mapper`        | 移除特定字符 |
| **Mapper** | `clean_email_mapper`                  | 移除文本中的电子邮件地址 |
| **Deduplicator** | `ray_document_deduplicator`      | 基于 Ray 并行计算框架进行文档去重 |
| **Deduplicator** | `ray_bts_minhash_deduplicator`   | 结合 Ray 和 MinHash 算法进行大规模文档去重 |
| **Common** | `special_characters`         | 处理特殊字符的转换、清理和替换 |

- **相对冷门的文本清洗算子**

| 算子类型 | 具体操作名称                          | 功能描述 |
|---------|---------------------------------|-------------------------------------------|
| **Filter** | `suffix_filter`               | 过滤特定后缀的文件或文本 |
| **Filter** | `text_action_filter`          | 过滤包含特定操作或动词的文本 |
| **Filter** | `text_entity_dependency_filter` | 过滤特定实体依赖结构的文本 |
| **Filter** | `specified_field_filter`      | 过滤特定字段不符合要求的样本 |
| **Filter** | `specified_numeric_field_filter` | 过滤数值字段不符合要求的样本 |
| **Deduplicator** | `ray_basic_deduplicator`         | 使用 Ray 进行并行处理的基本去重算子 |
| **Common** | `helper_func`                | 提供通用的辅助函数，支持不同算子的计算和数据处理 |
| **Mapper** | `clean_copyright_mapper`              | 删除文本中的版权声明 |
| **Mapper** | `clean_ip_mapper`                     | 移除文本中的 IP 地址 |
| **Mapper** | `python_file_mapper`                  | 处理 Python 文件格式 |
| **Mapper** | `python_lambda_mapper`                | 处理 Python Lambda 表达式 |
| **Mapper** | `remove_bibliography_mapper`          | 移除文献引用部分 |
| **Mapper** | `remove_header_mapper`                | 删除文本中的页眉信息 |
| **Mapper** | `remove_table_text_mapper`            | 删除表格文本 |
| **Mapper** | `remove_words_with_incorrect_substrings_mapper` | 删除包含错误子串的单词 |
| **Mapper** | `replace_content_mapper`              | 替换文本内容 |

---
    
- **文本的数据增强算子**

| 算子类型 | 具体操作名称                          | 功能描述 |
|---------|---------------------------------|-------------------------------------------|
| **Common** | `prompt2prompt_pipeline`     | 处理 `Prompt-to-Prompt` 任务，实现基于文本的提示词转换 |
| **Grouper** | `key_value_grouper`      | 根据键值对对数据进行分组，适用于结构化数据处理 |
| **Grouper** | `naive_grouper`          | 采用简单的规则将数据进行分组处理 |
| **Grouper** | `naive_reverse_grouper`  | 反向分组算子，将之前分组的数据恢复为原始形式 |
| **Mapper** | `sdxl_prompt2prompt_mapper`           | SDXL 模型的 Prompt-to-Prompt 处理 |
| **Mapper** | `calibrate_qa_mapper`                 | 重新校准问答数据，提高问答质量 |
| **Mapper** | `calibrate_query_mapper`              | 校准用户查询，使其更加标准化 |
| **Mapper** | `calibrate_response_mapper`           | 调整模型生成的响应，使其更符合预期 |
| **Mapper** | `dialog_intent_detection_mapper`      | 识别对话中的意图 |
| **Mapper** | `dialog_sentiment_detection_mapper`   | 进行对话的情感分析 |
| **Mapper** | `dialog_sentiment_intensity_mapper`   | 计算对话情感强度 |
| **Mapper** | `dialog_topic_detection_mapper`       | 识别对话主题 |
| **Mapper** | `expand_macro_mapper`                 | 展开 LaTeX 文档中的宏定义 |
| **Mapper** | `extract_entity_attribute_mapper`     | 提取实体及其属性信息 |
| **Mapper** | `extract_entity_relation_mapper`      | 抽取实体关系数据 |
| **Mapper** | `extract_event_mapper`                | 从文本中提取事件信息 |
| **Mapper** | `extract_keyword_mapper`              | 提取文本中的关键词 |
| **Mapper** | `extract_nickname_mapper`             | 提取昵称或别名信息 |
| **Mapper** | `extract_support_text_mapper`         | 提取文本中支持某观点的内容 |
| **Mapper** | `generate_qa_from_examples_mapper`    | 从示例生成 QA 数据 |
| **Mapper** | `generate_qa_from_text_mapper`        | 从文本内容生成 QA 数据 |
| **Mapper** | `nlpaug_en_mapper`                    | 进行英文 NLP 数据增强 |
| **Mapper** | `nlpcda_zh_mapper`                    | 进行中文 NLP 数据增强 |
| **Mapper** | `optimize_qa_mapper`                  | 优化 QA 数据 |
| **Mapper** | `optimize_query_mapper`               | 优化查询文本 |
| **Mapper** | `optimize_response_mapper`            | 优化模型响应文本 |
| **Mapper** | `pair_preference_mapper`              | 处理成对偏好数据 |
| **Mapper** | `query_intent_detection_mapper`       | 识别查询意图 |
| **Mapper** | `query_sentiment_detection_mapper`    | 进行查询的情感分析 |
| **Mapper** | `query_topic_detection_mapper`        | 识别查询主题 |
| **Mapper** | `relation_identity_mapper`            | 识别关系身份 |
| **Mapper** | `sentence_augmentation_mapper`        | 进行句子级别的数据增强 |
| **Mapper** | `sentence_split_mapper`               | 句子拆分 |
| **Mapper** | `text_chunk_mapper`                   | 进行文本切片 |

---

- **图像数据数据清洗算子**

| 算子类型 | 具体操作名称                          | 功能描述 |
|---------|---------------------------------|-------------------------------------------|
| **Filter** | `image_aesthetics_filter`     | 过滤美学质量较低的图片 |
| **Filter** | `image_aspect_ratio_filter`   | 过滤长宽比不符合要求的图片 |
| **Filter** | `image_face_count_filter`     | 过滤人脸数量不在设定范围内的图片 |
| **Filter** | `image_face_ratio_filter`     | 过滤人脸占比过高或过低的图片 |
| **Filter** | `image_nsfw_filter`           | 过滤包含不适宜内容（NSFW）的图片 |
| **Filter** | `image_pair_similarity_filter` | 过滤两张图片相似度过高的样本 |
| **Filter** | `image_shape_filter`          | 过滤尺寸不符合要求的图片 |
| **Filter** | `image_size_filter`           | 过滤文件大小超出设定值的图片 |
| **Filter** | `image_watermark_filter`      | 过滤带有水印的图片 |
 | **Deduplicator** | `ray_image_deduplicator`         | 使用 Ray 进行大规模图片去重 |
 | **Deduplicator** | `image_deduplicator`             | 基于视觉特征对图片进行去重，移除重复图像 |
 | **Deduplicator** | `ray_basic_deduplicator`         | 使用 Ray 进行并行处理的基本去重算子 |
 | **Common** | `helper_func`                | 提供通用的辅助函数，支持不同算子的计算和数据处理 |

---

- **图像数据数据增强算子**

| 算子类型 | 具体操作名称                          | 功能描述 |
|---------|---------------------------------|-------------------------------------------|
| **Mapper** | `image_blur_mapper`                   | 处理图像模糊化 |
| **Mapper** | `image_captioning_from_gpt4v_mapper`  | 使用 GPT-4V 生成图像描述 |
| **Mapper** | `image_captioning_mapper`             | 生成图像描述 |
| **Mapper** | `image_diffusion_mapper`              | 生成图像扩散特效 |
| **Mapper** | `image_face_blur_mapper`              | 进行人脸模糊化处理 |
| **Mapper** | `image_remove_background_mapper`      | 移除图像背景 |
| **Mapper** | `image_segment_mapper`                | 进行图像分割 |
| **Mapper** | `image_tagging_mapper`                | 给图像添加标签 |

---

- **音频数据数据清洗算子**
 
| 算子类型 | 具体操作名称                          | 功能描述 |
|---------|---------------------------------|-------------------------------------------|
| **Mapper** | `audio_ffmpeg_wrapped_mapper`          | 使用 FFmpeg 处理音频，如转换格式、降噪等 |
| **Mapper** | `video_captioning_from_audio_mapper`  | 从音频生成视频字幕 |

---

- **视频数据算子**
| 算子类型 | 具体操作名称                          | 功能描述 |
|---------|---------------------------------|-------------------------------------------|
| **Filter** | `video_aesthetics_filter`     | 过滤美学质量较低的视频 |
| **Filter** | `video_aspect_ratio_filter`   | 过滤长宽比不符合要求的视频 |
| **Filter** | `video_duration_filter`       | 过滤时长不符合要求的视频 |
| **Filter** | `video_motion_score_filter`   | 过滤运动评分过低的视频 |
| **Filter** | `video_motion_score_raft_filter` | 过滤基于 RAFT 计算的运动评分过低的视频 |
| **Filter** | `video_nsfw_filter`           | 过滤包含不适宜内容（NSFW）的视频 |
| **Filter** | `video_ocr_area_ratio_filter` | 过滤 OCR 文本区域占比过高或过低的视频 |
| **Filter** | `video_resolution_filter`     | 过滤分辨率不符合要求的视频 |
| **Filter** | `video_watermark_filter`      | 过滤带有水印的视频 |
| **Deduplicator** | `ray_video_deduplicator`         | 使用 Ray 进行大规模视频去重 |
| **Deduplicator** | `video_deduplicator`             | 基于视觉特征对视频进行去重，移除重复视频 |
| **Deduplicator** | `ray_basic_deduplicator`         | 使用 Ray 进行并行处理的基本去重算子 |
| **Common** | `helper_func`                | 提供通用的辅助函数，支持不同算子的计算和数据处理 |
| **Mapper** | `video_remove_watermark_mapper`       | 去除视频水印 |
| **Mapper** | `video_resize_aspect_ratio_mapper`    | 调整视频长宽比 |
| **Mapper** | `video_resize_resolution_mapper`      | 调整视频分辨率 |
| **Mapper** | `video_split_by_duration_mapper`      | 根据时间切割视频 |
| **Mapper** | `video_split_by_key_frame_mapper`     | 根据关键帧切割视频 |
| **Mapper** | `video_split_by_scene_mapper`         | 根据场景切割视频 |
| **Mapper** | `video_extract_frames_mapper`         | 从视频中提取帧 |
| **Mapper** | `video_face_blur_mapper`              | 进行视频人脸模糊处理 |
| **Mapper** | `video_ffmpeg_wrapped_mapper`         | 使用 FFmpeg 处理视频 |

---

- **多模态数据算子**
| 算子类型 | 具体操作名称                          | 功能描述 |
|---------|---------------------------------|-------------------------------------------|
| **Mapper** | `mllm_mapper`                         | 多模态大模型数据处理 |
| **Filter** | `image_text_matching_filter`  | 过滤图像和文本匹配度过低的样本 |
| **Filter** | `image_text_similarity_filter` | 过滤图像和文本相似度过低的样本 |
| **Filter** | `phrase_grounding_recall_filter` | 过滤短语对齐召回率较低的文本 |
| **Filter** | `video_frames_text_similarity_filter` | 过滤视频帧与文本相似度过低的样本 |
| **Filter** | `video_tagging_from_frames_filter` | 过滤视频帧标签不符合要求的视频 |
| **Common** | `helper_func`                | 提供通用的辅助函数，支持不同算子的计算和数据处理 |
| **Mapper** | `video_captioning_from_frames_mapper` | 从视频帧生成字幕 |
| **Mapper** | `video_captioning_from_summarizer_mapper` | 从摘要生成视频字幕 |
| **Mapper** | `video_captioning_from_video_mapper`  | 直接从视频生成字幕 |
| **Mapper** | `video_tagging_from_audio_mapper`     | 从音频为视频生成标签 |
| **Mapper** | `video_tagging_from_frames_mapper`    | 从帧中提取视频标签 |

---

- **其他通用算子**
| 算子类型 | 具体操作名称                          | 功能描述 |
|---------|---------------------------------|-------------------------------------------|
| **Selector** | `__init__.py`  | Selector 模块的初始化文件 |
| **Selector** | `base_op.py`   | 提供 Selector 的基础操作接口 |
| **Selector** | `load.py`      | 负责加载和管理数据选择操作 |
| **Selector** | `op_fusion.py` | 进行算子融合，优化多个 Selector 操作的执行效率 |

apt update && apt install -y lsof

lsof | grep deleted

pkill -9 python